In [1]:
import os
import pandas as pd
import numpy as np
import json
import plotly.express as px
import psutil
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dask.dataframe as dd
from sklearn.linear_model import LinearRegression
from sklearn.cluster import DBSCAN
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, median_absolute_error
import lightgbm as lgb
import statsmodels.formula.api as smf
from scipy.spatial import KDTree
from prophet import Prophet
import polars as pl
import time
import optuna
import itertools
from scipy.spatial import ConvexHull
from statsmodels.tsa.seasonal import seasonal_decompose
import calendar
from scipy import stats
import warnings
import traceback
import logging
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)  
pd.set_option('display.max_colwidth', None)  

In [3]:
def print_memory_usage():
    """
    Print the current memory usage of the Python process.
    Uses psutil if available, otherwise falls back to a more basic method.
    """
    try:
        process = psutil.Process()
        memory_info = process.memory_info()
        memory_mb = memory_info.rss / (1024 * 1024)
        print(f"Current memory usage: {memory_mb:.2f} MB")
    except ImportError:
        import sys
        memory_mb = sys.getsizeof(locals()) / (1024 * 1024)
        print(f"Approximate memory usage: {memory_mb:.2f} MB (psutil not available for precise measurement)")

# Data Loading & Pre-processing

In [4]:
path_sales = "DataExercise/Sales.csv"
path_store = "DataExercise/Store.csv"
path_shopper = "DataExercise/Shopper.csv"
path_loc = "DataExercise/ShopperLoc.csv"

print("Starting Sales Forecasting Analysis...")

print("\nLoading data...")

df_sales = pd.read_csv(path_sales)
df_store = pd.read_csv(path_store)
df_shopper = pd.read_csv(path_shopper)

# For the large ShopperLoc file, we need to use Dask
df_loc = dd.read_csv(path_loc, assume_missing=True)

print(f"Loaded data: {len(df_sales)} sales records, {len(df_store)} stores, {len(df_shopper)} shoppers")

Starting Sales Forecasting Analysis...

Loading data...
Loaded data: 36550 sales records, 223 stores, 1000 shoppers


In [5]:
df_loc.head()

,ShopperID,Datetime,X,Y
0,0.0,2020-01-01 08:00:00,26405.662417,13325.522001
1,339.0,2020-01-01 08:00:00,26429.884756,13328.565292
2,339.0,2020-01-01 09:00:00,26420.868732,13311.368993
3,624.0,2020-01-01 09:00:00,21698.494509,15084.045257
4,49.0,2020-01-01 10:00:00,26184.784911,13883.214246


In [6]:
df_shopper.head(10)

,ShopperID,ShopperTypeID
0,0,3
1,9,3
2,13,3
3,18,3
4,21,3
5,28,3
6,31,3
7,32,3
8,46,3
9,53,3


In [7]:
df_store.head(10)

,StoreID,FirmID,X,Y
0,0,0,22268.476745,23085.424400
1,1,0,18918.771884,16888.112448
2,2,0,24665.643151,29919.739741
3,3,0,27525.639088,17924.360970
4,4,0,11225.471167,29824.784689
5,5,1,5964.667519,14731.828352
6,6,1,20326.104450,14524.100305
7,7,1,15570.504828,26917.750627
8,8,1,28230.371732,16770.930242
9,9,1,15376.849412,4165.452042


In [8]:
df_sales.head(10)

,FirmID,Date,Sales
0,0,2020-01-01,2.607345e+03
1,0,2020-01-02,2.256055e+03
2,0,2020-01-03,2.007728e+03
3,0,2020-01-04,3.504672e+03
4,0,2020-01-05,2.634297e+03
5,0,2020-01-06,2.575741e+03
6,0,2020-01-07,2.915166e+08
7,0,2020-01-08,2.039520e+03
8,0,2020-01-09,2.586436e+03
9,0,2020-01-10,3.025750e+03


In [9]:
df_sales.describe()

,FirmID,Sales
count,36550.000000,3.515000e+04
mean,24.500000,1.833758e+05
std,14.431067,6.907250e+06
min,0.000000,3.865365e+01
25%,12.000000,7.062059e+02
50%,24.500000,1.200922e+03
75%,37.000000,1.888080e+03
max,49.000000,6.673288e+08


In [10]:
df_store.describe()

,StoreID,FirmID,X,Y
count,223.000000,223.000000,223.000000,223.000000
mean,111.000000,24.497758,20015.849371,19588.899424
std,64.518731,14.654235,7112.613098,7110.455319
min,0.000000,0.000000,535.703259,1013.826048
25%,55.500000,12.000000,15192.605148,14701.356380
50%,111.000000,24.000000,21122.781964,20575.081152
75%,166.500000,37.000000,26300.523429,25794.551801
max,222.000000,49.000000,29973.405271,29975.464843


In [11]:
df_shopper.describe()

,ShopperID,ShopperTypeID
count,1000.000000,1000.000000
mean,499.500000,1.989000
std,288.819436,1.398869
min,0.000000,0.000000
25%,249.750000,1.000000
50%,499.500000,2.000000
75%,749.250000,3.000000
max,999.000000,4.000000


# Clean & Explore Data

In [12]:
print("\nCleaning and exploring data...")

outlier_threshold = 1e6 
mask_outlier = df_sales['Sales'] > outlier_threshold

if mask_outlier.any():
    print(f"Found {mask_outlier.sum()} outliers in Sales data")
    
    firms_with_outliers = df_sales.loc[mask_outlier, 'FirmID'].unique()
    print(f"Outliers found in {len(firms_with_outliers)} firms")
    
    # For each firm with outliers, replace with that firm's mean (excluding outliers) as we don't have a fundamental understanding of why these outliers are occurring and if they are event driven or anything along those lines given the nature of the data and given the size of the data relative to outliers, thresholding by a hrad cap shouldn't introduce any data issues
    for firm in firms_with_outliers:
        firm_mask = df_sales['FirmID'] == firm
        firm_outlier_mask = firm_mask & mask_outlier
        
        firm_mean = df_sales.loc[firm_mask & ~mask_outlier, 'Sales'].mean()
        
        if pd.isna(firm_mean):
            firm_mean = df_sales.loc[~mask_outlier, 'Sales'].median()
            print(f"  Firm {firm} has all outliers, using global median: {firm_mean:.2f}")
        else:
            print(f"  Firm {firm}: Replaced {firm_outlier_mask.sum()} outliers with firm mean: {firm_mean:.2f}")
        
        df_sales.loc[firm_outlier_mask, 'Sales'] = firm_mean

# Test Linear Imputation for outlier removal

# if not pd.api.types.is_datetime64_dtype(df_sales['Date']):
#     print("Converting Date column to datetime format...")
#     df_sales['Date'] = pd.to_datetime(df_sales['Date'])

# if df_sales['Sales'].isna().any():
#     print("Found missing values in Sales data. Performing backwards linear imputation...")
    
#     df_sales_imputed = df_sales.copy()
    
#     for firm_id in df_sales['FirmID'].unique():
#         firm_data = df_sales[df_sales['FirmID'] == firm_id].copy().sort_values('Date')
        
#         if not firm_data['Sales'].isna().any():
#             continue
            
#         if firm_data['Sales'].notna().sum() < 2:
#             print(f"  Skipping Firm {firm_id}: Not enough non-missing values for imputation")
#             continue
            
#         firm_data['DateNum'] = (firm_data['Date'] - firm_data['Date'].min()).dt.days
        
#         missing_indices = firm_data.index[firm_data['Sales'].isna()]
#         for idx in missing_indices:
#             missing_date = firm_data.loc[idx, 'Date']
#             missing_datenum = firm_data.loc[idx, 'DateNum']
            
#             future_data = firm_data[(firm_data['Date'] > missing_date) & 
#                                    (firm_data['Sales'].notna())].copy()
            
#             if len(future_data) >= 2:
                
#                 model = LinearRegression()
#                 X = future_data[['DateNum']].values
#                 y = future_data['Sales'].values
#                 model.fit(X, y)
                
#                 imputed_value = model.predict([[missing_datenum]])[0]
                
#                 imputed_value = max(0, imputed_value)
                
#                 df_sales_imputed.loc[idx, 'Sales'] = imputed_value
                
#             else:
#                 next_valid = firm_data[firm_data['Date'] > missing_date]['Sales'].first_valid_index()
#                 if next_valid is not None:
#                     df_sales_imputed.loc[idx, 'Sales'] = firm_data.loc[next_valid, 'Sales']
                    
#         print(f"  Imputed {firm_data['Sales'].isna().sum()} values for Firm {firm_id}")
    
#     df_sales = df_sales_imputed
    
#     remaining_na = df_sales['Sales'].isna().sum()
#     if remaining_na > 0:
#         print(f"Warning: {remaining_na} Sales values still missing after imputation")
#     else:
#         print("Successfully imputed all missing Sales values")

if df_sales['Sales'].isna().any():
    print("Found missing values in Sales data. Backfilling missing values for each firm...")
    df_sales['Sales'] = df_sales.groupby('FirmID')['Sales'].transform(lambda x: x.backfill())

sample_fraction = 0.1
df_loc_sample = df_loc.sample(frac=sample_fraction, random_state=42).compute()
print(f"Created sample of {len(df_loc_sample)} shopper locations for radius estimation")


Cleaning and exploring data...
Found 41 outliers in Sales data
Outliers found in 28 firms
  Firm 0: Replaced 1 outliers with firm mean: 1997.45
  Firm 1: Replaced 2 outliers with firm mean: 1824.54
  Firm 2: Replaced 2 outliers with firm mean: 1193.20
  Firm 3: Replaced 2 outliers with firm mean: 1631.60
  Firm 4: Replaced 2 outliers with firm mean: 1557.47
  Firm 5: Replaced 1 outliers with firm mean: 982.83
  Firm 6: Replaced 1 outliers with firm mean: 1002.43
  Firm 7: Replaced 1 outliers with firm mean: 924.10
  Firm 8: Replaced 2 outliers with firm mean: 874.05
  Firm 11: Replaced 1 outliers with firm mean: 677.45
  Firm 13: Replaced 1 outliers with firm mean: 1124.74
  Firm 14: Replaced 1 outliers with firm mean: 735.18
  Firm 16: Replaced 2 outliers with firm mean: 5187.06
  Firm 18: Replaced 1 outliers with firm mean: 3611.89
  Firm 20: Replaced 2 outliers with firm mean: 512.40
  Firm 23: Replaced 1 outliers with firm mean: 1776.05
  Firm 24: Replaced 1 outliers with firm mea

In [13]:
df_loc_sample.head()

,ShopperID,Datetime,X,Y
1093580,890.0,2020-04-11 17:00:00,27311.959048,7106.265198
730008,682.0,2021-09-04 14:00:00,18152.418106,18755.832178
99204,680.0,2021-07-19 13:00:00,20843.511437,19965.719211
827740,393.0,2021-08-20 15:00:00,27180.620774,17067.426915
881384,663.0,2021-04-26 14:00:00,10407.426732,21386.226674


In [14]:
df_loc_sample.describe()

,ShopperID,X,Y
count,783298.000000,783298.000000,783298.000000
mean,499.187413,19885.789462,19445.009704
std,288.404488,6594.359258,6574.795435
min,0.000000,525.310772,1001.564195
25%,250.000000,15575.173862,14815.212758
50%,499.000000,20305.875021,20013.454095
75%,749.000000,25464.963663,24717.005449
max,999.000000,29990.209250,30001.766046


# Data-Driven Store Radius Estimation

In [15]:
print("\nEstimating store radii...")

def estimate_radius_percentile(store_center, candidate_points, percentile=95):
    """
    Estimate the store's radius using the percentile method.
    
    Args:
        store_center: Tuple or array (x, y) of the store center
        candidate_points: np.array of shape (n_points, 2) containing shopper coordinates
        percentile: Percentile to use (e.g., 90 or 95)
        
    Returns:
        radius_est: The estimated radius
    """
    if len(candidate_points) == 0:
        return 15.0 
        
    distances = np.linalg.norm(candidate_points - np.array(store_center), axis=1)
    radius_est = np.percentile(distances, percentile)
    return max(radius_est, 5.0)

def estimate_radius_density(store_center, candidate_points, percentile=95, eps=10, min_samples=5):
    """
    Estimate the store's radius using a density/clustering approach.
    
    This function uses DBSCAN to cluster candidate points. It then selects
    the cluster that is most likely to represent the "core" of the store traffic
    (the largest cluster) and computes the given percentile of distances
    within that cluster.
    
    Args:
        store_center: Tuple or array (x, y) of the store center
        candidate_points: np.array of shape (n_points, 2) containing shopper coordinates
        percentile: Percentile to use (e.g., 90 or 95)
        eps: The maximum distance between two samples for DBSCAN clustering
        min_samples: The number of samples in a neighborhood for a core point in DBSCAN
    
    Returns:
        radius_est: The estimated radius based on the dense cluster
    """
    if len(candidate_points) < min_samples + 1:
        return estimate_radius_percentile(store_center, candidate_points, percentile)

    db = DBSCAN(eps=eps, min_samples=min_samples)
    labels = db.fit_predict(candidate_points)
    
    if np.all(labels == -1):
        return estimate_radius_percentile(store_center, candidate_points, percentile)
    
    unique_labels = np.unique(labels[labels != -1])
    if len(unique_labels) == 0:
        return estimate_radius_percentile(store_center, candidate_points, percentile)
        
    counts = np.array([np.sum(labels == label) for label in unique_labels])
    core_label = unique_labels[np.argmax(counts)]
    
    core_points = candidate_points[labels == core_label]
    
    distances = np.linalg.norm(core_points - np.array(store_center), axis=1)
    radius_est = np.percentile(distances, percentile)
    return max(radius_est, 5.0)

CAPTURE_RADIUS = 200.0 
store_radius_percentile = {}
store_radius_density = {}

print("Estimating store radii using percentile and density-based methods...")

for idx, store in df_store.iterrows():
    store_id = store['StoreID']
    store_center = np.array([store['X'], store['Y']])
    
    dists = np.sqrt(((df_loc_sample[['X', 'Y']].values - store_center)**2).sum(axis=1))
    candidate_points = df_loc_sample[['X', 'Y']].values[dists <= CAPTURE_RADIUS]
    
    if len(candidate_points) > 0:
        radius_p = estimate_radius_percentile(store_center, candidate_points, percentile=95)
        
        if len(candidate_points) > 100:
            eps = 5
            min_samples = 10
        else:
            eps = 10
            min_samples = 5
            
        radius_d = estimate_radius_density(
            store_center, candidate_points, percentile=95, 
            eps=eps, min_samples=min_samples
        )
    else:
        radius_p = radius_d = 15.0
    
    store_radius_percentile[store_id] = radius_p
    store_radius_density[store_id] = radius_d
    
    if idx < 5 or idx % 20 == 0:
        print(f"Store {store_id}: Percentile radius = {radius_p:.2f}, Density radius = {radius_d:.2f}")

store_radius_dict = {}
for store_id in df_store['StoreID']:
    r_p = store_radius_percentile[store_id]
    r_d = store_radius_density[store_id]
    
    if r_d < 0.7 * r_p:
        store_radius_dict[store_id] = r_d
    elif r_p < 0.7 * r_d:
        store_radius_dict[store_id] = r_p
    else:
        store_radius_dict[store_id] = (r_p + r_d) / 2

df_store['EstimatedRadius'] = df_store['StoreID'].map(store_radius_dict)

df_store['EstimatedRadius'] = df_store['EstimatedRadius'].fillna(15.0)

print(f"Completed radius estimation. Average radius: {df_store['EstimatedRadius'].mean():.2f}")


Estimating store radii...
Estimating store radii using percentile and density-based methods...
Store 0.0: Percentile radius = 8.53, Density radius = 7.65
Store 1.0: Percentile radius = 8.01, Density radius = 7.12
Store 2.0: Percentile radius = 13.15, Density radius = 13.03
Store 3.0: Percentile radius = 24.41, Density radius = 23.93
Store 4.0: Percentile radius = 24.17, Density radius = 23.81
Store 20.0: Percentile radius = 16.52, Density radius = 16.35
Store 40.0: Percentile radius = 11.73, Density radius = 10.70
Store 60.0: Percentile radius = 12.82, Density radius = 11.46
Store 80.0: Percentile radius = 99.91, Density radius = 9.43
Store 100.0: Percentile radius = 23.54, Density radius = 23.05
Store 120.0: Percentile radius = 21.77, Density radius = 21.24
Store 140.0: Percentile radius = 96.82, Density radius = 9.24
Store 160.0: Percentile radius = 20.49, Density radius = 20.29
Store 180.0: Percentile radius = 8.32, Density radius = 7.91
Store 200.0: Percentile radius = 16.66, Dens

In [16]:
def create_combined_visualization(store_df, shopper_sample_df):
    """
    Create a single plot showing all stores, their estimated radii, and shopper points.
    
    Args:
        store_df: DataFrame with store information including coordinates and radii
        shopper_sample_df: DataFrame with shopper coordinates
    
    Returns:
        Plotly figure object
    """
    fig = go.Figure()
    
    fig.add_trace(
        go.Scatter(
            x=shopper_sample_df['X'],
            y=shopper_sample_df['Y'],
            mode='markers',
            marker=dict(
                color='lightblue',
                size=3,
                opacity=0.3
            ),
            name='Shopper Locations'
        )
    )
    
    for _, store in store_df.iterrows():
        store_x, store_y = store['X'], store['Y']
        store_radius = store['EstimatedRadius']
        store_id = store['StoreID']
        firm_id = store['FirmID'] if 'FirmID' in store else None
        
        theta = np.linspace(0, 2*np.pi, 100)
        circle_x = store_radius * np.cos(theta) + store_x
        circle_y = store_radius * np.sin(theta) + store_y
        
        fig.add_trace(
            go.Scatter(
                x=circle_x,
                y=circle_y,
                mode='lines',
                line=dict(color='rgba(255, 0, 0, 0.5)', width=1),
                name=f'Store {store_id} Radius',
                legendgroup=f'store_{store_id}',
                showlegend=False,
                hoverinfo='skip'
            )
        )
        
        hover_text = f'Store ID: {store_id}'
        if firm_id is not None:
            hover_text += f'<br>Firm ID: {firm_id}'
        hover_text += f'<br>Radius: {store_radius:.2f}'
        
        fig.add_trace(
            go.Scatter(
                x=[store_x],
                y=[store_y],
                mode='markers',
                marker=dict(
                    color='red',
                    size=8,
                    symbol='star'
                ),
                name=f'Store {store_id}',
                legendgroup=f'store_{store_id}',
                showlegend=True,
                hovertext=hover_text,
                hoverinfo='text'
            )
        )
    
    try:
        store_points = store_df[['X', 'Y']].values
        if len(store_points) >= 3: 
            hull = ConvexHull(store_points)
            hull_x = store_points[hull.vertices, 0].tolist() + [store_points[hull.vertices[0], 0]]
            hull_y = store_points[hull.vertices, 1].tolist() + [store_points[hull.vertices[0], 1]]
            
            fig.add_trace(
                go.Scatter(
                    x=hull_x,
                    y=hull_y,
                    fill='toself',
                    fillcolor='rgba(255, 165, 0, 0.1)',
                    line=dict(color='orange', width=2, dash='dash'),
                    name='Store Coverage Area'
                )
            )
    except:
        pass
    
    fig.add_trace(
        go.Histogram2dContour(
            x=shopper_sample_df['X'],
            y=shopper_sample_df['Y'],
            colorscale='Blues',
            opacity=0.3,
            showscale=False,
            hoverinfo='skip',
            name='Shopper Density'
        )
    )
    
    fig.update_layout(
        title='Store Locations, Radii, and Shopper Distribution',
        xaxis_title='X Coordinate',
        yaxis_title='Y Coordinate',
        legend_title='Legend',
        height=800,
        width=1000,
        hovermode='closest',
        showlegend=True,
        yaxis=dict(
            scaleanchor="x",
            scaleratio=1
        )
    )
    
    updatemenus = [
        dict(
            type="buttons",
            direction="left",
            buttons=[
                dict(
                    args=[{"visible": [True, True, True, True]},
                          {"title": "All Elements Visible"}],
                    label="Show All",
                    method="update"
                ),
                dict(
                    args=[{"visible": [False, True, True, False]},
                          {"title": "Stores and Radii Only"}],
                    label="Stores Only",
                    method="update"
                ),
                dict(
                    args=[{"visible": [True, False, False, False]},
                          {"title": "Shoppers Only"}],
                    label="Shoppers Only",
                    method="update"
                ),
                dict(
                    args=[{"visible": [True, True, True, False]},
                          {"title": "Hide Density Heatmap"}],
                    label="Hide Heatmap",
                    method="update"
                )
            ],
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.11,
            xanchor="left",
            y=1.1,
            yanchor="top"
        )
    ]
    
    fig.update_layout(updatemenus=updatemenus)
    
    return fig

def create_firms_visualization(store_df, shopper_sample_df):
    """
    Create a visualization color-coded by firm, showing stores and their radii.
    
    Args:
        store_df: DataFrame with store information including coordinates, radii, and firm IDs
        shopper_sample_df: DataFrame with shopper coordinates
    
    Returns:
        Plotly figure object
    """
    if 'FirmID' not in store_df.columns:
        return None
    
    fig = go.Figure()
    
    fig.add_trace(
        go.Scatter(
            x=shopper_sample_df['X'],
            y=shopper_sample_df['Y'],
            mode='markers',
            marker=dict(
                color='lightgray',
                size=3,
                opacity=0.2
            ),
            name='Shopper Locations'
        )
    )
    
    firms = store_df['FirmID'].unique()
    
    colors = px.colors.qualitative.Plotly
    
    for firm_idx, firm_id in enumerate(firms):
        firm_color = colors[firm_idx % len(colors)]
        firm_stores = store_df[store_df['FirmID'] == firm_id]
        
        fig.add_trace(
            go.Scatter(
                x=firm_stores['X'],
                y=firm_stores['Y'],
                mode='markers',
                marker=dict(
                    color=firm_color,
                    size=10,
                    symbol='star',
                    line=dict(width=1, color='black')
                ),
                name=f'Firm {firm_id}',
                legendgroup=f'firm_{firm_id}',
                text=[f'Store {s}<br>Firm {f}<br>Radius: {r:.2f}' 
                      for s, f, r in zip(firm_stores['StoreID'], firm_stores['FirmID'], firm_stores['EstimatedRadius'])],
                hoverinfo='text'
            )
        )
        
        for _, store in firm_stores.iterrows():
            store_x, store_y = store['X'], store['Y']
            store_radius = store['EstimatedRadius']
            
            theta = np.linspace(0, 2*np.pi, 100)
            circle_x = store_radius * np.cos(theta) + store_x
            circle_y = store_radius * np.sin(theta) + store_y
            
            fig.add_trace(
                go.Scatter(
                    x=circle_x,
                    y=circle_y,
                    mode='lines',
                    line=dict(color=firm_color, width=1, dash='dot'),
                    name=f'Firm {firm_id} Radii',
                    legendgroup=f'firm_{firm_id}',
                    showlegend=False,
                    hoverinfo='skip'
                )
            )
        
        try:
            firm_points = firm_stores[['X', 'Y']].values
            if len(firm_points) >= 3: 
                hull = ConvexHull(firm_points)
                hull_x = firm_points[hull.vertices, 0].tolist() + [firm_points[hull.vertices[0], 0]]
                hull_y = firm_points[hull.vertices, 1].tolist() + [firm_points[hull.vertices[0], 1]]
                
                fig.add_trace(
                    go.Scatter(
                        x=hull_x,
                        y=hull_y,
                        fill='toself',
                        fillcolor=f'rgba{tuple(int(firm_color.lstrip("#")[i:i+2], 16) for i in (0, 2, 4)) + (0.1,)}',
                        line=dict(color=firm_color, width=2),
                        name=f'Firm {firm_id} Area',
                        legendgroup=f'firm_{firm_id}',
                        showlegend=True
                    )
                )
        except:
            pass
    
    fig.update_layout(
        title='Stores and Estimated Radii by Firm',
        xaxis_title='X Coordinate',
        yaxis_title='Y Coordinate',
        legend_title='Firms',
        height=800,
        width=1000,
        hovermode='closest',
        yaxis=dict(
            scaleanchor="x",
            scaleratio=1
        ),
        legend=dict(
            groupclick="toggleitem"
        )
    )
    
    return fig

combined_fig = create_combined_visualization(df_store, df_loc_sample)
firms_fig = create_firms_visualization(df_store, df_loc_sample)

combined_fig.write_html('all_stores_and_shoppers.html')
if firms_fig is not None:
    firms_fig.write_html('stores_by_firm.html')

print("Created combined visualization of all stores and shopper points.")
print("Features of the visualization:")
print("- Red stars represent store locations")
print("- Red circles show the estimated radius for each store")
print("- Light blue points show sampled shopper locations")
print("- Blue heatmap indicates areas of high shopper density")
print("- Orange dashed line shows the overall coverage area of all stores")
print("- Interactive buttons allow toggling different elements")
print("- Hover over stores to see details including Store ID and radius")
print("- Visualization saved to 'output/all_stores_and_shoppers.html'")

if firms_fig is not None:
    print("\nAlso created visualization by firm:")
    print("- Stores and their radii are color-coded by firm")
    print("- Solid colored lines show the territory covered by each firm")
    print("- Visualization saved to 'output/stores_by_firm.html'")

Created combined visualization of all stores and shopper points.
Features of the visualization:
- Red stars represent store locations
- Red circles show the estimated radius for each store
- Light blue points show sampled shopper locations
- Blue heatmap indicates areas of high shopper density
- Orange dashed line shows the overall coverage area of all stores
- Interactive buttons allow toggling different elements
- Hover over stores to see details including Store ID and radius
- Visualization saved to 'output/all_stores_and_shoppers.html'

Also created visualization by firm:
- Stores and their radii are color-coded by firm
- Solid colored lines show the territory covered by each firm
- Visualization saved to 'output/stores_by_firm.html'


# Match Customers to Stores - Chunked Processing with Polars

In [17]:
print("\nMatching shoppers to stores using full dataset...")

def get_memory_usage():
    """Return the memory usage in a human-readable format."""
    process = psutil.Process(os.getpid())
    mem = process.memory_info().rss / 1024 / 1024 
    return f"{mem:.2f} MB"

def process_shopper_chunk(shopper_chunk, store_df, tree, store_centers, store_ids, store_radii, 
                         multiple_assignment=False):
    """
    Process a chunk of shopper data to assign to stores.
    
    Args:
        shopper_chunk: Chunk of shopper data as a Polars DataFrame
        store_df: DataFrame with store information
        tree: KDTree for store locations
        store_centers: Numpy array of store coordinates
        store_ids: Array of store IDs
        store_radii: Array of store radii
        multiple_assignment: Whether to assign shoppers to multiple stores
        
    Returns:
        DataFrame with assignments
    """
    shopper_ids = shopper_chunk.get_column("ShopperID").to_numpy()
    datetimes = shopper_chunk.get_column("Datetime").to_numpy()

    x_coords = shopper_chunk.get_column("X").to_numpy()
    y_coords = shopper_chunk.get_column("Y").to_numpy()
    shopper_points = np.column_stack((x_coords, y_coords))
    
    max_radius = np.max(store_radii)
    
    assignments = []
    
    for i in range(len(shopper_points)):
        nearby_indices = tree.query_ball_point(shopper_points[i], max_radius)
        
        if len(nearby_indices) == 0:
            assignments.append({
                'ShopperID': shopper_ids[i],
                'Datetime': datetimes[i],
                'StoreID': None,
                'Distance': None
            })
            continue
            
        nearby_centers = store_centers[nearby_indices]
        distances = np.linalg.norm(shopper_points[i] - nearby_centers, axis=1)
        
        nearby_radii = store_radii[nearby_indices]
        valid_mask = distances <= nearby_radii
        
        if not np.any(valid_mask):
            assignments.append({
                'ShopperID': shopper_ids[i],
                'Datetime': datetimes[i],
                'StoreID': None,
                'Distance': None
            })
        else:
            valid_indices = np.array(nearby_indices)[valid_mask]
            valid_distances = distances[valid_mask]
            valid_store_ids = store_ids[valid_indices]
            
            if multiple_assignment:
                for j in range(len(valid_store_ids)):
                    assignments.append({
                        'ShopperID': shopper_ids[i],
                        'Datetime': datetimes[i],
                        'StoreID': valid_store_ids[j],
                        'Distance': valid_distances[j]
                    })
            else:
                min_idx = np.argmin(valid_distances)
                assignments.append({
                    'ShopperID': shopper_ids[i],
                    'Datetime': datetimes[i],
                    'StoreID': valid_store_ids[min_idx],
                    'Distance': valid_distances[min_idx]
                })
    
    return pl.DataFrame(assignments)

def assign_full_dataset(store_df, shopper_lazyframe, multiple_assignment=False, chunk_size=500000):
    """
    Process the full dataset in chunks using Polars.
    
    Args:
        store_df: Polars DataFrame with store information
        shopper_lazyframe: Polars LazyFrame for shopper data
        multiple_assignment: Whether to assign to multiple stores
        chunk_size: Size of chunks to process
        
    Returns:
        DataFrame with all assignments
    """
    start_time = time.time()
    
    store_centers = store_df.select(['X', 'Y']).to_numpy()
    store_ids = store_df.select('StoreID').to_numpy().flatten()
    store_radii = store_df.select('EstimatedRadius').to_numpy().flatten()
    
    tree = KDTree(store_centers)
    
    try:
        total_rows = shopper_lazyframe.select(pl.count()).collect().item()
        print(f"Processing approximately {total_rows:,} shopper records in chunks")
    except:
        print(f"Processing unknown number of shopper records in chunks")
        total_rows = None
    
    all_assignments = []
    processed_rows = 0
    
    chunk_size = min(chunk_size, 500000)
    print(f"Processing in smaller chunks for memory efficiency")
    
    print("Collecting data into memory - this might take some time for large datasets...")
    
    try:
        full_df = shopper_lazyframe.collect()
        print(f"Data collected, total rows: {len(full_df)}")
        
        total_rows = len(full_df)
        for i in range(0, total_rows, chunk_size):
            chunk_start_time = time.time()
            
            end_idx = min(i + chunk_size, total_rows)
            chunk = full_df.slice(i, end_idx - i)
            chunk_size_actual = len(chunk)
            processed_rows += chunk_size_actual
            
            print(f"Processing chunk {i//chunk_size + 1} with {chunk_size_actual:,} rows (total: {processed_rows:,}), memory: {get_memory_usage()}")
            
            assignments = process_shopper_chunk(
                chunk, store_df, tree, store_centers, store_ids, store_radii, 
                multiple_assignment=multiple_assignment
            )

            all_assignments.append(assignments)
            
            chunk_time = time.time() - chunk_start_time
            print(f"  Chunk {i//chunk_size + 1} processed in {chunk_time:.2f} seconds ({chunk_size_actual/chunk_time:.1f} rows/second)")
            
            if (i//chunk_size + 1) % 5 == 0 and len(all_assignments) > 1:
                intermediate_df = pl.concat(all_assignments)
                all_assignments = [intermediate_df]  
                print(f"  Saved intermediate results, memory: {get_memory_usage()}")

    except Exception as e:
        print(f"Error collecting full dataset: {e}")
        print("Switching to streaming with manual chunking...")
        
        chunk_buffer = []
        current_chunk_size = 0
        chunk_counter = 1
        
        for row in shopper_lazyframe.collect(streaming=True):
            chunk_buffer.append(row)
            current_chunk_size += 1
            
            if current_chunk_size >= chunk_size:
                chunk_start_time = time.time()
                
                chunk = pl.DataFrame(chunk_buffer)
                processed_rows += current_chunk_size
                
                print(f"Processing chunk {chunk_counter} with {current_chunk_size:,} rows (total: {processed_rows:,}), memory: {get_memory_usage()}")
                
                assignments = process_shopper_chunk(
                    chunk, store_df, tree, store_centers, store_ids, store_radii, 
                    multiple_assignment=multiple_assignment
                )

                all_assignments.append(assignments)
                
                chunk_time = time.time() - chunk_start_time
                print(f"  Chunk {chunk_counter} processed in {chunk_time:.2f} seconds ({current_chunk_size/chunk_time:.1f} rows/second)")
                
                chunk_buffer = []
                current_chunk_size = 0
                chunk_counter += 1
                
                if chunk_counter % 5 == 0 and len(all_assignments) > 1:
                    intermediate_df = pl.concat(all_assignments)
                    all_assignments = [intermediate_df]
                    print(f"  Saved intermediate results, memory: {get_memory_usage()}")
        
        if chunk_buffer:
            chunk = pl.DataFrame(chunk_buffer)
            processed_rows += len(chunk)
            
            print(f"Processing final chunk with {len(chunk):,} rows (total: {processed_rows:,})")
            
            assignments = process_shopper_chunk(
                chunk, store_df, tree, store_centers, store_ids, store_radii, 
                multiple_assignment=multiple_assignment
            )
            
            all_assignments.append(assignments)
    
    if all_assignments:
        result_df = pl.concat(all_assignments)
        total_time = time.time() - start_time
        
        print(f"\nAssignment complete!")
        print(f"Processed {processed_rows:,} shopper records in {total_time:.2f} seconds")
        print(f"Average processing speed: {processed_rows/total_time:.1f} records/second")
        
        return result_df
    else:
        print("No assignments found!")
        sample_assignment = {
            'ShopperID': None,
            'Datetime': None,
            'StoreID': None,
            'Distance': None
        }
        return pl.DataFrame([sample_assignment])

path_loc = "DataExercise/ShopperLoc.csv"

print("Converting store data to Polars format...")
store_df_pl = pl.from_pandas(df_store)

MULTIPLE_ASSIGNMENT = False
print(f"Using {'multiple' if MULTIPLE_ASSIGNMENT else 'single'} store assignment method")

print("Loading full ShopperLoc data file...")
start_time = time.time()

shopper_lf = pl.scan_csv(
    path_loc
)

print(f"Data prepared for streaming in {time.time() - start_time:.2f} seconds")

df_assign_pl = assign_full_dataset(
    store_df_pl,
    shopper_lf,
    multiple_assignment=MULTIPLE_ASSIGNMENT,
    chunk_size=500000 
)

print("Converting results to Pandas for further processing...")
df_assign = df_assign_pl.to_pandas()

total_shoppers = len(df_assign)
assigned_shoppers = df_assign['StoreID'].notna().sum()
assignment_rate = assigned_shoppers / total_shoppers * 100

print(f"\nAssignment statistics:")
print(f"  Total shopper locations: {total_shoppers:,}")
print(f"  Assigned to stores: {assigned_shoppers:,} ({assignment_rate:.2f}%)")
print(f"  Unassigned: {total_shoppers - assigned_shoppers:,} ({100 - assignment_rate:.2f}%)")

if assignment_rate < 70:
    print("Warning: Low assignment rate. Consider increasing store radii or checking for data issues.")

df_assign['Date'] = pd.to_datetime(df_assign['Datetime']).dt.date


Matching shoppers to stores using full dataset...
Converting store data to Polars format...
Using single store assignment method
Loading full ShopperLoc data file...
Data prepared for streaming in 0.01 seconds
Processing approximately 7,833,000 shopper records in chunks
Processing in smaller chunks for memory efficiency
Data collected, total rows: 7833000
Processing chunk 1 with 500,000 rows (total: 500,000), memory: 933.95 MB
  Chunk 1 processed in 20.49 seconds (24398.6 rows/second)
Processing chunk 2 with 500,000 rows (total: 1,000,000), memory: 1095.68 MB
  Chunk 2 processed in 22.56 seconds (22164.2 rows/second)
Processing chunk 3 with 500,000 rows (total: 1,500,000), memory: 1095.22 MB
  Chunk 3 processed in 16.84 seconds (29683.8 rows/second)
Processing chunk 4 with 500,000 rows (total: 2,000,000), memory: 1156.27 MB
  Chunk 4 processed in 14.42 seconds (34665.3 rows/second)
Processing chunk 5 with 500,000 rows (total: 2,500,000), memory: 931.45 MB
  Chunk 5 processed in 14.79 

# Aggregate Daily Traffic

In [18]:
print("Columns in df_assign:", df_assign.columns.tolist())
print("Sample of first few rows:")
print(df_assign.head())

print("Number of rows with StoreID:", df_assign['StoreID'].notna().sum())

if 'Date' not in df_assign.columns and 'Datetime' in df_assign.columns:
    print("Creating Date column from Datetime...")
    df_assign['Date'] = pd.to_datetime(df_assign['Datetime']).dt.date

df_daily_store_traffic = (
    df_assign.dropna(subset=['StoreID'])
    .groupby(['StoreID', 'Date'])['ShopperID']
    .nunique()
    .reset_index(name='DailyStoreTraffic')
)

print("Daily store traffic shape:", df_daily_store_traffic.shape)

df_daily_store_traffic = df_daily_store_traffic.merge(
    df_store[['StoreID', 'FirmID']], 
    on='StoreID', 
    how='left'
)

print()
print("After merge with store data shape:", df_daily_store_traffic.shape)
print("Sample after merge:")
print(df_daily_store_traffic.head())
print()

df_daily_firm_traffic = (
    df_daily_store_traffic
    .groupby(['FirmID', 'Date'])['DailyStoreTraffic']
    .sum()
    .reset_index(name='FirmDailyTraffic')
)

print("Firm daily traffic shape:", df_daily_firm_traffic.shape)
print("Sample of firm traffic:")
print(df_daily_firm_traffic.head())

Columns in df_assign: ['ShopperID', 'Datetime', 'StoreID', 'Distance', 'Date']
Sample of first few rows:
   ShopperID             Datetime  StoreID   Distance        Date
0          0  2020-01-01 08:00:00    149.0  20.733549  2020-01-01
1        339  2020-01-01 08:00:00    149.0   7.882423  2020-01-01
2        339  2020-01-01 09:00:00    149.0  11.538913  2020-01-01
3        624  2020-01-01 09:00:00      NaN        NaN  2020-01-01
4         49  2020-01-01 10:00:00      NaN        NaN  2020-01-01
Number of rows with StoreID: 5675845
Daily store traffic shape: (162980, 3)

After merge with store data shape: (162980, 4)
Sample after merge:
   StoreID        Date  DailyStoreTraffic  FirmID
0      0.0  2020-01-01                 17       0
1      0.0  2020-01-02                 12       0
2      0.0  2020-01-03                 10       0
3      0.0  2020-01-04                 13       0
4      0.0  2020-01-05                 17       0

Firm daily traffic shape: (36550, 3)
Sample of firm tr

# Merge Traffic with Sales & Build Features

In [19]:
print("\nBuilding features for modeling...")

print("df_sales dtypes:")
print(df_sales[['FirmID', 'Date']].dtypes)
print("\ndf_daily_firm_traffic dtypes:")
print(df_daily_firm_traffic[['FirmID', 'Date']].dtypes)

print("\ndf_sales sample:")
print(df_sales[['FirmID', 'Date']].head())
print("\ndf_daily_firm_traffic sample:")
print(df_daily_firm_traffic[['FirmID', 'Date']].head())

print("\nConverting Date columns to consistent datetime format...")
if not pd.api.types.is_datetime64_dtype(df_sales['Date']):
    df_sales['Date'] = pd.to_datetime(df_sales['Date'])
if not pd.api.types.is_datetime64_dtype(df_daily_firm_traffic['Date']):
    df_daily_firm_traffic['Date'] = pd.to_datetime(df_daily_firm_traffic['Date'])

print("Converting FirmID columns to consistent numeric format...")
df_sales['FirmID'] = df_sales['FirmID'].astype(float)
df_daily_firm_traffic['FirmID'] = df_daily_firm_traffic['FirmID'].astype(float)

print("\nChecking for overlapping values...")
sales_firms = set(df_sales['FirmID'].unique())
traffic_firms = set(df_daily_firm_traffic['FirmID'].unique())
common_firms = sales_firms.intersection(traffic_firms)
print(f"Firms in sales data: {len(sales_firms)}")
print(f"Firms in traffic data: {len(traffic_firms)}")
print(f"Common firms: {len(common_firms)}")

sales_dates = set(df_sales['Date'].dt.date)
traffic_dates = set(df_daily_firm_traffic['Date'].dt.date)
common_dates = sales_dates.intersection(traffic_dates)
print(f"Dates in sales data: {len(sales_dates)}")
print(f"Dates in traffic data: {len(traffic_dates)}")
print(f"Common dates: {len(common_dates)}")

print("\nPerforming merge with consistent data types...")
df_model = df_sales.merge(
    df_daily_firm_traffic,
    on=['FirmID', 'Date'],
    how='left',
    indicator=True 
)

print("\nMerge results:")
print(df_model['_merge'].value_counts())

if (df_model['_merge'] == 'left_only').any():
    print("\nSample of unmatched rows:")
    print(df_model[df_model['_merge'] == 'left_only'][['FirmID', 'Date', 'Sales']].head())

df_model = df_model.drop(columns=['_merge'])

df_model = df_model.sort_values(['FirmID', 'Date']).reset_index(drop=True)

df_model['Sales_is_missing'] = df_model['Sales'].isna()

print(f"\nFound {df_model['Sales_is_missing'].sum()} rows with missing sales values")
print(f"These rows will form our test/forecast set")

df_train = df_model[~df_model['Sales_is_missing']].copy()
df_test = df_model[df_model['Sales_is_missing']].copy()

print(f"Training set: {len(df_train)} rows")
print(f"Test set: {len(df_test)} rows")

print("\nFilling missing traffic values using only past data...")

for firm in df_train['FirmID'].unique():
    firm_mask = df_train['FirmID'] == firm
    firm_data = df_train[firm_mask].copy().sort_values('Date')
    
    for i in range(len(firm_data)):
        if pd.isna(firm_data.iloc[i]['FirmDailyTraffic']):
            past_values = firm_data.iloc[:i]['FirmDailyTraffic'].dropna()
            if len(past_values) > 0:
                firm_data.iloc[i, firm_data.columns.get_loc('FirmDailyTraffic')] = past_values.mean()
            else:
                firm_data.iloc[i, firm_data.columns.get_loc('FirmDailyTraffic')] = 0
    
    df_train.loc[firm_mask, 'FirmDailyTraffic'] = firm_data['FirmDailyTraffic'].values

for firm in df_test['FirmID'].unique():
    hist_traffic = df_train[df_train['FirmID'] == firm]['FirmDailyTraffic'].dropna()
    
    if len(hist_traffic) > 0:
        hist_mean = hist_traffic.mean()
    else:
        hist_mean = df_train['FirmDailyTraffic'].dropna().mean()
    
    test_firm_mask = df_test['FirmID'] == firm
    df_test.loc[test_firm_mask & df_test['FirmDailyTraffic'].isna(), 'FirmDailyTraffic'] = hist_mean

print("\nFeature engineering for training set...")

df_train['TrafficPctChange'] = df_train.groupby('FirmID')['FirmDailyTraffic'].pct_change().fillna(0)

df_train['DayOfWeek'] = df_train['Date'].dt.dayofweek
df_train['Month'] = df_train['Date'].dt.month
df_train['DayOfMonth'] = df_train['Date'].dt.day
df_train['IsWeekend'] = df_train['DayOfWeek'].isin([5, 6]).astype(int)

df_train['FirmDailyTraffic_lag1'] = df_train.groupby('FirmID')['FirmDailyTraffic'].shift(1).fillna(0)
df_train['FirmDailyTraffic_lag2'] = df_train.groupby('FirmID')['FirmDailyTraffic'].shift(2).fillna(0)
df_train['FirmDailyTraffic_lag7'] = df_train.groupby('FirmID')['FirmDailyTraffic'].shift(7).fillna(0)

df_train['Traffic_7day_avg'] = df_train.groupby('FirmID')['FirmDailyTraffic'].transform(
    lambda x: x.rolling(window=7, min_periods=1).mean()
)

df_train['Sales_lag1'] = df_train.groupby('FirmID')['Sales'].shift(1).fillna(0)
df_train['Sales_lag7'] = df_train.groupby('FirmID')['Sales'].shift(7).fillna(0)

df_train['Sales_7day_avg'] = df_train.groupby('FirmID')['Sales'].transform(
    lambda x: x.rolling(window=7, min_periods=1).mean()
)

df_train['Sales_7day_std'] = df_train.groupby('FirmID')['Sales'].transform(
    lambda x: x.rolling(window=7, min_periods=3).std()
).fillna(0)

df_train['SalesPctChange'] = df_train.groupby('FirmID')['Sales'].pct_change().fillna(0)

df_train['Sales_normalized'] = df_train.groupby('FirmID')['Sales'].transform(
    lambda x: (x - x.mean()) / (x.std() if x.std() != 0 else 1)
).fillna(0)

for firm in df_train['FirmID'].unique():
    firm_mask = df_train['FirmID'] == firm
    
    sales_data = df_train.loc[firm_mask, 'Sales'].values
    if len(sales_data) > 5 and np.std(sales_data) > 0: 
        try:
            scaler = RobustScaler()
            scaled_sales = scaler.fit_transform(sales_data.reshape(-1, 1)).flatten()
            df_train.loc[firm_mask, 'Sales_robust_scaled'] = scaled_sales
        except Exception as e:
            print(f"Error scaling data for firm {firm}: {e}")
            df_train.loc[firm_mask, 'Sales_robust_scaled'] = 0
    else:
        df_train.loc[firm_mask, 'Sales_robust_scaled'] = 0

print("\nFeature engineering for test set...")

df_test['DayOfWeek'] = df_test['Date'].dt.dayofweek
df_test['Month'] = df_test['Date'].dt.month
df_test['DayOfMonth'] = df_test['Date'].dt.day
df_test['IsWeekend'] = df_test['DayOfWeek'].isin([5, 6]).astype(int)

df_test = df_test.sort_values(['Date', 'FirmID']).reset_index(drop=True)

unique_test_dates = df_test['Date'].unique()
print(f"Test set contains {len(unique_test_dates)} unique dates")

df_combined = pd.concat([df_train, df_test]).sort_values(['FirmID', 'Date']).reset_index(drop=True)

df_combined['TrafficPctChange'] = df_combined.groupby('FirmID')['FirmDailyTraffic'].pct_change().fillna(0)
df_combined['FirmDailyTraffic_lag1'] = df_combined.groupby('FirmID')['FirmDailyTraffic'].shift(1).fillna(0)
df_combined['FirmDailyTraffic_lag2'] = df_combined.groupby('FirmID')['FirmDailyTraffic'].shift(2).fillna(0)
df_combined['FirmDailyTraffic_lag7'] = df_combined.groupby('FirmID')['FirmDailyTraffic'].shift(7).fillna(0)

df_combined['Traffic_7day_avg'] = df_combined.groupby('FirmID')['FirmDailyTraffic'].transform(
    lambda x: x.rolling(window=7, min_periods=1).mean()
)

df_test = df_combined[df_combined['Sales_is_missing']].copy()

for firm in df_test['FirmID'].unique():
    firm_train = df_train[df_train['FirmID'] == firm]
    
    if len(firm_train) > 0:
        last_sales = firm_train.sort_values('Date', ascending=False)['Sales'].iloc[0]
        last_date = firm_train.sort_values('Date', ascending=False)['Date'].iloc[0]
        
        if len(firm_train) >= 7:
            last_7day_avg = firm_train.sort_values('Date', ascending=False).head(7)['Sales'].mean()
            last_7day_std = firm_train.sort_values('Date', ascending=False).head(7)['Sales'].std()
        else:
            last_7day_avg = firm_train['Sales'].mean()
            last_7day_std = firm_train['Sales'].std() if len(firm_train) > 1 else 0
            
        if len(firm_train) > 1:
            last_pct_change = (last_sales / firm_train.sort_values('Date', ascending=False)['Sales'].iloc[1] - 1) if firm_train.sort_values('Date', ascending=False)['Sales'].iloc[1] != 0 else 0
        else:
            last_pct_change = 0
            
        for test_date in df_test[df_test['FirmID'] == firm]['Date'].unique():
            days_diff = (test_date - last_date).days

            if days_diff == 1:
                df_test.loc[(df_test['FirmID'] == firm) & (df_test['Date'] == test_date), 'Sales_lag1'] = last_sales
            else:
                df_test.loc[(df_test['FirmID'] == firm) & (df_test['Date'] == test_date), 'Sales_lag1'] = 0
                
            if days_diff == 7:
                df_test.loc[(df_test['FirmID'] == firm) & (df_test['Date'] == test_date), 'Sales_lag7'] = last_sales
            else:
                df_test.loc[(df_test['FirmID'] == firm) & (df_test['Date'] == test_date), 'Sales_lag7'] = 0
                
            df_test.loc[(df_test['FirmID'] == firm) & (df_test['Date'] == test_date), 'Sales_7day_avg'] = last_7day_avg
            df_test.loc[(df_test['FirmID'] == firm) & (df_test['Date'] == test_date), 'Sales_7day_std'] = last_7day_std
            df_test.loc[(df_test['FirmID'] == firm) & (df_test['Date'] == test_date), 'SalesPctChange'] = last_pct_change
            
            mean_sales = firm_train['Sales'].mean()
            std_sales = firm_train['Sales'].std() if firm_train['Sales'].std() != 0 else 1
            df_test.loc[(df_test['FirmID'] == firm) & (df_test['Date'] == test_date), 'Sales_normalized'] = (last_sales - mean_sales) / std_sales

            df_test.loc[(df_test['FirmID'] == firm) & (df_test['Date'] == test_date), 'Sales_robust_scaled'] = 0

for col in df_test.columns:
    if col not in ['Sales', 'Date', 'FirmID', 'Sales_is_missing'] and df_test[col].isna().any():
        if pd.api.types.is_numeric_dtype(df_test[col]):
            df_test[col] = df_test[col].fillna(0)
            print(f"Filled {df_test[col].isna().sum()} missing values in {col} with 0")

print("\nFinal feature set for training:")
for col in df_train.columns:
    if col not in ['Date', 'FirmID', 'Sales', 'Sales_is_missing']:
        print(f"  - {col}")

print("\nFinal feature set for testing:")
for col in df_test.columns:
    if col not in ['Date', 'FirmID', 'Sales', 'Sales_is_missing']:
        print(f"  - {col}")

print(f"\nTraining set shape: {df_train.shape}")
print(f"Test set shape: {df_test.shape}")

train_nan_cols = [col for col in df_train.columns if df_train[col].isna().any() and col != 'Sales']
if train_nan_cols:
    print(f"Warning: NaN values found in training set columns: {train_nan_cols}")
    
test_nan_cols = [col for col in df_test.columns if df_test[col].isna().any() and col != 'Sales']
if test_nan_cols:
    print(f"Warning: NaN values found in test set columns: {test_nan_cols}")

print("\nFeature engineering completed successfully")
print(f"Final dataset shape: {df_model.shape}")
print("Available features for modeling:")
for col in df_model.columns:
    if col not in ['Date', 'FirmID', 'Sales']:
        print(f"  - {col}")


Building features for modeling...
df_sales dtypes:
FirmID     int64
Date      object
dtype: object

df_daily_firm_traffic dtypes:
FirmID     int64
Date      object
dtype: object

df_sales sample:
   FirmID        Date
0       0  2020-01-01
1       0  2020-01-02
2       0  2020-01-03
3       0  2020-01-04
4       0  2020-01-05

df_daily_firm_traffic sample:
   FirmID        Date
0       0  2020-01-01
1       0  2020-01-02
2       0  2020-01-03
3       0  2020-01-04
4       0  2020-01-05

Converting Date columns to consistent datetime format...
Converting FirmID columns to consistent numeric format...

Checking for overlapping values...
Firms in sales data: 50
Firms in traffic data: 50
Common firms: 50
Dates in sales data: 731
Dates in traffic data: 731
Common dates: 731

Performing merge with consistent data types...

Merge results:
_merge
both          36550
left_only         0
right_only        0
Name: count, dtype: int64

Found 1400 rows with missing sales values
These rows will for

In [20]:
df_test.tail(30)

,FirmID,Date,Sales,FirmDailyTraffic,Sales_is_missing,TrafficPctChange,DayOfWeek,Month,DayOfMonth,IsWeekend,FirmDailyTraffic_lag1,FirmDailyTraffic_lag2,FirmDailyTraffic_lag7,Traffic_7day_avg,Sales_lag1,Sales_lag7,Sales_7day_avg,Sales_7day_std,SalesPctChange,Sales_normalized,Sales_robust_scaled
35817,48.0,2021-12-30,NaN,67.0,True,-0.082192,3,12,30,0,73.0,66.0,73.0,67.857143,0.000000,0.000000,903.689493,69.178750,-0.115128,-0.94845,0.0
35818,48.0,2021-12-31,NaN,65.0,True,-0.029851,4,12,31,0,67.0,73.0,62.0,68.285714,0.000000,0.000000,903.689493,69.178750,-0.115128,-0.94845,0.0
36522,49.0,2021-12-04,NaN,108.0,True,0.080000,5,12,4,1,100.0,118.0,99.0,112.571429,1397.808867,0.000000,1739.179452,575.572498,0.324032,-1.05055,0.0
36523,49.0,2021-12-05,NaN,111.0,True,0.027778,6,12,5,1,108.0,100.0,103.0,113.714286,0.000000,0.000000,1739.179452,575.572498,0.324032,-1.05055,0.0
36524,49.0,2021-12-06,NaN,106.0,True,-0.045045,0,12,6,0,111.0,108.0,115.0,112.428571,0.000000,0.000000,1739.179452,575.572498,0.324032,-1.05055,0.0
36525,49.0,2021-12-07,NaN,114.0,True,0.075472,1,12,7,0,106.0,111.0,120.0,111.571429,0.000000,0.000000,1739.179452,575.572498,0.324032,-1.05055,0.0
36526,49.0,2021-12-08,NaN,105.0,True,-0.078947,2,12,8,0,114.0,106.0,124.0,108.857143,0.000000,0.000000,1739.179452,575.572498,0.324032,-1.05055,0.0
36527,49.0,2021-12-09,NaN,105.0,True,0.000000,3,12,9,0,105.0,114.0,118.0,107.000000,0.000000,0.000000,1739.179452,575.572498,0.324032,-1.05055,0.0
36528,49.0,2021-12-10,NaN,106.0,True,0.009524,4,12,10,0,105.0,105.0,100.0,107.857143,0.000000,1397.808867,1739.179452,575.572498,0.324032,-1.05055,0.0
36529,49.0,2021-12-11,NaN,126.0,True,0.188679,5,12,11,1,106.0,105.0,108.0,110.428571,0.000000,0.000000,1739.179452,575.572498,0.324032,-1.05055,0.0


In [21]:
df_train.tail(30)

,FirmID,Date,Sales,FirmDailyTraffic,Sales_is_missing,TrafficPctChange,DayOfWeek,Month,DayOfMonth,IsWeekend,FirmDailyTraffic_lag1,FirmDailyTraffic_lag2,FirmDailyTraffic_lag7,Traffic_7day_avg,Sales_lag1,Sales_lag7,Sales_7day_avg,Sales_7day_std,SalesPctChange,Sales_normalized,Sales_robust_scaled
36492,49.0,2021-11-04,1047.599175,94,False,-0.168142,3,11,4,0,113.0,111.0,109.0,104.428571,1688.620569,1309.311880,1713.610822,948.212110,-0.379612,-1.443398,-0.852448
36493,49.0,2021-11-05,1831.061661,120,False,0.276596,4,11,5,0,94.0,113.0,117.0,104.857143,1047.599175,1237.982355,1798.336437,924.838769,0.747865,-0.564548,-0.261601
36494,49.0,2021-11-06,2074.301601,97,False,-0.191667,5,11,6,1,120.0,94.0,119.0,101.714286,1831.061661,3670.787213,1570.267064,472.217442,0.132841,-0.291694,-0.078162
36495,49.0,2021-11-07,2834.257179,111,False,0.144330,6,11,7,1,97.0,120.0,93.0,104.285714,2074.301601,2124.056406,1671.724317,652.797727,0.366367,0.560788,0.494956
36496,49.0,2021-11-08,1199.787564,105,False,-0.054054,0,11,8,0,111.0,97.0,84.0,107.285714,2834.257179,1157.966670,1677.698730,647.481886,-0.576684,-1.272681,-0.737675
36497,49.0,2021-11-09,1270.488626,116,False,0.104762,1,11,9,0,105.0,111.0,111.0,108.000000,1199.787564,1068.263364,1706.588053,619.672388,0.058928,-1.193372,-0.684356
36498,49.0,2021-11-10,1263.127509,106,False,-0.086207,2,11,10,0,116.0,105.0,113.0,107.000000,1270.488626,1688.620569,1645.803331,642.188206,-0.005794,-1.201629,-0.689907
36499,49.0,2021-11-11,1604.580515,110,False,0.037736,3,11,11,0,106.0,116.0,94.0,109.285714,1263.127509,1047.599175,1725.372093,587.929552,0.270323,-0.818604,-0.432401
36500,49.0,2021-11-12,930.441300,101,False,-0.081818,4,11,12,0,110.0,106.0,120.0,106.571429,1604.580515,1831.061661,1596.712042,655.596137,-0.420134,-1.574820,-0.940802
36501,49.0,2021-11-13,3828.806906,119,False,0.178218,5,11,13,1,101.0,110.0,97.0,109.714286,930.441300,2074.301601,1847.355657,1071.854927,3.115044,1.676425,1.244994


In [22]:
df_test.describe()

,FirmID,Date,Sales,FirmDailyTraffic,TrafficPctChange,DayOfWeek,Month,DayOfMonth,IsWeekend,FirmDailyTraffic_lag1,FirmDailyTraffic_lag2,FirmDailyTraffic_lag7,Traffic_7day_avg,Sales_lag1,Sales_lag7,Sales_7day_avg,Sales_7day_std,SalesPctChange,Sales_normalized,Sales_robust_scaled
count,1400.000000,1400,0.0,1400.000000,1400.000000,1400.000000,1400.0,1400.000000,1400.000000,1400.000000,1400.000000,1400.000000,1400.000000,1400.000000,1400.000000,1400.000000,1400.000000,1400.000000,1400.000000,1400.0
mean,24.500000,2021-12-17 12:00:00,NaN,84.515714,0.014253,3.000000,12.0,17.500000,0.285714,84.436429,84.468571,84.478571,84.494388,34.711545,34.711545,1083.719559,395.184396,0.122171,-0.930213,0.0
min,0.000000,2021-12-04 00:00:00,NaN,31.000000,-0.412844,0.000000,12.0,4.000000,0.000000,31.000000,31.000000,34.000000,39.142857,0.000000,0.000000,159.604587,65.761851,-0.775568,-1.841337,0.0
25%,12.000000,2021-12-10 18:00:00,NaN,65.000000,-0.090909,1.000000,12.0,10.750000,0.000000,65.000000,65.000000,65.000000,65.571429,0.000000,0.000000,618.900137,191.522829,-0.123845,-1.314320,0.0
50%,24.500000,2021-12-17 12:00:00,NaN,80.000000,0.000000,3.000000,12.0,17.500000,0.000000,80.000000,79.000000,79.000000,78.571429,0.000000,0.000000,903.942966,334.098426,-0.008975,-1.014584,0.0
75%,37.000000,2021-12-24 06:00:00,NaN,101.000000,0.111111,5.000000,12.0,24.250000,1.000000,100.000000,100.000000,100.250000,100.142857,0.000000,0.000000,1346.753125,538.285625,0.294203,-0.582764,0.0
max,49.000000,2021-12-31 00:00:00,NaN,198.000000,0.690476,6.000000,12.0,31.000000,1.000000,198.000000,198.000000,198.000000,178.285714,4547.372847,4547.372847,3902.408719,1307.906912,3.242085,0.518328,0.0
std,14.436026,NaN,NaN,26.573158,0.161959,2.000715,0.0,8.080634,0.451915,26.732524,26.761390,26.911118,25.309319,239.090808,239.090808,752.222875,276.203836,0.611977,0.525829,0.0


In [23]:
df_train.describe()

,FirmID,Date,Sales,FirmDailyTraffic,TrafficPctChange,DayOfWeek,Month,DayOfMonth,IsWeekend,FirmDailyTraffic_lag1,FirmDailyTraffic_lag2,FirmDailyTraffic_lag7,Traffic_7day_avg,Sales_lag1,Sales_lag7,Sales_7day_avg,Sales_7day_std,SalesPctChange,Sales_normalized,Sales_robust_scaled
count,35150.000000,35150,35150.000000,35150.000000,35150.000000,35150.000000,35150.000000,35150.000000,35150.000000,35150.000000,35150.000000,35150.000000,35150.000000,35150.000000,35150.000000,35150.000000,35150.000000,35150.000000,3.515000e+04,35150.000000
mean,24.500000,2020-12-17 00:00:00,1452.387267,84.580284,0.013095,3.000000,6.301565,15.668563,0.284495,84.458407,84.334708,83.736728,84.578366,1451.004730,1441.596318,1456.305806,400.772430,0.095109,-1.778884e-17,0.113955
min,0.000000,2020-01-01 00:00:00,38.653651,25.000000,-0.534483,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,34.857143,0.000000,0.000000,113.041813,0.000000,-0.872923,-2.800831e+00,-2.074553
25%,12.000000,2020-06-24 00:00:00,705.852250,65.000000,-0.100000,1.000000,3.000000,8.000000,0.000000,65.000000,65.000000,64.000000,65.285714,703.458865,692.458995,772.632402,213.581798,-0.233334,-7.617291e-01,-0.430934
50%,24.500000,2020-12-17 00:00:00,1199.841163,80.000000,0.000000,3.000000,6.000000,16.000000,0.000000,80.000000,80.000000,79.000000,79.000000,1199.116230,1192.529214,1275.431632,340.595978,0.001310,-1.565672e-01,0.000000
75%,37.000000,2021-06-11 00:00:00,1885.013224,100.000000,0.111111,5.000000,9.000000,23.000000,1.000000,100.000000,100.000000,100.000000,100.428571,1884.072940,1879.711164,1832.466989,538.647444,0.307695,6.361991e-01,0.567331
max,49.000000,2021-12-03 00:00:00,10282.143694,222.000000,1.178571,6.000000,12.000000,31.000000,1.000000,222.000000,222.000000,222.000000,186.142857,10282.143694,10282.143694,7738.411583,2224.002526,6.463783,5.187815e+00,4.410478
std,14.431075,NaN,1093.988957,27.006681,0.164840,1.996469,3.336144,8.824456,0.451180,27.168445,27.332043,28.142533,25.526295,1094.761503,1099.423886,1002.122141,261.346133,0.505268,9.993027e-01,0.721814


# Seasonality Analysis

In [24]:
if not os.path.exists('firm_seasonality_visualizations'):
    os.makedirs('firm_seasonality_visualizations')

print("\nPerforming enhanced seasonality analysis for sales and traffic...")

def analyze_seasonality(df_model, target_column, cutoff_date=None, by_firm=True):
    """
    Perform comprehensive seasonality analysis on a time series
    
    Args:
        df_model: DataFrame with time series data
        target_column: Column to analyze (e.g., 'Sales' or 'FirmDailyTraffic')
        cutoff_date: Optional date to split training/forecast periods
        by_firm: Whether to perform analysis for each firm separately
    
    Returns:
        Dictionary with analysis results
    """
    print(f"\nAnalyzing seasonality patterns for {target_column}...")
    results = {}
    
    if cutoff_date:
        print(f"Using data up to {cutoff_date} for analysis")
        analysis_data = df_model[df_model['Date'] <= cutoff_date].copy()
    else:
        analysis_data = df_model.copy()
    
    required_cols = ['FirmID', 'Date', 'DayOfWeek', 'Month', 'IsWeekend', target_column]
    missing_cols = [col for col in required_cols if col not in analysis_data.columns]
    
    if missing_cols:
        print(f"Warning: Missing required columns for analysis: {missing_cols}")
        if 'Date' in analysis_data.columns:
            if 'DayOfWeek' not in analysis_data.columns:
                analysis_data['DayOfWeek'] = analysis_data['Date'].dt.dayofweek
                print("Created 'DayOfWeek' column from Date")
            
            if 'Month' not in analysis_data.columns:
                analysis_data['Month'] = analysis_data['Date'].dt.month
                print("Created 'Month' column from Date")
            
            if 'IsWeekend' not in analysis_data.columns:
                analysis_data['IsWeekend'] = analysis_data['DayOfWeek'].isin([5, 6]).astype(int)
                print("Created 'IsWeekend' column from DayOfWeek")
    
    print("Analyzing overall daily patterns (day of week)...")
    dow_data = analysis_data.groupby('DayOfWeek')[target_column].agg(['mean', 'median', 'std', 'count'])
    dow_data['day_name'] = [calendar.day_name[day] for day in dow_data.index]
    dow_data = dow_data.sort_index() 
    
    fig_dow = go.Figure()
    
    fig_dow.add_trace(go.Bar(
        x=dow_data['day_name'], 
        y=dow_data['mean'],
        name='Mean',
        error_y=dict(
            type='data', 
            array=dow_data['std'] / np.sqrt(dow_data['count']),  
            visible=True
        )
    ))
    
    fig_dow.add_trace(go.Scatter(
        x=dow_data['day_name'],
        y=dow_data['median'],
        mode='lines+markers',
        name='Median',
        line=dict(color='red', width=2)
    ))
    
    fig_dow.update_layout(
        title=f"Overall {target_column} by Day of Week",
        xaxis_title="Day of Week",
        yaxis_title=target_column,
        template="plotly_white",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    
    fig_dow.write_html(f"overall_{target_column.lower()}_by_day_of_week.html")
    print(f"Overall day of week analysis for {target_column} saved")
    
    results['overall_day_of_week'] = dow_data
    
    print("Analyzing overall monthly patterns...")
    month_data = analysis_data.groupby('Month')[target_column].agg(['mean', 'median', 'std', 'count'])
    month_data['month_name'] = [calendar.month_name[month] for month in month_data.index]
    month_data = month_data.sort_index()

    fig_month = go.Figure()
    
    fig_month.add_trace(go.Bar(
        x=month_data['month_name'], 
        y=month_data['mean'],
        name='Mean',
        error_y=dict(
            type='data', 
            array=month_data['std'] / np.sqrt(month_data['count']), 
            visible=True
        )
    ))
    
    fig_month.add_trace(go.Scatter(
        x=month_data['month_name'],
        y=month_data['median'],
        mode='lines+markers',
        name='Median',
        line=dict(color='red', width=2)
    ))
    
    fig_month.update_layout(
        title=f"Overall {target_column} by Month",
        xaxis_title="Month",
        yaxis_title=target_column,
        template="plotly_white",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    
    fig_month.write_html(f"overall_{target_column.lower()}_by_month.html")
    print(f"Overall monthly analysis for {target_column} saved")
    
    results['overall_month'] = month_data
    
    if by_firm:
        print(f"Performing firm-level seasonality analysis for {target_column}...")
        
        firm_counts = analysis_data.groupby('FirmID').size()
        firms_to_analyze = firm_counts[firm_counts >= 30].index
        
        print(f"Found {len(firms_to_analyze)} firms with sufficient data for analysis")
        
        firm_results = {}
        
        seasonal_strengths = []
        
        for i, firm_id in enumerate(firms_to_analyze):
            print(f"Analyzing firm {firm_id} ({i+1}/{len(firms_to_analyze)})")
            
            firm_data = analysis_data[analysis_data['FirmID'] == firm_id].sort_values('Date')
            
            try:
                firm_dow = firm_data.groupby('DayOfWeek')[target_column].agg(['mean', 'median', 'std', 'count'])
                firm_dow['day_name'] = [calendar.day_name[day] for day in firm_dow.index]
                firm_dow = firm_dow.sort_index()
                
                fig_firm_dow = go.Figure()
                
                fig_firm_dow.add_trace(go.Bar(
                    x=firm_dow['day_name'], 
                    y=firm_dow['mean'],
                    name='Mean',
                    error_y=dict(
                        type='data', 
                        array=firm_dow['std'] / np.sqrt(firm_dow['count']),
                        visible=True
                    )
                ))
                
                fig_firm_dow.add_trace(go.Scatter(
                    x=firm_dow['day_name'],
                    y=firm_dow['median'],
                    mode='lines+markers',
                    name='Median',
                    line=dict(color='red', width=2)
                ))
                
                fig_firm_dow.update_layout(
                    title=f"Firm {firm_id}: {target_column} by Day of Week",
                    xaxis_title="Day of Week",
                    yaxis_title=target_column,
                    template="plotly_white",
                    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
                )
                
                fig_firm_dow.write_html(f"firm_seasonality_visualizations/firm_{firm_id}_{target_column.lower()}_by_day_of_week.html")
            except Exception as e:
                print(f"Error in day of week analysis for firm {firm_id}: {e}")
            
            try:
                num_months = firm_data['Month'].nunique()
                if num_months >= 2:
                    firm_month = firm_data.groupby('Month')[target_column].agg(['mean', 'median', 'std', 'count'])
                    firm_month['month_name'] = [calendar.month_name[month] for month in firm_month.index]
                    firm_month = firm_month.sort_index()
                    
                    fig_firm_month = go.Figure()
                    
                    fig_firm_month.add_trace(go.Bar(
                        x=firm_month['month_name'], 
                        y=firm_month['mean'],
                        name='Mean',
                        error_y=dict(
                            type='data', 
                            array=firm_month['std'] / np.sqrt(firm_month['count']),
                            visible=True
                        )
                    ))
                    
                    fig_firm_month.add_trace(go.Scatter(
                        x=firm_month['month_name'],
                        y=firm_month['median'],
                        mode='lines+markers',
                        name='Median',
                        line=dict(color='red', width=2)
                    ))
                    fig_firm_month.update_layout(
                        title=f"Firm {firm_id}: {target_column} by Month",
                        xaxis_title="Month",
                        yaxis_title=target_column,
                        template="plotly_white",
                        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
                    )
                    fig_firm_month.write_html(f"firm_seasonality_visualizations/firm_{firm_id}_{target_column.lower()}_by_month.html")
            except Exception as e:
                print(f"Error in monthly analysis for firm {firm_id}: {e}")
                
            try:
                ts_data = firm_data.set_index('Date')[target_column].resample('D').mean()
                ts_data = ts_data.interpolate(method='linear')
                
                if len(ts_data) >= 14:
                    decomposition = seasonal_decompose(ts_data, model='additive', period=7)
                    
                    fig_decomp = make_subplots(
                        rows=4, cols=1,
                        subplot_titles=["Original", "Trend", "Seasonality", "Residual"],
                        vertical_spacing=0.1,
                        shared_xaxes=True
                    )

                    fig_decomp.add_trace(
                        go.Scatter(
                            x=decomposition.observed.index, 
                            y=decomposition.observed.values,
                            mode='lines',
                            name='Original',
                            line=dict(color='blue')
                        ),
                        row=1, col=1
                    )

                    fig_decomp.add_trace(
                        go.Scatter(
                            x=decomposition.trend.index, 
                            y=decomposition.trend.values,
                            mode='lines',
                            name='Trend',
                            line=dict(color='red')
                        ),
                        row=2, col=1
                    )

                    fig_decomp.add_trace(
                        go.Scatter(
                            x=decomposition.seasonal.index, 
                            y=decomposition.seasonal.values,
                            mode='lines',
                            name='Seasonality',
                            line=dict(color='green')
                        ),
                        row=3, col=1
                    )

                    fig_decomp.add_trace(
                        go.Scatter(
                            x=decomposition.resid.index, 
                            y=decomposition.resid.values,
                            mode='lines',
                            name='Residual',
                            line=dict(color='purple')
                        ),
                        row=4, col=1
                    )

                    fig_decomp.update_layout(
                        height=800,
                        width=1000,
                        title_text=f"Firm {firm_id}: Time Series Decomposition for {target_column}",
                        template="plotly_white",
                        showlegend=True,
                        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
                    )

                    fig_decomp.write_html(f"firm_seasonality_visualizations/firm_{firm_id}_{target_column.lower()}_decomposition.html")

                    try:
                        seasonal_strength = 1 - (np.var(decomposition.resid.dropna()) / 
                                              np.var((decomposition.seasonal + decomposition.resid).dropna()))
                        seasonal_strength = max(0, seasonal_strength)
                        
                        seasonal_strengths.append({
                            'FirmID': firm_id,
                            'SeasonalStrength': seasonal_strength,
                            'DataPoints': len(firm_data),
                            'DateRange': f"{firm_data['Date'].min().date()} to {firm_data['Date'].max().date()}"
                        })
                    except Exception as e:
                        print(f"Error calculating seasonal strength for firm {firm_id}: {e}")
                    
                else:
                    print(f"Not enough data for time series decomposition for firm {firm_id}")
            except Exception as e:
                print(f"Error in time series decomposition for firm {firm_id}: {e}")

            try:
                firm_weekend = firm_data.groupby('IsWeekend')[target_column].agg(['mean', 'median', 'std', 'count'])
                if len(firm_weekend) >= 2:  # Need both weekend and weekday data
                    firm_weekend['day_type'] = ['Weekday', 'Weekend']

                    fig_weekend = go.Figure()

                    fig_weekend.add_trace(go.Bar(
                        x=firm_weekend['day_type'], 
                        y=firm_weekend['mean'],
                        name='Mean',
                        error_y=dict(
                            type='data', 
                            array=firm_weekend['std'] / np.sqrt(firm_weekend['count']),  # Standard error
                            visible=True
                        )
                    ))

                    fig_weekend.add_trace(go.Scatter(
                        x=firm_weekend['day_type'],
                        y=firm_weekend['median'],
                        mode='markers',
                        name='Median',
                        marker=dict(color='red', size=10, symbol='diamond')
                    ))

                    fig_weekend.update_layout(
                        title=f"Firm {firm_id}: {target_column} Weekend vs. Weekday Comparison",
                        xaxis_title="Day Type",
                        yaxis_title=target_column,
                        template="plotly_white",
                        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
                    )

                    fig_weekend.write_html(f"firm_seasonality_visualizations/firm_{firm_id}_{target_column.lower()}_weekend_comparison.html")
            except Exception as e:
                print(f"Error in weekend vs. weekday analysis for firm {firm_id}: {e}")

        if seasonal_strengths:
            seasonal_df = pd.DataFrame(seasonal_strengths)

            seasonal_df = seasonal_df.sort_values('SeasonalStrength', ascending=False)

            fig_seasonal = go.Figure()

            fig_seasonal.add_trace(go.Bar(
                x=seasonal_df['FirmID'],
                y=seasonal_df['SeasonalStrength'],
                text=[f"{s:.3f}" for s in seasonal_df['SeasonalStrength']],
                textposition='outside',
                marker_color='blue'
            ))

            fig_seasonal.add_shape(
                type='line',
                x0=-0.5,
                x1=len(seasonal_df)-0.5,
                y0=0.5,
                y1=0.5,
                line=dict(color='green', dash='dash', width=2)
            )
            
            fig_seasonal.add_shape(
                type='line',
                x0=-0.5,
                x1=len(seasonal_df)-0.5,
                y0=0.3,
                y1=0.3,
                line=dict(color='orange', dash='dash', width=2)
            )

            fig_seasonal.add_annotation(
                x=len(seasonal_df)-1,
                y=0.5,
                text="Strong Seasonality",
                showarrow=False,
                yshift=10,
                font=dict(color='green')
            )
            
            fig_seasonal.add_annotation(
                x=len(seasonal_df)-1,
                y=0.3,
                text="Moderate Seasonality",
                showarrow=False,
                yshift=10,
                font=dict(color='orange')
            )

            fig_seasonal.update_layout(
                title=f"{target_column} Seasonal Strength by Firm",
                xaxis_title="Firm ID",
                yaxis_title="Seasonal Strength (0-1)",
                template="plotly_white",
                yaxis=dict(range=[0, 1]),
                height=600,
                width=max(800, len(seasonal_df) * 50)
            )

            fig_seasonal.write_html(f"{target_column.lower()}_seasonal_strength_by_firm.html")
            print(f"Seasonal strength comparison by firm for {target_column} saved")

            results['firm_seasonal_strength'] = seasonal_df

        results['firm_results'] = firm_results

    print("Analyzing overall weekend vs. weekday patterns...")
    weekend_data = analysis_data.groupby('IsWeekend')[target_column].agg(['mean', 'median', 'std', 'count'])
    weekend_data['day_type'] = ['Weekday', 'Weekend']

    fig_weekend = go.Figure()

    fig_weekend.add_trace(go.Bar(
        x=weekend_data['day_type'], 
        y=weekend_data['mean'],
        name='Mean',
        error_y=dict(
            type='data', 
            array=weekend_data['std'] / np.sqrt(weekend_data['count']), 
            visible=True
        )
    ))

    fig_weekend.add_trace(go.Scatter(
        x=weekend_data['day_type'],
        y=weekend_data['median'],
        mode='markers',
        name='Median',
        marker=dict(color='red', size=10, symbol='diamond')
    ))

    fig_weekend.update_layout(
        title=f"Overall {target_column}: Weekend vs. Weekday Comparison",
        xaxis_title="Day Type",
        yaxis_title=target_column,
        template="plotly_white",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )

    fig_weekend.write_html(f"overall_{target_column.lower()}_weekend_comparison.html")
    print(f"Overall weekend vs. weekday analysis for {target_column} saved")
    
    results['overall_weekend'] = weekend_data

    print("Creating overall day of week by month heatmap...")

    heatmap_data = analysis_data.groupby(['Month', 'DayOfWeek'])[target_column].mean().reset_index()

    if len(heatmap_data) > 0:
        heatmap_pivot = heatmap_data.pivot(index='Month', columns='DayOfWeek', values=target_column)

        day_names = [calendar.day_name[i] for i in range(7)]
        month_names = [calendar.month_name[i] for i in range(1, 13)]

        fig_heatmap = go.Figure(data=go.Heatmap(
            z=heatmap_pivot.values,
            x=[day_names[i] if i in heatmap_pivot.columns else "" for i in range(7)],
            y=[month_names[i-1] if i in heatmap_pivot.index else "" for i in range(1, 13)],
            colorscale='Viridis',
            colorbar=dict(title=target_column)
        ))

        fig_heatmap.update_layout(
            title=f"Overall {target_column} by Day of Week and Month",
            xaxis_title="Day of Week",
            yaxis_title="Month",
            template="plotly_white"
        )

        fig_heatmap.write_html(f"overall_{target_column.lower()}_dow_month_heatmap.html")
        print(f"Overall day of week by month heatmap for {target_column} saved")
        
        results['overall_heatmap'] = heatmap_pivot
    else:
        print(f"Not enough data for day of week by month heatmap for {target_column}")
    
    return results

try:
    last_date = df_train['Date'].max()

    cutoff_date = last_date
    
    print(f"Using data up to {cutoff_date} for seasonality analysis")

    sales_seasonality = analyze_seasonality(df_train, 'Sales', cutoff_date)

    traffic_seasonality = analyze_seasonality(df_train, 'FirmDailyTraffic', cutoff_date)
except Exception as e:
    print(f"Error in seasonality analysis: {e}")

#-----------------------------------------------------------
# Enhanced Analysis of Relationship between Sales and Traffic 
#-----------------------------------------------------------
try:
    print("\nAnalyzing relationship between Sales and Traffic seasonality patterns...")

    dow_sales = df_train.groupby(['FirmID', 'DayOfWeek'])['Sales'].mean().reset_index()
    dow_traffic = df_train.groupby(['FirmID', 'DayOfWeek'])['FirmDailyTraffic'].mean().reset_index()

    dow_sales['DayOfWeek'] = dow_sales['DayOfWeek'].astype(int)
    dow_traffic['DayOfWeek'] = dow_traffic['DayOfWeek'].astype(int)

    dow_merged = dow_sales.merge(dow_traffic, on=['FirmID', 'DayOfWeek'], how='inner')

    correlations = []
    for day in range(7):
        day_data = dow_merged[dow_merged['DayOfWeek'] == day]

        day_data = day_data.dropna(subset=['Sales', 'FirmDailyTraffic'])
        
        if len(day_data) > 5:

            if day_data['Sales'].std() > 0 and day_data['FirmDailyTraffic'].std() > 0:
                try:
                    corr, p_value = stats.pearsonr(day_data['Sales'], day_data['FirmDailyTraffic'])
                    correlations.append({
                        'DayOfWeek': day,
                        'DayName': calendar.day_name[day],
                        'Correlation': corr,
                        'P_Value': p_value,
                        'Significant': p_value < 0.05
                    })
                    print(f"Day {day} ({calendar.day_name[day]}): Correlation = {corr:.4f}, p-value = {p_value:.4f}, n = {len(day_data)}")
                except Exception as e:
                    print(f"Error for day {day}: {e}")

    if correlations:
        corr_df = pd.DataFrame(correlations)
        
        fig_corr = go.Figure()

        fig_corr.add_trace(go.Bar(
            x=corr_df['DayName'],
            y=corr_df['Correlation'],
            marker_color=['red' if sig else 'blue' for sig in corr_df['Significant']],
            text=[f"p={p:.3f}" for p in corr_df['P_Value']],
            textposition='outside'
        ))

        fig_corr.add_shape(
            type='line',
            x0=-0.5,
            x1=len(corr_df)-0.5,
            y0=0,
            y1=0,
            line=dict(color='black', dash='dash')
        )

        fig_corr.update_layout(
            title="Sales-Traffic Correlation by Day of Week",
            xaxis_title="Day of Week",
            yaxis_title="Correlation Coefficient",
            template="plotly_white",
            yaxis=dict(range=[-1, 1]) 
        )

        fig_corr.write_html("sales_traffic_correlation_by_dow.html")
        print("Sales-Traffic correlation by day of week analysis saved")
    else:
        print("Insufficient data for correlation analysis by day of week")
    
except Exception as e:
    print(f"Error in correlation analysis by day of week: {str(e)}")
    print("Attempting to proceed with analysis...")

try:

    firm_correlations = []

    for firm_id in df_train['FirmID'].unique():
        firm_data = df_train[df_train['FirmID'] == firm_id]

        valid_data = firm_data.dropna(subset=['Sales', 'FirmDailyTraffic'])

        if len(valid_data) >= 10:

            if valid_data['Sales'].std() > 0 and valid_data['FirmDailyTraffic'].std() > 0:
                try:
                    corr, p_value = stats.pearsonr(valid_data['Sales'], valid_data['FirmDailyTraffic'])
                    
                    firm_correlations.append({
                        'FirmID': firm_id,
                        'Correlation': corr,
                        'P_Value': p_value,
                        'Significant': p_value < 0.05,
                        'DataPoints': len(valid_data)
                    })
                    print(f"Firm {firm_id}: Correlation = {corr:.4f}, p-value = {p_value:.4f}, n = {len(valid_data)}")
                except Exception as e:
                    print(f"Error calculating correlation for firm {firm_id}: {e}")
    
    if firm_correlations:
        firm_corr_df = pd.DataFrame(firm_correlations)

        firm_corr_df = firm_corr_df.sort_values('Correlation', ascending=False)

        fig_firm_corr = go.Figure()

        fig_firm_corr.add_trace(go.Bar(
            x=firm_corr_df['FirmID'],
            y=firm_corr_df['Correlation'],
            marker_color=['green' if sig else 'gray' for sig in firm_corr_df['Significant']],
            text=[f"p={p:.3f}" for p in firm_corr_df['P_Value']],
            textposition='outside'
        ))

        fig_firm_corr.add_shape(
            type='line',
            x0=-0.5,
            x1=len(firm_corr_df)-0.5,
            y0=0,
            y1=0,
            line=dict(color='black', dash='dash')
        )

        fig_firm_corr.update_layout(
            title="Sales-Traffic Correlation by Firm",
            xaxis_title="Firm ID",
            yaxis_title="Correlation Coefficient",
            template="plotly_white",
            yaxis=dict(range=[-1, 1]),  
            height=600,
            width=max(800, len(firm_corr_df) * 50)  
        )

        fig_firm_corr.write_html("sales_traffic_correlation_by_firm.html")
        print("Sales-Traffic correlation by firm analysis saved")

        print("\nSummary of Sales-Traffic correlations by firm:")
        print(f"Number of firms analyzed: {len(firm_corr_df)}")
        print(f"Firms with positive correlation: {(firm_corr_df['Correlation'] > 0).sum()} ({(firm_corr_df['Correlation'] > 0).sum() / len(firm_corr_df) * 100:.1f}%)")
        print(f"Firms with negative correlation: {(firm_corr_df['Correlation'] < 0).sum()} ({(firm_corr_df['Correlation'] < 0).sum() / len(firm_corr_df) * 100:.1f}%)")
        print(f"Firms with statistically significant correlation (p < 0.05): {firm_corr_df['Significant'].sum()} ({firm_corr_df['Significant'].sum() / len(firm_corr_df) * 100:.1f}%)")
        print(f"Average correlation: {firm_corr_df['Correlation'].mean():.3f}")
        print(f"Median correlation: {firm_corr_df['Correlation'].median():.3f}")

        firm_corr_df['CorrelationStrength'] = pd.cut(
            firm_corr_df['Correlation'].abs(), 
            bins=[0, 0.2, 0.4, 0.6, 0.8, 1.0],
            labels=['Very Weak (0-0.2)', 'Weak (0.2-0.4)', 'Moderate (0.4-0.6)', 'Strong (0.6-0.8)', 'Very Strong (0.8-1.0)']
        )

        strength_counts = firm_corr_df['CorrelationStrength'].value_counts().sort_index()

        fig_corr_pie = go.Figure(data=[go.Pie(
            labels=strength_counts.index,
            values=strength_counts.values,
            hole=.3,
            marker_colors=['#e6f2ff', '#99ccff', '#4da6ff', '#0066cc', '#003d7a']
        )])
        
        fig_corr_pie.update_layout(
            title="Distribution of Sales-Traffic Correlation Strengths Across Firms",
            template="plotly_white"
        )

        fig_corr_pie.write_html("sales_traffic_correlation_strength_distribution.html")
        print("Sales-Traffic correlation strength distribution visualization saved")
    
except Exception as e:
    print(f"Error in correlation analysis by firm: {e}")

try:
    firm_counts = df_train.groupby('FirmID').size()
    top_firms = firm_counts.nlargest(5).index

    fig_firm_dow_compare = make_subplots(
        rows=2, cols=3,
        subplot_titles=[f"Firm {firm_id}" for firm_id in top_firms] + ["Overall Average"],
        specs=[[{"type": "scatter"}, {"type": "scatter"}, {"type": "scatter"}], 
               [{"type": "scatter"}, {"type": "scatter"}, {"type": "scatter"}]],
        vertical_spacing=0.15,
        horizontal_spacing=0.08
    )

    for i, firm_id in enumerate(top_firms):
        firm_data = df_train[df_train['FirmID'] == firm_id]

        if 'DayOfWeek' not in firm_data.columns:
            if 'Date' in firm_data.columns:
                firm_data['DayOfWeek'] = firm_data['Date'].dt.dayofweek
            else:
                continue 

        valid_days = sorted(firm_data['DayOfWeek'].unique())
        if len(valid_days) == 0:
            continue  # Skip if no valid days
            
        dow_means = firm_data.groupby('DayOfWeek')['Sales'].mean()
        dow_names = [calendar.day_name[d] for d in range(7) if d in dow_means.index]
        dow_values = [dow_means[d] if d in dow_means.index else 0 for d in range(7)]

        traffic_means = firm_data.groupby('DayOfWeek')['FirmDailyTraffic'].mean()
        traffic_values = [traffic_means[d] if d in traffic_means.index else 0 for d in range(7)]

        sales_max = max(dow_values)
        if sales_max > 0:
            sales_normalized = [v/sales_max for v in dow_values]
        else:
            sales_normalized = dow_values
            
        traffic_max = max(traffic_values)
        if traffic_max > 0:
            traffic_normalized = [v/traffic_max for v in traffic_values]
        else:
            traffic_normalized = traffic_values

        row = i // 3 + 1
        col = i % 3 + 1
        
        fig_firm_dow_compare.add_trace(
            go.Scatter(
                x=dow_names,
                y=sales_normalized,
                mode='lines+markers',
                name=f'Firm {firm_id} - Sales',
                line=dict(color='blue'),
                showlegend=i==0
            ),
            row=row, col=col
        )

        fig_firm_dow_compare.add_trace(
            go.Scatter(
                x=dow_names,
                y=traffic_normalized,
                mode='lines+markers',
                name=f'Firm {firm_id} - Traffic',
                line=dict(color='red'),
                showlegend=i==0
            ),
            row=row, col=col
        )

        if len(sales_normalized) == len(traffic_normalized) and len(sales_normalized) >= 2:

            if np.std(sales_normalized) > 0 and np.std(traffic_normalized) > 0:
                try:
                    corr, pval = stats.pearsonr(sales_normalized, traffic_normalized)

                    fig_firm_dow_compare.add_annotation(
                        x=0.5, y=0.9,
                        text=f"Pattern Corr: {corr:.2f}",
                        showarrow=False,
                        xref=f"x{i+1}" if i > 0 else "x",
                        yref=f"y{i+1}" if i > 0 else "y"
                    )
                except Exception as e:
                    print(f"Error calculating correlation for firm {firm_id}: {str(e)}")

    if 'DayOfWeek' not in df_train.columns:
        if 'Date' in df_train.columns:
            df_train['DayOfWeek'] = df_train['Date'].dt.dayofweek

    if 'DayOfWeek' in df_train.columns:
        overall_sales = df_train.groupby('DayOfWeek')['Sales'].mean()
        overall_traffic = df_train.groupby('DayOfWeek')['FirmDailyTraffic'].mean()
        
        dow_names = [calendar.day_name[d] for d in range(7) if d in overall_sales.index]

        all_dow_names = [calendar.day_name[d] for d in range(7)]

        sales_max = overall_sales.max()
        if sales_max > 0:
            sales_norm = overall_sales / sales_max
        else:
            sales_norm = overall_sales
            
        traffic_max = overall_traffic.max()
        if traffic_max > 0:
            traffic_norm = overall_traffic / traffic_max
        else:
            traffic_norm = overall_traffic

        sales_values = [sales_norm[d] if d in sales_norm.index else 0 for d in range(7)]
        traffic_values = [traffic_norm[d] if d in traffic_norm.index else 0 for d in range(7)]
        
        fig_firm_dow_compare.add_trace(
            go.Scatter(
                x=all_dow_names,
                y=sales_values,
                mode='lines+markers',
                name='Overall - Sales',
                line=dict(color='blue', width=3),
                showlegend=True
            ),
            row=2, col=3
        )
        
        fig_firm_dow_compare.add_trace(
            go.Scatter(
                x=all_dow_names,
                y=traffic_values,
                mode='lines+markers',
                name='Overall - Traffic',
                line=dict(color='red', width=3),
                showlegend=True
            ),
            row=2, col=3
        )

        if len(sales_values) == len(traffic_values) and len(sales_values) >= 2:

            if np.std(sales_values) > 0 and np.std(traffic_values) > 0:
                try:
                    corr, pval = stats.pearsonr(sales_values, traffic_values)

                    fig_firm_dow_compare.add_annotation(
                        x=0.5, y=0.9,
                        text=f"Pattern Corr: {corr:.2f}",
                        showarrow=False,
                        xref="x6",
                        yref="y6"
                    )
                except Exception as e:
                    print(f"Error calculating overall correlation: {str(e)}")

    fig_firm_dow_compare.update_layout(
        height=700,
        width=1200,
        title_text="Sales vs. Traffic Day-of-Week Patterns (Normalized) by Firm",
        template="plotly_white",
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-0.2,
            xanchor="center",
            x=0.5
        )
    )

    for i in range(1, 7):
        fig_firm_dow_compare.update_yaxes(title_text="Normalized Value", range=[0, 1.1], row=(i-1)//3+1, col=(i-1)%3+1)
        fig_firm_dow_compare.update_xaxes(title_text="Day of Week", row=(i-1)//3+1, col=(i-1)%3+1)

    fig_firm_dow_compare.write_html("firm_day_of_week_pattern_comparison.html")
    print("Firm-specific day of week pattern comparison saved")
    
except Exception as e:
    print(f"Error in firm-specific day of week pattern comparison: {str(e)}")
    print("Continuing with analysis...")

try:

    fig = make_subplots(
        rows=3, cols=3,
        subplot_titles=[calendar.day_name[i] for i in range(7)] + ["All Days", ""],
        specs=[[{"type": "scatter"}, {"type": "scatter"}, {"type": "scatter"}], 
               [{"type": "scatter"}, {"type": "scatter"}, {"type": "scatter"}], 
               [{"type": "scatter"}, {"type": "scatter"}, {"type": "scatter"}]],
        vertical_spacing=0.1,
        horizontal_spacing=0.05
    )

    for i, day in enumerate(range(7)):
        day_data = df_train[df_train['DayOfWeek'] == day]

        if len(day_data) > 1000:
            day_data = day_data.sample(1000, random_state=42)

        if day_data['Sales'].std() > 0 and day_data['FirmDailyTraffic'].std() > 0 and len(day_data) > 1:
            corr, _ = stats.pearsonr(day_data['Sales'], day_data['FirmDailyTraffic'])
        else:
            corr = float('nan')

        row = i // 3 + 1
        col = i % 3 + 1

        fig.add_trace(
            go.Scatter(
                x=day_data['FirmDailyTraffic'],
                y=day_data['Sales'],
                mode='markers',
                marker=dict(opacity=0.5, size=5),
                name=calendar.day_name[day]
            ),
            row=row, col=col
        )

        fig.update_xaxes(title_text="Traffic", row=row, col=col)
        fig.update_yaxes(title_text="Sales", row=row, col=col)

        if not np.isnan(corr):
            fig.add_annotation(
                x=0.5, y=0.9,
                text=f"r = {corr:.3f}",
                showarrow=False,
                xref=f"x{i+1}" if i > 0 else "x",
                yref=f"y{i+1}" if i > 0 else "y"
            )

    all_data = df_train.sample(min(1000, len(df_train)), random_state=42)

    if all_data['Sales'].std() > 0 and all_data['FirmDailyTraffic'].std() > 0:
        corr, _ = stats.pearsonr(all_data['Sales'], all_data['FirmDailyTraffic'])
    else:
        corr = float('nan')

    fig.add_trace(
        go.Scatter(
            x=all_data['FirmDailyTraffic'],
            y=all_data['Sales'],
            mode='markers',
            marker=dict(opacity=0.5, size=5, color='red'),
            name="All Days"
        ),
        row=3, col=1
    )

    fig.update_xaxes(title_text="Traffic", row=3, col=1)
    fig.update_yaxes(title_text="Sales", row=3, col=1)

    if not np.isnan(corr):
        fig.add_annotation(
            x=0.5, y=0.9,
            text=f"r = {corr:.3f}",
            showarrow=False,
            xref="x7",
            yref="y7"
        )

    fig.update_layout(
        height=900,
        width=1000,
        title_text="Sales vs. Traffic by Day of Week",
        template="plotly_white",
        showlegend=False
    )

    fig.write_html("sales_vs_traffic_by_dow.html")
    print("Sales vs. Traffic by day of week visualization saved")
    
except Exception as e:
    print(f"Error in Sales vs. Traffic by day of week analysis: {e}")

try:
    if 'Month' not in df_train.columns:
        if 'Date' in df_train.columns:
            df_train['Month'] = df_train['Date'].dt.month
        else:
            raise ValueError("Cannot create Month column - Date column not found")

    sales_month = df_train.groupby(['FirmID', 'Month'])['Sales'].mean().reset_index()
    traffic_month = df_train.groupby(['FirmID', 'Month'])['FirmDailyTraffic'].mean().reset_index()

    sales_month['Month'] = sales_month['Month'].astype(int)
    traffic_month['Month'] = traffic_month['Month'].astype(int)
    
    month_merged = sales_month.merge(traffic_month, on=['FirmID', 'Month'], how='inner')

    month_correlations = []
    for month in range(1, 13):
        month_data = month_merged[month_merged['Month'] == month]
        if len(month_data) > 5:
            if month_data['Sales'].std() > 0 and month_data['FirmDailyTraffic'].std() > 0:
                try:
                    corr, p_value = stats.pearsonr(month_data['Sales'], month_data['FirmDailyTraffic'])
                    month_correlations.append({
                        'Month': month,
                        'MonthName': calendar.month_name[month],
                        'Correlation': corr,
                        'P_Value': p_value,
                        'Significant': p_value < 0.05
                    })
                except Exception as e:
                    print(f"Error calculating correlation for month {month}: {str(e)}")

    if month_correlations:
        month_corr_df = pd.DataFrame(month_correlations)
        
        fig_month_corr = go.Figure()

        fig_month_corr.add_trace(go.Bar(
            x=month_corr_df['MonthName'],
            y=month_corr_df['Correlation'],
            marker_color=['green' if sig else 'orange' for sig in month_corr_df['Significant']],
            text=[f"p={p:.3f}" for p in month_corr_df['P_Value']],
            textposition='outside'
        ))

        fig_month_corr.add_shape(
            type='line',
            x0=-0.5,
            x1=len(month_corr_df)-0.5,
            y0=0,
            y1=0,
            line=dict(color='black', dash='dash')
        )

        fig_month_corr.update_layout(
            title="Sales-Traffic Correlation by Month",
            xaxis_title="Month",
            yaxis_title="Correlation Coefficient",
            template="plotly_white",
            yaxis=dict(range=[-1, 1]) 
        )

        fig_month_corr.write_html("sales_traffic_correlation_by_month.html")
        print("Sales-Traffic correlation by month analysis saved")
    else:
        print("Insufficient data for correlation analysis by month")
    
except Exception as e:
    print(f"Error in correlation analysis by month: {str(e)}")
    print("Continuing with analysis...")

try:

    if 'DayOfWeek' not in df_train.columns:
        if 'Date' in df_train.columns:
            df_train['DayOfWeek'] = df_train['Date'].dt.dayofweek
        else:
            raise ValueError("Cannot create DayOfWeek column - Date column not found")

    sales_dow = df_train.groupby('DayOfWeek')['Sales'].mean()

    sales_max = sales_dow.max()
    if sales_max > 0:
        sales_dow_norm = sales_dow / sales_max
    else:
        sales_dow_norm = sales_dow
    
    traffic_dow = df_train.groupby('DayOfWeek')['FirmDailyTraffic'].mean()

    traffic_max = traffic_dow.max()
    if traffic_max > 0:
        traffic_dow_norm = traffic_dow / traffic_max
    else:
        traffic_dow_norm = traffic_dow

    compare_dow = pd.DataFrame({
        'Sales': sales_dow_norm,
        'Traffic': traffic_dow_norm
    })
    compare_dow['DayName'] = [calendar.day_name[i] if i in compare_dow.index else "Unknown" for i in compare_dow.index]

    if not compare_dow.empty:

        fig = make_subplots(
            rows=1, cols=3,
            subplot_titles=["Sales by Day", "Traffic by Day", "Comparison"],
            specs=[[{"type": "bar"}, {"type": "bar"}, {"type": "scatter"}]],
            horizontal_spacing=0.1
        )

        fig.add_trace(
            go.Bar(
                x=compare_dow['DayName'],
                y=compare_dow['Sales'],
                name='Sales',
                marker_color='blue'
            ),
            row=1, col=1
        )

        fig.add_trace(
            go.Bar(
                x=compare_dow['DayName'],
                y=compare_dow['Traffic'],
                name='Traffic',
                marker_color='red'
            ),
            row=1, col=2
        )

        fig.add_trace(
            go.Scatter(
                x=compare_dow['DayName'],
                y=compare_dow['Sales'],
                mode='lines+markers',
                name='Sales',
                line=dict(color='blue')
            ),
            row=1, col=3
        )
        
        fig.add_trace(
            go.Scatter(
                x=compare_dow['DayName'],
                y=compare_dow['Traffic'],
                mode='lines+markers',
                name='Traffic',
                line=dict(color='red')
            ),
            row=1, col=3
        )

        fig.update_layout(
            height=500,
            width=1200,
            title_text="Sales and Traffic Patterns Comparison",
            template="plotly_white"
        )

        fig.write_html("sales_traffic_pattern_comparison.html")
        print("Sales and Traffic pattern comparison saved")
    else:
        print("Insufficient data for Sales and Traffic pattern comparison")
    
except Exception as e:
    print(f"Error in Sales and Traffic pattern comparison: {str(e)}")
    print("Continuing with analysis...")

try:
    print("\nPerforming cluster analysis of seasonality patterns...")

    if 'DayOfWeek' not in df_train.columns:
        if 'Date' in df_train.columns:
            df_train['DayOfWeek'] = df_train['Date'].dt.dayofweek
            print("Created 'DayOfWeek' column from Date for clustering analysis")
        else:
            raise ValueError("Cannot create DayOfWeek column - Date column not found")

    firm_counts = df_train.groupby('FirmID').size()
    firms_with_data = firm_counts[firm_counts >= 30].index
    
    if len(firms_with_data) >= 5:
        pattern_data = []
        
        for firm_id in firms_with_data:
            firm_data = df_train[df_train['FirmID'] == firm_id]

            if 'DayOfWeek' not in firm_data.columns or 'Sales' not in firm_data.columns:
                continue

            days_present = sorted(firm_data['DayOfWeek'].unique())

            if len(days_present) >= 5:
                try:
                    sales_dow = firm_data.groupby('DayOfWeek')['Sales'].mean()

                    full_pattern = np.zeros(7)
                    for day, value in sales_dow.items():
                        if 0 <= day < 7: 
                            full_pattern[day] = value

                    pattern_max = np.max(full_pattern)
                    if pattern_max > 0:
                        normalized_pattern = full_pattern / pattern_max
                    else:
                        normalized_pattern = full_pattern
                    
                    pattern_data.append({
                        'FirmID': firm_id,
                        'Pattern': normalized_pattern,
                        'DataPoints': len(firm_data)
                    })
                except Exception as e:
                    print(f"Error processing pattern for firm {firm_id}: {str(e)}")

        pattern_df = pd.DataFrame(pattern_data)
        
        if len(pattern_df) >= 5:
            pattern_matrix = np.vstack(pattern_df['Pattern'].values)

            from sklearn.cluster import KMeans
            from sklearn.metrics import silhouette_score

            max_clusters = min(10, len(pattern_df) // 2) 
            scores = []
            
            for n_clusters in range(2, max_clusters + 1):
                try:
                    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
                    cluster_labels = kmeans.fit_predict(pattern_matrix)
                    
                    if len(set(cluster_labels)) > 1:  # Need at least 2 clusters for silhouette score
                        silhouette_avg = silhouette_score(pattern_matrix, cluster_labels)
                        scores.append((n_clusters, silhouette_avg))
                except Exception as e:
                    print(f"Error computing silhouette score for {n_clusters} clusters: {str(e)}")
            
            if scores:
                best_n_clusters, _ = max(scores, key=lambda x: x[1])

                kmeans = KMeans(n_clusters=best_n_clusters, random_state=42, n_init=10)
                cluster_labels = kmeans.fit_predict(pattern_matrix)

                pattern_df['Cluster'] = cluster_labels

                cluster_centers = kmeans.cluster_centers_

                fig_clusters = go.Figure()

                for i, center in enumerate(cluster_centers):
                    fig_clusters.add_trace(go.Scatter(
                        x=[calendar.day_name[d] for d in range(7)],
                        y=center,
                        mode='lines+markers',
                        name=f'Cluster {i+1}',
                        line=dict(width=3)
                    ))

                fig_clusters.update_layout(
                    title=f"Sales Seasonality Pattern Clusters (n={best_n_clusters})",
                    xaxis_title="Day of Week",
                    yaxis_title="Normalized Sales",
                    template="plotly_white",
                    legend_title="Cluster"
                )

                fig_clusters.write_html("sales_seasonality_clusters.html")
                print(f"Sales seasonality cluster analysis saved with {best_n_clusters} clusters")

                cluster_counts = pattern_df['Cluster'].value_counts().sort_index()
                
                fig_cluster_counts = go.Figure(data=[go.Pie(
                    labels=[f'Cluster {i+1} (n={count})' for i, count in enumerate(cluster_counts)],
                    values=cluster_counts.values
                )])
                
                fig_cluster_counts.update_layout(
                    title="Distribution of Firms Across Seasonality Pattern Clusters",
                    template="plotly_white"
                )
                
                fig_cluster_counts.write_html("sales_seasonality_cluster_distribution.html")
                print("Sales seasonality cluster distribution visualization saved")

                print(f"\nSales seasonality pattern clustering summary:")
                print(f"Number of firms analyzed: {len(pattern_df)}")
                print(f"Optimal number of clusters: {best_n_clusters}")
                print("Cluster distribution:")
                for i, count in enumerate(cluster_counts):
                    print(f"  Cluster {i+1}: {count} firms ({count/len(pattern_df)*100:.1f}%)")

                try:
                    fig_cluster_examples = make_subplots(
                        rows=best_n_clusters, cols=1,
                        subplot_titles=[f"Cluster {i+1} Examples" for i in range(best_n_clusters)],
                        vertical_spacing=0.1
                    )

                    for cluster_idx in range(best_n_clusters):
                        cluster_firms = pattern_df[pattern_df['Cluster'] == cluster_idx]['FirmID'].values

                        sample_firms = cluster_firms[:min(3, len(cluster_firms))]

                        fig_cluster_examples.add_trace(
                            go.Scatter(
                                x=[calendar.day_name[d] for d in range(7)],
                                y=cluster_centers[cluster_idx],
                                mode='lines',
                                name=f'Cluster {cluster_idx+1} Center',
                                line=dict(color='black', width=3, dash='dash')
                            ),
                            row=cluster_idx+1, col=1
                        )

                        for i, firm_id in enumerate(sample_firms):
                            firm_data = df_train[df_train['FirmID'] == firm_id]

                            dow_means = firm_data.groupby('DayOfWeek')['Sales'].mean()
                            dow_values = [dow_means[d] if d in dow_means.index else 0 for d in range(7)]

                            if max(dow_values) > 0:
                                normalized = [v / max(dow_values) for v in dow_values]
                            else:
                                normalized = dow_values

                            fig_cluster_examples.add_trace(
                                go.Scatter(
                                    x=[calendar.day_name[d] for d in range(7)],
                                    y=normalized,
                                    mode='lines+markers',
                                    name=f'Firm {firm_id}',
                                    line=dict(width=2),
                                    opacity=0.7
                                ),
                                row=cluster_idx+1, col=1
                            )

                    fig_cluster_examples.update_layout(
                        height=300 * best_n_clusters,
                        width=900,
                        title_text="Example Firms from Each Seasonality Pattern Cluster",
                        template="plotly_white",
                        showlegend=True
                    )

                    for i in range(1, best_n_clusters+1):
                        fig_cluster_examples.update_yaxes(
                            title_text="Normalized Sales",
                            range=[0, 1.1],
                            row=i, col=1
                        )
                        fig_cluster_examples.update_xaxes(
                            title_text="Day of Week" if i == best_n_clusters else "",
                            row=i, col=1
                        )

                    fig_cluster_examples.write_html("sales_seasonality_cluster_examples.html")
                    print("Sales seasonality cluster examples visualization saved")
                    
                except Exception as e:
                    print(f"Error creating cluster examples visualization: {e}")
            else:
                print("Could not determine optimal number of clusters - insufficient variation in patterns")
        else:
            print("Not enough firms with complete patterns for clustering")
    else:
        print("Not enough firms with sufficient data for clustering analysis")
        
except Exception as e:
    print(f"Error in seasonal pattern clustering: {str(e)}")
    print("Continuing with analysis...")

try:
    print("\nAnalyzing daily differences in Traffic-Sales relationship...")

    if 'DayOfWeek' not in df_train.columns:
        if 'Date' in df_train.columns:
            df_train['DayOfWeek'] = df_train['Date'].dt.dayofweek
            print("Created 'DayOfWeek' column from Date for daily differences analysis")
        else:
            raise ValueError("Cannot create DayOfWeek column - Date column not found")

    dow_models = {}
    dow_r2_values = []
    
    for day in range(7):
        day_data = df_train[df_train['DayOfWeek'] == day]

        valid_data = day_data.dropna(subset=['Sales', 'FirmDailyTraffic'])
        
        if len(valid_data) >= 30:
            if valid_data['Sales'].std() > 0 and valid_data['FirmDailyTraffic'].std() > 0:
                try:
                    slope, intercept, r_value, p_value, std_err = stats.linregress(
                        valid_data['FirmDailyTraffic'], 
                        valid_data['Sales']
                    )

                    dow_models[day] = {
                        'slope': slope,
                        'intercept': intercept,
                        'r_value': r_value,
                        'p_value': p_value,
                        'std_err': std_err,
                        'r_squared': r_value**2,
                        'day_name': calendar.day_name[day]
                    }
                    
                    dow_r2_values.append({
                        'day': day,
                        'day_name': calendar.day_name[day],
                        'r_squared': r_value**2,
                        'slope': slope,
                        'significant': p_value < 0.05
                    })
                except Exception as e:
                    print(f"Error modeling relationship for day {day}: {str(e)}")
    
    if dow_r2_values:
        dow_r2_df = pd.DataFrame(dow_r2_values)

        fig_r2 = go.Figure()

        fig_r2.add_trace(go.Bar(
            x=dow_r2_df['day_name'],
            y=dow_r2_df['r_squared'],
            marker_color=['green' if sig else 'lightblue' for sig in dow_r2_df['significant']],
            text=[f"Slope: {slope:.4f}" for slope in dow_r2_df['slope']],
            textposition='outside'
        ))

        fig_r2.update_layout(
            title="Traffic-Sales Relationship Strength by Day of Week",
            xaxis_title="Day of Week",
            yaxis_title="R² Value",
            template="plotly_white",
            yaxis=dict(range=[0, max(dow_r2_df['r_squared']) * 1.2]) 
        )

        fig_r2.write_html("traffic_sales_relationship_by_dow.html")
        print("Traffic-Sales relationship analysis by day of week saved")

        fig_slope = go.Figure()

        fig_slope.add_trace(go.Bar(
            x=dow_r2_df['day_name'],
            y=dow_r2_df['slope'],
            marker_color=['green' if sig else 'lightblue' for sig in dow_r2_df['significant']],
            text=[f"R²: {r2:.4f}" for r2 in dow_r2_df['r_squared']],
            textposition='outside'
        ))

        fig_slope.update_layout(
            title="Traffic-to-Sales Conversion Rate by Day of Week",
            xaxis_title="Day of Week",
            yaxis_title="Slope (Sales per Traffic Unit)",
            template="plotly_white"
        )

        fig_slope.write_html("traffic_sales_conversion_by_dow.html")
        print("Traffic-to-Sales conversion rate analysis by day of week saved")
        
    else:
        print("Not enough data to model Traffic-Sales relationship by day of week")
    
except Exception as e:
    print(f"Error in daily differences analysis: {str(e)}")
    print("Continuing with analysis...")

print("\nEnhanced seasonality analysis complete!")


Performing enhanced seasonality analysis for sales and traffic...
Using data up to 2021-12-03 00:00:00 for seasonality analysis

Analyzing seasonality patterns for Sales...
Using data up to 2021-12-03 00:00:00 for analysis
Analyzing overall daily patterns (day of week)...
Overall day of week analysis for Sales saved
Analyzing overall monthly patterns...
Overall monthly analysis for Sales saved
Performing firm-level seasonality analysis for Sales...
Found 50 firms with sufficient data for analysis
Analyzing firm 0.0 (1/50)
Analyzing firm 1.0 (2/50)
Analyzing firm 2.0 (3/50)
Analyzing firm 3.0 (4/50)
Analyzing firm 4.0 (5/50)
Analyzing firm 5.0 (6/50)
Analyzing firm 6.0 (7/50)
Analyzing firm 7.0 (8/50)
Analyzing firm 8.0 (9/50)
Analyzing firm 9.0 (10/50)
Analyzing firm 10.0 (11/50)
Analyzing firm 11.0 (12/50)
Analyzing firm 12.0 (13/50)
Analyzing firm 13.0 (14/50)
Analyzing firm 14.0 (15/50)
Analyzing firm 15.0 (16/50)
Analyzing firm 16.0 (17/50)
Analyzing firm 17.0 (18/50)
Analyzing fi

# Benchmark Model: Univariate OLS

In [25]:
print("\nPerforming time-based train-validation split...")

df_train = df_train.sort_values('Date').reset_index(drop=True)

split_idx = int(len(df_train) * 0.8)
df_train_subset = df_train.iloc[:split_idx].copy()
df_validate = df_train.iloc[split_idx:].copy()

print(f"Training subset: {len(df_train_subset)} rows ({df_train_subset['Date'].min()} to {df_train_subset['Date'].max()})")
print(f"Validation set: {len(df_validate)} rows ({df_validate['Date'].min()} to {df_validate['Date'].max()})")

print("\nTraining benchmark global OLS model...")

ols_model = smf.ols(
    "Sales ~ C(FirmID) + TrafficPctChange", 
    data=df_train_subset
).fit()

df_validate['Sales_pred_global'] = ols_model.predict(df_validate)
print(df_validate[['FirmID', 'Date', 'Sales', 'Sales_pred_global']].head(10))
print(f"Global OLS model summary (showing select coefficients):\n{ols_model.summary().tables[1]}")


print("\nTraining firm-specific OLS models...")
firm_models = {}
firm_metrics = []

for firm_id in df_train_subset['FirmID'].unique():
    firm_data = df_train_subset[df_train_subset['FirmID'] == firm_id].copy()

    if len(firm_data) < 30: 
        print(f"Skipping Firm {firm_id}: insufficient data ({len(firm_data)} rows)")
        continue

    if firm_data['Sales'].std() == 0 or firm_data['FirmDailyTraffic'].std() == 0:
        print(f"Skipping Firm {firm_id}: no variation in data (constant values)")
        continue

    try:
        formula = "Sales ~ TrafficPctChange + FirmDailyTraffic"

        if 'DayOfWeek' in firm_data.columns:
            formula += " + C(DayOfWeek)"
        if 'Month' in firm_data.columns:
            formula += " + C(Month)"
        if 'IsWeekend' in firm_data.columns:
            formula += " + IsWeekend"

        firm_model = smf.ols(formula, data=firm_data).fit()

        firm_models[firm_id] = firm_model

        firm_validate = df_validate[df_validate['FirmID'] == firm_id].copy()
        if len(firm_validate) > 0:
            try:
                firm_validate['Sales_pred_firm'] = firm_model.predict(firm_validate)

                valid_indices = firm_validate.index[firm_validate['Sales'].notna() & firm_validate['Sales_pred_firm'].notna()]
                
                if len(valid_indices) > 0:
                    valid_actual = firm_validate.loc[valid_indices, 'Sales']
                    valid_pred = firm_validate.loc[valid_indices, 'Sales_pred_firm']
                    
                    rmse = np.sqrt(mean_squared_error(valid_actual, valid_pred))
                    mae = mean_absolute_error(valid_actual, valid_pred)
                    r2 = r2_score(valid_actual, valid_pred)

                    traffic_sales_corr = firm_data['FirmDailyTraffic'].corr(firm_data['Sales'])

                    firm_metrics.append({
                        'FirmID': firm_id,
                        'RMSE': rmse,
                        'MAE': mae,
                        'R2': r2,
                        'DataPoints': len(valid_indices),
                        'Coefficient_Traffic': firm_model.params.get('FirmDailyTraffic', np.nan),
                        'P_Value_Traffic': firm_model.pvalues.get('FirmDailyTraffic', np.nan),
                        'Coefficient_TrafficPct': firm_model.params.get('TrafficPctChange', np.nan),
                        'P_Value_TrafficPct': firm_model.pvalues.get('TrafficPctChange', np.nan),
                        'Traffic_Sales_Correlation': traffic_sales_corr
                    })

                    column_name = f'Sales_pred_firm_{int(firm_id)}'
                    if column_name not in df_validate.columns:
                        df_validate[column_name] = np.nan

                    for idx in valid_indices:
                        df_validate.at[idx, column_name] = firm_validate.at[idx, 'Sales_pred_firm']
                    
                    print(f"Firm {firm_id} model: R² = {r2:.4f}, Traffic coef = {firm_model.params.get('FirmDailyTraffic', np.nan):.4f}, Traffic-Sales corr = {traffic_sales_corr:.4f}")
                else:
                    print(f"No valid predictions for Firm {firm_id}")
            except Exception as e:
                print(f"Error type: {type(e)}")
                print(f"Error predicting for Firm {firm_id}: {str(e)}")
        
    except Exception as e:
        print(f"Error modeling Firm {firm_id}: {e}")

if firm_metrics:
    firm_metrics_df = pd.DataFrame(firm_metrics)

    firm_metrics_df = firm_metrics_df.sort_values('R2', ascending=False)
    
    print("\nFirm-specific model performance (top 10 by R²):")
    print(firm_metrics_df[['FirmID', 'R2', 'RMSE', 'MAE', 'DataPoints', 'Traffic_Sales_Correlation']].head(10))

    firm_metrics_df['Traffic_Significant'] = firm_metrics_df['P_Value_Traffic'] < 0.05
    
    print(f"\nFirms with significant traffic relationship: {firm_metrics_df['Traffic_Significant'].sum()} out of {len(firm_metrics_df)} ({firm_metrics_df['Traffic_Significant'].mean()*100:.1f}%)")

    positive_corr = (firm_metrics_df['Traffic_Sales_Correlation'] > 0).sum()
    negative_corr = (firm_metrics_df['Traffic_Sales_Correlation'] < 0).sum()
    significant_positive = ((firm_metrics_df['Traffic_Sales_Correlation'] > 0) & firm_metrics_df['Traffic_Significant']).sum()
    significant_negative = ((firm_metrics_df['Traffic_Sales_Correlation'] < 0) & firm_metrics_df['Traffic_Significant']).sum()
    
    print("\nTraffic-Sales correlation analysis:")
    print(f"Firms with positive correlation: {positive_corr} ({positive_corr/len(firm_metrics_df)*100:.1f}%)")
    print(f"Firms with negative correlation: {negative_corr} ({negative_corr/len(firm_metrics_df)*100:.1f}%)")
    print(f"Firms with significant positive correlation: {significant_positive} ({significant_positive/len(firm_metrics_df)*100:.1f}%)")
    print(f"Firms with significant negative correlation: {significant_negative} ({significant_negative/len(firm_metrics_df)*100:.1f}%)")
    print(f"Average correlation: {firm_metrics_df['Traffic_Sales_Correlation'].mean():.4f}")
    print(f"Median correlation: {firm_metrics_df['Traffic_Sales_Correlation'].median():.4f}")

    df_validate['Sales_pred_combined'] = df_validate['Sales_pred_global']  

    for firm_id in firm_models.keys():
        column_name = f'Sales_pred_firm_{int(firm_id)}'
        if column_name in df_validate.columns:
            mask = df_validate[column_name].notna()
            if mask.any():
                df_validate.loc[mask, 'Sales_pred_combined'] = df_validate.loc[mask, column_name]

    valid_mask_global = df_validate['Sales'].notna() & df_validate['Sales_pred_global'].notna()
    valid_mask_combined = df_validate['Sales'].notna() & df_validate['Sales_pred_combined'].notna()
    
    if valid_mask_global.sum() > 0 and valid_mask_combined.sum() > 0:
        valid_actual_global = df_validate.loc[valid_mask_global, 'Sales']
        valid_pred_global = df_validate.loc[valid_mask_global, 'Sales_pred_global']
        
        valid_actual_combined = df_validate.loc[valid_mask_combined, 'Sales']
        valid_pred_combined = df_validate.loc[valid_mask_combined, 'Sales_pred_combined']
        
        global_rmse = np.sqrt(mean_squared_error(valid_actual_global, valid_pred_global))
        global_mae = mean_absolute_error(valid_actual_global, valid_pred_global)
        global_r2 = r2_score(valid_actual_global, valid_pred_global)
        
        combined_rmse = np.sqrt(mean_squared_error(valid_actual_combined, valid_pred_combined))
        combined_mae = mean_absolute_error(valid_actual_combined, valid_pred_combined)
        combined_r2 = r2_score(valid_actual_combined, valid_pred_combined)
        
        print("\nComparison of models:")

        avg_r2 = np.average(firm_metrics_df['R2'], weights=firm_metrics_df['DataPoints'])
        avg_rmse = np.average(firm_metrics_df['RMSE'], weights=firm_metrics_df['DataPoints'])
        avg_mae = np.average(firm_metrics_df['MAE'], weights=firm_metrics_df['DataPoints'])
        
        print(f"Individual firm-specific models - Weighted Avg R²: {avg_r2:.4f}, RMSE: {avg_rmse:.2f}, MAE: {avg_mae:.2f}")
        print(f"Combined model - R²: {combined_r2:.4f}, RMSE: {combined_rmse:.2f}, MAE: {combined_mae:.2f}")
        print(f"Global model - R²: {global_r2:.4f}, RMSE: {global_rmse:.2f}, MAE: {global_mae:.2f}")

        if global_r2 != 0:
            r2_improvement = (combined_r2 - global_r2) / abs(global_r2) * 100
        else:
            r2_improvement = float('inf') if combined_r2 > 0 else float('-inf') if combined_r2 < 0 else 0
            
        rmse_improvement = (global_rmse - combined_rmse) / global_rmse * 100
        mae_improvement = (global_mae - combined_mae) / global_mae * 100
        
        print(f"\nImprovement of combined model over global model:")
        print(f"  R² improvement: {r2_improvement:.1f}%")
        print(f"  RMSE reduction: {rmse_improvement:.1f}%")
        print(f"  MAE reduction: {mae_improvement:.1f}%")
    else:
        print("Not enough valid predictions to compare models")

try:
    if 'firm_metrics_df' in locals() and not firm_metrics_df.empty:
        plot_df = firm_metrics_df.sort_values('Coefficient_Traffic', ascending=False)

        fig = make_subplots(specs=[[{"secondary_y": True}]])

        fig.add_trace(
            go.Bar(
                x=plot_df['FirmID'],
                y=plot_df['Coefficient_Traffic'],
                name="Traffic Coefficient",
                marker_color=['green' if sig else 'lightgreen' for sig in plot_df['Traffic_Significant']],
                opacity=0.7
            ),
            secondary_y=False,
        )

        fig.add_trace(
            go.Scatter(
                x=plot_df['FirmID'],
                y=plot_df['R2'],
                mode='markers',
                name="R² Value",
                marker=dict(
                    size=8,
                    color='red',
                    symbol='circle'
                )
            ),
            secondary_y=True,
        )

        fig.add_trace(
            go.Scatter(
                x=plot_df['FirmID'],
                y=plot_df['Traffic_Sales_Correlation'],
                mode='markers',
                name="Traffic-Sales Correlation",
                marker=dict(
                    size=8,
                    color='blue',
                    symbol='diamond'
                )
            ),
            secondary_y=True,
        )

        fig.add_shape(
            type="line",
            x0=plot_df['FirmID'].iloc[0],
            y0=0,
            x1=plot_df['FirmID'].iloc[-1],
            y1=0,
            line=dict(color="black", width=1, dash="dash"),
            xref="x",
            yref="y"
        )

        fig.update_layout(
            title_text="Firm-Specific Models: Traffic Coefficient and Model Performance",
            template="plotly_white",
            height=600,
            width=max(1000, len(plot_df)*20),
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1
            ),
            xaxis=dict(
                title="Firm ID",
                tickangle=45
            )
        )

        fig.update_yaxes(title_text="Traffic Coefficient", secondary_y=False)
        fig.update_yaxes(title_text="R² / Correlation", secondary_y=True)

        fig.write_html("firm_specific_models_performance.html")
        print("\nFirm-specific models visualization saved")

        corr_fig = go.Figure()

        corr_fig.add_trace(
            go.Histogram(
                x=firm_metrics_df['Traffic_Sales_Correlation'],
                nbinsx=20,
                marker_color='blue',
                opacity=0.7,
                name="All Correlations"
            )
        )

        significant_mask = firm_metrics_df['Traffic_Significant']
        if significant_mask.sum() > 0:
            corr_fig.add_trace(
                go.Histogram(
                    x=firm_metrics_df.loc[significant_mask, 'Traffic_Sales_Correlation'],
                    nbinsx=20,
                    marker_color='green',
                    opacity=0.7,
                    name="Significant Correlations"
                )
            )

        corr_fig.add_shape(
            type="line",
            x0=0, y0=0,
            x1=0, y1=1,
            yref="paper",
            line=dict(color="red", width=2, dash="dash")
        )

        corr_fig.update_layout(
            title_text="Distribution of Traffic-Sales Correlations Across Firms",
            template="plotly_white",
            height=500,
            width=800,
            xaxis_title="Correlation Coefficient",
            yaxis_title="Number of Firms",
            bargap=0.1,
            barmode='overlay'
        )

        corr_fig.write_html("traffic_sales_correlation_distribution.html")
        print("Traffic-Sales correlation distribution visualization saved")
        
except Exception as e:
    print(f"Error creating firm-specific model visualizations: {e}")

try:
    df_validate['Sales_pred_combined'] = np.nan
    
    for firm_id in firm_models.keys():
        firm_mask = df_validate['FirmID'] == firm_id
        pred_col = f'Sales_pred_firm_{firm_id}'
        
        if pred_col in df_validate.columns:
            df_validate.loc[firm_mask, 'Sales_pred_combined'] = df_validate.loc[firm_mask, pred_col]

    mask = df_validate['Sales_pred_combined'].isna()
    df_validate.loc[mask, 'Sales_pred_combined'] = df_validate.loc[mask, 'Sales_pred_global']

    valid_mask_combined = df_validate['Sales'].notna() & df_validate['Sales_pred_combined'].notna()
    
    if valid_mask_combined.sum() > 0:
        valid_actual_combined = df_validate.loc[valid_mask_combined, 'Sales']
        valid_pred_combined = df_validate.loc[valid_mask_combined, 'Sales_pred_combined']
        
        combined_rmse = np.sqrt(mean_squared_error(valid_actual_combined, valid_pred_combined))
        combined_mae = mean_absolute_error(valid_actual_combined, valid_pred_combined)
        combined_r2 = r2_score(valid_actual_combined, valid_pred_combined)
        
        print("\nCombined model performance (firm-specific where available, global otherwise):")
        print(f"R²: {combined_r2:.4f}, RMSE: {combined_rmse:.2f}, MAE: {combined_mae:.2f}")

        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=[
                "Global vs. Firm-Specific Predictions", 
                "Prediction Errors", 
                "Predictions Over Time (Sample Firms)",
                "R² by Firm"
            ],
            specs=[
                [{"type": "scatter"}, {"type": "histogram"}],
                [{"type": "scatter"}, {"type": "bar"}]
            ]
        )

        fig.add_trace(
            go.Scatter(
                x=valid_actual_combined,
                y=valid_pred_combined,
                mode='markers',
                marker=dict(
                    size=8,
                    color='blue',
                    opacity=0.6
                ),
                name="Firm-Specific Models"
            ),
            row=1, col=1
        )

        valid_global = df_validate['Sales'].notna() & df_validate['Sales_pred_global'].notna()
        fig.add_trace(
            go.Scatter(
                x=df_validate.loc[valid_global, 'Sales'],
                y=df_validate.loc[valid_global, 'Sales_pred_global'],
                mode='markers',
                marker=dict(
                    size=8,
                    color='red',
                    opacity=0.4
                ),
                name="Global Model"
            ),
            row=1, col=1
        )

        max_val = max(max(valid_actual_combined), max(valid_pred_combined))
        min_val = min(min(valid_actual_combined), min(valid_pred_combined))
        fig.add_trace(
            go.Scatter(
                x=[min_val, max_val],
                y=[min_val, max_val],
                mode='lines',
                line=dict(color='black', dash='dash'),
                name="Perfect Prediction"
            ),
            row=1, col=1
        )

        df_validate['error_global'] = df_validate['Sales'] - df_validate['Sales_pred_global']
        df_validate['error_combined'] = df_validate['Sales'] - df_validate['Sales_pred_combined']

        fig.add_trace(
            go.Histogram(
                x=df_validate['error_global'],
                nbinsx=30,
                marker_color='red',
                opacity=0.5,
                name="Global Model Errors"
            ),
            row=1, col=2
        )
        
        fig.add_trace(
            go.Histogram(
                x=df_validate['error_combined'],
                nbinsx=30,
                marker_color='blue',
                opacity=0.5,
                name="Firm-Specific Model Errors"
            ),
            row=1, col=2
        )

        if 'firm_metrics_df' in locals() and not firm_metrics_df.empty:
            top_firms = firm_metrics_df.sort_values('R2', ascending=False).head(5)['FirmID'].tolist()
            
            for i, firm_id in enumerate(top_firms):
                firm_data = df_validate[df_validate['FirmID'] == firm_id].sort_values('Date')
                
                if len(firm_data) > 0:
                    pred_col = f'Sales_pred_firm_{firm_id}'
                    if pred_col in firm_data.columns:
                        fig.add_trace(
                            go.Scatter(
                                x=firm_data['Date'],
                                y=firm_data['Sales'],
                                mode='lines',
                                name=f"Firm {firm_id} - Actual",
                                line=dict(width=2)
                            ),
                            row=2, col=1
                        )

                        fig.add_trace(
                            go.Scatter(
                                x=firm_data['Date'],
                                y=firm_data[pred_col],
                                mode='lines',
                                name=f"Firm {firm_id} - Predicted",
                                line=dict(width=2, dash='dash')
                            ),
                            row=2, col=1
                        )

        if 'firm_metrics_df' in locals() and not firm_metrics_df.empty:
            plot_firms = firm_metrics_df.sort_values('R2', ascending=False).head(20)
            
            fig.add_trace(
                go.Bar(
                    x=plot_firms['FirmID'],
                    y=plot_firms['R2'],
                    marker_color='blue',
                    name="R² by Firm"
                ),
                row=2, col=2
            )

            fig.add_shape(
                type="line",
                x0=plot_firms['FirmID'].iloc[0],
                y0=global_r2,
                x1=plot_firms['FirmID'].iloc[-1],
                y1=global_r2,
                line=dict(color="red", width=2, dash="dash"),
                row=2, col=2
            )

            fig.add_annotation(
                text=f"Global Model R² = {global_r2:.4f}",
                x=plot_firms['FirmID'].iloc[-1],
                y=global_r2,
                showarrow=False,
                font=dict(color="red"),
                xref="x3",
                yref="y3"
            )

        fig.update_layout(
            title_text="Global vs. Firm-Specific Models Performance",
            template="plotly_white",
            showlegend=True,
            height=900,
            width=1200,
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=-0.2,
                xanchor="center",
                x=0.5
            )
        )

        fig.update_xaxes(title_text="Actual Sales", row=1, col=1)
        fig.update_yaxes(title_text="Predicted Sales", row=1, col=1)
        fig.update_xaxes(title_text="Prediction Error", row=1, col=2)
        fig.update_yaxes(title_text="Frequency", row=1, col=2)
        fig.update_xaxes(title_text="Date", row=2, col=1)
        fig.update_yaxes(title_text="Sales", row=2, col=1)
        fig.update_xaxes(title_text="Firm ID", row=2, col=2, tickangle=45)
        fig.update_yaxes(title_text="R²", row=2, col=2)

        fig.add_annotation(
            xref="paper", yref="paper",
            x=0.25, y=0.05, 
            text=f"Global - R²: {global_r2:.4f}, RMSE: {global_rmse:.2f}, MAE: {global_mae:.2f}<br>Firm-Specific - R²: {combined_r2:.4f}, RMSE: {combined_rmse:.2f}, MAE: {combined_mae:.2f}",
            showarrow=False,
            font=dict(size=12),
            bgcolor="rgba(255, 255, 255, 0.8)",
            bordercolor="black",
            borderwidth=1,
            borderpad=4
        )

        fig.write_html("global_vs_firm_specific_models.html")
        print("Global vs. firm-specific models comparison visualization saved")
        
except Exception as e:
    print(f"Error creating model comparison visualizations: {e}")


Performing time-based train-validation split...
Training subset: 28120 rows (2020-01-01 00:00:00 to 2021-07-16 00:00:00)
Validation set: 7030 rows (2021-07-16 00:00:00 to 2021-12-03 00:00:00)

Training benchmark global OLS model...
       FirmID       Date        Sales  Sales_pred_global
28120    16.0 2021-07-16  3384.403700        5412.022243
28121     2.0 2021-07-16  1109.660432        1265.946645
28122     0.0 2021-07-16  1023.047051        2092.900826
28123    31.0 2021-07-16   169.120030         228.358348
28124    23.0 2021-07-16  1478.048269        1868.809268
28125    39.0 2021-07-16   760.310225        1975.129103
28126    34.0 2021-07-16   170.549860         290.827770
28127    24.0 2021-07-16   968.838594        1672.318213
28128     9.0 2021-07-16  1270.735680        1143.006672
28129    37.0 2021-07-16   434.464832         833.195094
Global OLS model summary (showing select coefficients):
                        coef    std err          t      P>|t|      [0.025      0.975

# Hyperparameter Tuning

In [26]:
print("\n" + "=" * 50)
print("FOCUSED MULTI-FIRM HYPERPARAMETER TUNING")
print("=" * 50)

print("Using existing training data from feature engineering step")

print("Creating validation split while preserving time series structure...")
train_dfs = []
valid_dfs = []

for firm in df_train['FirmID'].unique():
    firm_data = df_train[df_train['FirmID'] == firm].copy().sort_values('Date')

    train_size = int(0.8 * len(firm_data))

    if train_size >= len(firm_data):
        train_size = max(int(0.7 * len(firm_data)), len(firm_data) - 5)

    train_dfs.append(firm_data.iloc[:train_size])
    valid_dfs.append(firm_data.iloc[train_size:])

df_train_model = pd.concat(train_dfs).reset_index(drop=True)
df_valid = pd.concat(valid_dfs).reset_index(drop=True)

print(f"Training set: {len(df_train_model)} rows, Validation set: {len(df_valid)} rows")
print(f"Number of firms in training: {df_train_model['FirmID'].nunique()}")
print(f"Number of firms in validation: {df_valid['FirmID'].nunique()}")

volatility_features = [
        'FirmID', 'FirmDailyTraffic',  # Core features
        'DayOfWeek', 'Month', 'IsWeekend',  # Time components
        'Sales_lag1', 'Sales_lag7'  # Minimal lag features
    ]

tscv = TimeSeriesSplit(n_splits=5)

X_train = df_train_model[volatility_features]
y_train = df_train_model['Sales']
X_valid = df_valid[volatility_features]
y_valid = df_valid['Sales']

def calculate_r2(y_true, y_pred):
    """
    Calculate R-squared (coefficient of determination) with proper error handling

    Args:
        y_true: Actual values
        y_pred: Predicted values

    Returns:
        r2 value (raw, not clamped)
    """
    try:
        return r2_score(y_true, y_pred)
    except Exception as e:
        logging.error(f"Error calculating R2: {e}")
        return float('nan')

def plot_optimization_history(study, filename="optuna_optimization_history.html"):
    """
    Plot the optimization history of an Optuna study using Plotly
    """
    trials = study.trials
    values = [t.value for t in trials if t.value is not None]
    best_values = np.minimum.accumulate(values)

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=list(range(len(values))),
        y=values,
        mode='markers',
        name='Trial Value',
        marker=dict(color='blue', size=8)
    ))

    fig.add_trace(go.Scatter(
        x=list(range(len(best_values))),
        y=best_values,
        mode='lines',
        name='Best Value',
        line=dict(color='red', width=2)
    ))

    fig.update_layout(
        title="Optimization History",
        xaxis_title="Trial",
        yaxis_title="Objective Value (RMSE)",
        template="plotly_white"
    )

    fig.write_html(filename)
    print(f"Optimization history saved to {filename}")
    return fig


def plot_param_importances(study, filename="optuna_param_importances.html"):
    """
    Plot the parameter importances of an Optuna study using Plotly
    """
    importances = optuna.importance.get_param_importances(study)
    sorted_importances = dict(sorted(importances.items(), key=lambda x: x[1], reverse=True))

    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=list(sorted_importances.keys()),
        y=list(sorted_importances.values()),
        marker_color='royalblue'
    ))

    fig.update_layout(
        title="Parameter Importances",
        xaxis_title="Parameter",
        yaxis_title="Importance",
        template="plotly_white"
    )

    fig.write_html(filename)
    print(f"Parameter importances saved to {filename}")
    return fig


def stratified_firm_sample(X, y, max_per_firm=2000, min_per_firm=50, max_total=10000):
    """
    Create a stratified sample across firms to ensure all firms are represented
    while preserving time dependencies within each firm's data

    Args:
        X: Feature DataFrame with FirmID column
        y: Target variable
        max_per_firm: Maximum samples to take from any single firm
        min_per_firm: Minimum samples to include from a firm when available
        max_total: Maximum total samples to return

    Returns:
        List of indices for the stratified sample
    """
    firms = X['FirmID'].unique()
    sampled_indices = []

    for firm in firms:
        firm_indices = X[X['FirmID'] == firm].index
        if len(firm_indices) <= min_per_firm:
            firm_sample = firm_indices
        else:
            sorted_indices = sorted(firm_indices)
            sample_size = min(max_per_firm, len(firm_indices))

            step = len(sorted_indices) / sample_size
            systematic_indices = [sorted_indices[int(i * step)] for i in range(sample_size)]
            firm_sample = systematic_indices

        sampled_indices.extend(firm_sample)

    if len(sampled_indices) > max_total:
        firm_counts = X.loc[sampled_indices, 'FirmID'].value_counts()
        prop_sample_indices = []

        for firm, count in firm_counts.items():
            firm_indices = [idx for idx in sampled_indices if X.loc[idx, 'FirmID'] == firm]

            alloc_size = int(max_total * (count / len(sampled_indices)))

            alloc_size = max(1, alloc_size)

            if len(firm_indices) <= alloc_size:
                prop_sample_indices.extend(firm_indices)
            else:
                step = len(firm_indices) / alloc_size
                systematic_indices = [firm_indices[int(i * step)] for i in range(alloc_size)]
                prop_sample_indices.extend(systematic_indices)

        sampled_indices = prop_sample_indices[:max_total]

    return sampled_indices


def tune_lightgbm_optuna(X_train, y_train, X_valid, y_valid, tscv, n_trials=100):
    """
    Tune LightGBM hyperparameters using Optuna with Bayesian optimization

    Args:
        X_train: Training features
        y_train: Target variable
        X_valid: Validation features
        y_valid: Validation target
        tscv: TimeSeriesSplit cross-validation object
        n_trials: Number of trials for optimization

    Returns:
        Best parameters and performance metrics
    """
    print("\nTuning LightGBM hyperparameters using Optuna...")
    
    categorical_columns = ['FirmID', 'DayOfWeek', 'Month', 'DayOfMonth', 'IsWeekend']
    for col in categorical_columns:
        if col in X_train.columns:
            X_train[col] = X_train[col].astype(int)
            X_valid[col] = X_valid[col].astype(int)
    
    categorical_features = [X_train.columns.get_loc(col) for col in categorical_columns 
                          if col in X_train.columns]

    # print(X_train.info())

    def objective(trial):
        param = {
            'objective': 'regression',
            'metric': 'rmse',
            'verbosity': -1,
            'boosting_type': 'gbdt',
            'lambda_l1': trial.suggest_float('lambda_l1', 0.01, 10.0, log=True),
            'lambda_l2': trial.suggest_float('lambda_l2', 0.01, 10.0, log=True),
            'num_leaves': trial.suggest_int('num_leaves', 20, 100),
            'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 0.9),
            'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 0.9),
            'bagging_freq': trial.suggest_int('bagging_freq', 1, 5),
            'min_child_samples': trial.suggest_int('min_child_samples', 10, 50),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
            'max_depth': trial.suggest_int('max_depth', 3, 8),
            'n_estimators': trial.suggest_int('n_estimators', 100, 500)
        }

        cv_scores_rmse = []
        cv_scores_r2 = []

        for train_idx, val_idx in tscv.split(X_train):
            X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
            y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]

            train_data = lgb.Dataset(X_train_fold, label=y_train_fold, categorical_feature=categorical_features)
            val_data = lgb.Dataset(X_val_fold, label=y_val_fold, categorical_feature=categorical_features)

            model = lgb.train(
                param,
                train_data,
                num_boost_round=param['n_estimators'],
                valid_sets=[val_data],
                valid_names=['validation'],
                callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)]
            )

            y_pred = model.predict(X_val_fold)
            rmse = np.sqrt(mean_squared_error(y_val_fold, y_pred))
            r2 = calculate_r2(y_val_fold, y_pred)

            cv_scores_rmse.append(rmse)
            cv_scores_r2.append(r2)

        return np.mean(cv_scores_rmse)

    sampler = optuna.samplers.TPESampler(seed=42)

    try:
        study = optuna.create_study(direction='minimize', sampler=sampler)
        study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

        best_params = study.best_params
        best_score = study.best_value

        print(f"Number of finished trials: {len(study.trials)}")
        print(f"Best LightGBM parameters: {best_params}")
        print(f"Best RMSE: {best_score:.4f}")

        final_model = lgb.LGBMRegressor(**best_params, categorical_feature=categorical_features)
        final_model.fit(X_train, y_train)

        y_valid_pred = final_model.predict(X_valid)

        valid_rmse = np.sqrt(mean_squared_error(y_valid, y_valid_pred))
        valid_r2 = calculate_r2(y_valid, y_valid_pred)

        print(f"Validation RMSE: {valid_rmse:.4f}")
        print(f"Validation R²: {valid_r2:.4f}")

        if len(study.trials) > 5:
            try:
                plot_optimization_history(study, "lightgbm_optimization_history.html")

                importances = optuna.importance.get_param_importances(study)
                print("\nParameter importance:")
                for key, value in importances.items():
                    print(f"  {key}: {value:.3f}")
                plot_param_importances(study, "lightgbm_param_importances.html")

                if len(y_valid) > 1000:
                    sample_idx = np.random.choice(len(y_valid), 1000, replace=False)
                    y_true_sample = y_valid.iloc[sample_idx]
                    y_pred_sample = y_valid_pred[sample_idx]
                else:
                    y_true_sample = y_valid
                    y_pred_sample = y_valid_pred

                fig = go.Figure()

                fig.add_trace(go.Scatter(
                    x=y_true_sample,
                    y=y_pred_sample,
                    mode='markers',
                    marker=dict(color='blue', size=8, opacity=0.6),
                    name='Predictions'
                ))

                max_val = max(max(y_true_sample), max(y_pred_sample))
                min_val = min(min(y_true_sample), min(y_pred_sample))

                fig.add_trace(go.Scatter(
                    x=[min_val, max_val],
                    y=[min_val, max_val],
                    mode='lines',
                    line=dict(color='red', dash='dash'),
                    name='Perfect Prediction'
                ))

                fig.update_layout(
                    title=f"LightGBM: Actual vs Predicted Sales (Validation R² = {valid_r2:.4f})",
                    xaxis_title="Actual Sales",
                    yaxis_title="Predicted Sales",
                    template="plotly_white"
                )

                fig.write_html("lightgbm_actual_vs_predicted.html")
                print("LightGBM actual vs predicted plot saved")

                firm_metrics = {}
                for firm in X_valid['FirmID'].unique():
                    firm_mask = X_valid['FirmID'] == firm
                    firm_y_true = y_valid[firm_mask]
                    firm_y_pred = y_valid_pred[firm_mask]

                    if len(firm_y_true) > 0:
                        firm_rmse = np.sqrt(mean_squared_error(firm_y_true, firm_y_pred))
                        firm_r2 = calculate_r2(firm_y_true, firm_y_pred)
                        firm_metrics[firm] = {'rmse': firm_rmse, 'r2': firm_r2, 'count': len(firm_y_true)}

                firms = list(firm_metrics.keys())
                if len(firms) > 0:
                    firms_to_show = sorted(firms, key=lambda x: firm_metrics[x]['count'], reverse=True)[:15]

                    fig = make_subplots(rows=2, cols=1,
                                        subplot_titles=["LightGBM RMSE by Firm",
                                                        "LightGBM R² by Firm"],
                                        vertical_spacing=0.15)

                    fig.add_trace(
                        go.Bar(
                            x=[f"Firm {f}" for f in firms_to_show],
                            y=[firm_metrics[f]['rmse'] for f in firms_to_show],
                            name="RMSE",
                            marker_color='red',
                            text=[f"{firm_metrics[f]['rmse']:.4f}" for f in firms_to_show],
                            textposition="outside"
                        ),
                        row=1, col=1
                    )

                    fig.add_trace(
                        go.Bar(
                            x=[f"Firm {f}" for f in firms_to_show],
                            y=[firm_metrics[f]['r2'] for f in firms_to_show],
                            name="R²",
                            marker_color='blue',
                            text=[f"{firm_metrics[f]['r2']:.4f}" for f in firms_to_show],
                            textposition="outside"
                        ),
                        row=2, col=1
                    )

                    fig.update_layout(
                        height=800,
                        title_text="LightGBM Performance by Firm",
                        showlegend=False
                    )

                    fig.write_html("lightgbm_firm_performance.html")
                    print("LightGBM firm-specific performance visualization saved")

            except Exception as e:
                print(f"Error creating visualization: {e}")

        if 'n_estimators' in best_params:
            best_params['n_estimators'] = int(best_params['n_estimators'])

        return best_params, best_score, valid_r2, final_model
    except Exception as e:
        print(f"Error during LightGBM Optuna tuning: {e}")
        print("Full traceback:")
        traceback.print_exc()
        
        fallback_params = {
            'lambda_l1': 0.1,
            'lambda_l2': 0.1,
            'num_leaves': 31,
            'feature_fraction': 0.8,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'min_child_samples': 20,
            'learning_rate': 0.05,
            'max_depth': 5,
            'n_estimators': 100
        }
        print(f"Using fallback parameters: {fallback_params}")
        return fallback_params, None, None, None


def tune_prophet_for_firms(df, top_firms=None, param_combinations=15, max_firms=10):
    """
    Tune Prophet hyperparameters for multiple firms and return a dictionary of models

    Args:
        df: DataFrame with Date, Sales, FirmDailyTraffic and FirmID
        top_firms: List of firm IDs to focus on (if None, uses firms with most data)
        param_combinations: Number of random parameter combinations to try
        max_firms: Maximum number of firms to tune for

    Returns:
        Dictionary with firm-specific prophet parameters, models and metrics
    """
    print(
        f"\nTuning Prophet for up to {max_firms} different firms with {param_combinations} parameter combinations each...")

    firm_counts = df['FirmID'].value_counts()
    print(f"Total firms with non-missing data: {len(firm_counts)}")

    if top_firms is None:
        valid_firms = firm_counts[firm_counts >= 30].index.tolist()

        top_firms = valid_firms[:max_firms]
    else:
        top_firms = [firm for firm in top_firms if firm in firm_counts.index and firm_counts[firm] >= 30]
        top_firms = top_firms[:max_firms]

    print(f"Selected {len(top_firms)} firms for Prophet tuning: {top_firms}")
    print(f"Firm data points: {', '.join([f'Firm {f}: {firm_counts[f]}' for f in top_firms])}")

    firm_results = {}

    param_grid = {
        'changepoint_prior_scale': [0.001, 0.01, 0.05, 0.1, 0.5],
        'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
        'holidays_prior_scale': [0.01, 0.1, 1.0, 10.0],
        'seasonality_mode': ['additive'],
        'changepoint_range': [0.8, 0.9, 0.95]
    }

    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    np.random.seed(42)
    if len(all_params) > param_combinations:
        param_samples = np.random.choice(range(len(all_params)), param_combinations, replace=False)
        param_list = [all_params[i] for i in param_samples]
    else:
        param_list = all_params

    print(f"Testing {len(param_list)} parameter combinations for each firm")

    prophet_regressors = [
        'FirmDailyTraffic',
        'TrafficPctChange'
    ]

    combined_fig = make_subplots(rows=min(len(top_firms), 6), cols=1,
                                 subplot_titles=[f"Firm {firm}" for firm in top_firms[:6]],
                                 vertical_spacing=0.1)

    param_importance_dfs = []

    for i, firm_id in enumerate(top_firms):
        print(f"\nTuning Prophet for Firm {firm_id}")

        firm_data = df[df['FirmID'] == firm_id].copy()
        print(f"  Data points for Firm {firm_id}: {len(firm_data)}")

        train_size = int(0.8 * len(firm_data))
        if train_size >= len(firm_data):
            train_size = max(int(0.7 * len(firm_data)), len(firm_data) - 5)

        train_df = firm_data.iloc[:train_size].copy()
        val_df = firm_data.iloc[train_size:].copy()

        train_prophet = train_df.rename(columns={'Date': 'ds', 'Sales': 'y'})
        val_prophet = val_df.rename(columns={'Date': 'ds', 'Sales': 'y'})

        for regressor in prophet_regressors:
            if regressor in train_prophet.columns:
                if train_prophet[regressor].isna().any():

                    train_prophet[regressor] = train_prophet[regressor].fillna(train_prophet[regressor].median())

                if val_prophet[regressor].isna().any():

                    val_prophet[regressor] = val_prophet[regressor].fillna(train_prophet[regressor].median())

        print(f"  Training data: {len(train_prophet)} rows")
        print(f"  Validation data: {len(val_prophet)} rows")

        if len(train_prophet) < 10 or len(val_prophet) < 5:
            print(f"  Not enough data for Firm {firm_id}, skipping...")
            continue

        param_results = []
        rmses = []
        r2s = []
        models = []

        for params in param_list:
            try:
                model = Prophet(
                    changepoint_prior_scale=params['changepoint_prior_scale'],
                    seasonality_prior_scale=params['seasonality_prior_scale'],
                    holidays_prior_scale=params['holidays_prior_scale'],
                    seasonality_mode=params['seasonality_mode'],
                    changepoint_range=params['changepoint_range'],
                    yearly_seasonality=True,
                    weekly_seasonality=True,
                    daily_seasonality=False
                )

                for regressor in prophet_regressors:
                    if regressor in train_prophet.columns:
                        model.add_regressor(regressor)

                try:
                    with warnings.catch_warnings():
                        warnings.filterwarnings("ignore")
                        model.fit(train_prophet)
                except Exception as e:
                    print(f"  Error during Prophet model fit with parameters {params}: {e}")
                    continue

                future = pd.DataFrame(val_prophet['ds'])

                for regressor in prophet_regressors:
                    if regressor in val_prophet.columns:
                        future[regressor] = val_prophet[regressor].values

                forecast = model.predict(future)

                y_true = val_prophet['y'].values
                y_pred = forecast['yhat'].values
                rmse = np.sqrt(mean_squared_error(y_true, y_pred))
                r2 = calculate_r2(y_true, y_pred)

                rmses.append((params, rmse))
                r2s.append((params, r2))
                models.append((params, model))

                param_results.append({
                    'firm_id': firm_id,
                    'changepoint_prior_scale': params['changepoint_prior_scale'],
                    'seasonality_prior_scale': params['seasonality_prior_scale'],
                    'holidays_prior_scale': params['holidays_prior_scale'],
                    'changepoint_range': params['changepoint_range'],
                    'rmse': rmse,
                    'r2': r2
                })

                print(f"  Parameters {params} - RMSE: {rmse:.4f}, R²: {r2:.4f}")
            except Exception as e:
                print(f"  Error with Prophet parameters {params} for Firm {firm_id}: {e}")

        if param_results:
            param_df = pd.DataFrame(param_results)
            param_importance_dfs.append(param_df)

        if rmses:
            best_params, best_rmse = min(rmses, key=lambda x: x[1])

            best_r2 = next((r2 for params, r2 in r2s if params == best_params), float('nan'))

            best_model = next((model for params, model in models if params == best_params), None)

            print(f"  Best Prophet parameters for Firm {firm_id}: {best_params}")
            print(f"  Best RMSE: {best_rmse:.4f}")
            print(f"  Best validation R²: {best_r2:.4f}")

            future = pd.DataFrame(val_prophet['ds'])

            for regressor in prophet_regressors:
                if regressor in val_prophet.columns:
                    future[regressor] = val_prophet[regressor].values

            forecast = best_model.predict(future)

            if i < 6:
                combined_fig.add_trace(
                    go.Scatter(
                        x=val_prophet['ds'],
                        y=val_prophet['y'],
                        mode='markers',
                        name=f'Actual (Firm {firm_id})',
                        marker=dict(color='blue')
                    ),
                    row=i + 1, col=1
                )

                combined_fig.add_trace(
                    go.Scatter(
                        x=val_prophet['ds'],
                        y=forecast['yhat'],
                        mode='lines',
                        name=f'Predicted (Firm {firm_id})',
                        line=dict(color='red')
                    ),
                    row=i + 1, col=1
                )

            firm_results[firm_id] = {
                'best_params': best_params,
                'best_rmse': best_rmse,
                'best_r2': best_r2,
                'best_model': best_model,
                'training_data': train_prophet,
                'validation_data': val_prophet,
                'all_results': param_results
            }

            if len(param_results) > 5:
                try:
                    param_df = pd.DataFrame(param_results)

                    corr_rmse = param_df[['changepoint_prior_scale', 'seasonality_prior_scale',
                                          'holidays_prior_scale', 'changepoint_range']].corrwith(param_df['rmse'])

                    corr_r2 = param_df[['changepoint_prior_scale', 'seasonality_prior_scale',
                                        'holidays_prior_scale', 'changepoint_range']].corrwith(param_df['r2'])

                    param_fig = make_subplots(rows=1, cols=2,
                                              subplot_titles=[f"Parameter Correlation with RMSE",
                                                              f"Parameter Correlation with R²"])

                    param_fig.add_trace(
                        go.Bar(
                            x=corr_rmse.index,
                            y=corr_rmse.values,
                            name="Correlation with RMSE",
                            marker_color='red'
                        ),
                        row=1, col=1
                    )

                    param_fig.add_trace(
                        go.Bar(
                            x=corr_r2.index,
                            y=corr_r2.values,
                            name="Correlation with R²",
                            marker_color='blue'
                        ),
                        row=1, col=2
                    )

                    param_fig.update_layout(
                        title=f"Firm {firm_id} - Prophet Parameter Importance",
                        height=400,
                        width=800
                    )

                    param_fig.write_html(f"prophet_firm_{firm_id}_param_importance.html")
                    print(f"  Parameter importance visualization saved for Firm {firm_id}")

                except Exception as e:
                    print(f"  Error creating parameter importance visualization for Firm {firm_id}: {e}")

        else:
            print(f"  No valid models for Firm {firm_id}")

    combined_fig.update_layout(
        height=300 * min(len(top_firms), 6),
        title_text="Prophet Model Predictions by Firm",
        showlegend=True
    )

    combined_fig.write_html("prophet_predictions_by_firm.html")
    print("\nProphet predictions by firm visualization saved")

    firm_ids = list(firm_results.keys())

    if firm_ids:
        summary_table = go.Figure(data=[go.Table(
            header=dict(
                values=["Firm ID", "changepoint_prior_scale", "seasonality_prior_scale",
                        "holidays_prior_scale", "changepoint_range", "RMSE", "R²"],
                fill_color='paleturquoise',
                align='left'
            ),
            cells=dict(
                values=[
                    firm_ids,
                    [firm_results[firm]['best_params']['changepoint_prior_scale'] for firm in firm_ids],
                    [firm_results[firm]['best_params']['seasonality_prior_scale'] for firm in firm_ids],
                    [firm_results[firm]['best_params']['holidays_prior_scale'] for firm in firm_ids],
                    [firm_results[firm]['best_params']['changepoint_range'] for firm in firm_ids],
                    [f"{firm_results[firm]['best_rmse']:.4f}" for firm in firm_ids],
                    [f"{firm_results[firm]['best_r2']:.4f}" for firm in firm_ids]
                ],
                fill_color='lavender',
                align='left'
            )
        )])

        summary_table.update_layout(title="Prophet Best Parameters by Firm")
        summary_table.write_html("prophet_parameters_by_firm.html")
        print("Prophet parameters by firm summary saved")

        if len(param_importance_dfs) > 1:
            try:
                all_params_df = pd.concat(param_importance_dfs)

                param_dist_fig = make_subplots(rows=2, cols=2,
                                               subplot_titles=["changepoint_prior_scale", "seasonality_prior_scale",
                                                               "holidays_prior_scale", "changepoint_range"],
                                               vertical_spacing=0.2)

                for i, param in enumerate(['changepoint_prior_scale', 'seasonality_prior_scale',
                                           'holidays_prior_scale', 'changepoint_range']):
                    row, col = i // 2 + 1, i % 2 + 1

                    param_groups = all_params_df.groupby(param)['rmse'].agg(['mean', 'median', 'std']).reset_index()
                    param_groups = param_groups.sort_values(param)

                    param_dist_fig.add_trace(
                        go.Bar(
                            x=param_groups[param].astype(str),
                            y=param_groups['mean'],
                            name="Mean RMSE",
                            marker_color='red',
                            error_y=dict(
                                type='data',
                                array=param_groups['std'],
                                visible=True
                            )
                        ),
                        row=row, col=col
                    )

                param_dist_fig.update_layout(
                    title="Parameter Value Impact on RMSE Across All Firms",
                    height=700,
                    showlegend=False
                )

                param_dist_fig.write_html("prophet_parameter_impact_all_firms.html")
                print("Cross-firm parameter impact visualization saved")

            except Exception as e:
                print(f"Error creating cross-firm parameter analysis: {e}")

    return firm_results


def build_firm_specific_ensemble(X_train, y_train, X_valid, y_valid,
                                 lgb_model, prophet_firm_results,
                                 weight_steps=21):
    """
    Build firm-specific ensemble models by tuning weights for each firm

    Args:
        X_train: Training features for LightGBM
        y_train: Target variable for training
        X_valid: Validation features for LightGBM
        y_valid: Validation target
        lgb_model: Trained LightGBM model
        prophet_firm_results: Dictionary of Prophet models by firm
        weight_steps: Number of weight combinations to try (resolution)

    Returns:
        Dictionary of firm-specific ensemble weights and metrics
    """
    print("\nTuning ensemble weights for each firm...")

    validation_firms = X_valid['FirmID'].unique()
    print(f"Validation set contains {len(validation_firms)} unique firms")

    lgb_preds = lgb_model.predict(X_valid)

    validation_df = X_valid.copy()
    validation_df['true_sales'] = y_valid.values
    validation_df['lgb_pred'] = lgb_preds
    validation_df['Date'] = df_valid['Date'] 

    weight_grid = np.linspace(0, 1, weight_steps) 

    ensemble_results = {}

    firm_weights = []
    firm_names = []
    firm_rmses = []
    firm_r2s = []

    for firm_id in prophet_firm_results.keys():
        print(f"\nTuning ensemble for Firm {firm_id}")

        firm_valid = validation_df[validation_df['FirmID'] == firm_id].copy()

        if len(firm_valid) == 0:
            print(f"  No validation data for Firm {firm_id}, skipping...")
            continue

        prophet_model = prophet_firm_results[firm_id]['best_model']

        prophet_valid = firm_valid.rename(columns={'Date': 'ds'})

        required_regressors = [col for col in prophet_model.extra_regressors.keys()
                               if col in prophet_valid.columns]

        try:
            with warnings.catch_warnings():
                warnings.filterwarnings("ignore")
                prophet_future = prophet_valid[['ds'] + required_regressors]

                for col in required_regressors:
                    if prophet_future[col].isna().any():
                        prophet_future[col] = prophet_future[col].fillna(prophet_future[col].median())

                prophet_forecast = prophet_model.predict(prophet_future)
                prophet_preds = prophet_forecast['yhat'].values

                firm_valid['prophet_pred'] = prophet_preds

                weight_results = []

                for lgb_weight in weight_grid:
                    prophet_weight = 1 - lgb_weight

                    ensemble_preds = (
                            lgb_weight * firm_valid['lgb_pred'].values +
                            prophet_weight * firm_valid['prophet_pred'].values
                    )

                    rmse = np.sqrt(mean_squared_error(firm_valid['true_sales'].values, ensemble_preds))
                    r2 = calculate_r2(firm_valid['true_sales'].values, ensemble_preds)

                    weight_results.append({
                        'lgb_weight': lgb_weight,
                        'prophet_weight': prophet_weight,
                        'rmse': rmse,
                        'r2': r2
                    })

                    print(
                        f"  LightGBM weight: {lgb_weight:.2f}, Prophet weight: {prophet_weight:.2f} - RMSE: {rmse:.4f}, R²: {r2:.4f}")

                best_result = min(weight_results, key=lambda x: x['rmse'])

                print(f"  Best ensemble weights for Firm {firm_id}:")
                print(f"    LightGBM: {best_result['lgb_weight']:.2f}, Prophet: {best_result['prophet_weight']:.2f}")
                print(f"    RMSE: {best_result['rmse']:.4f}, R²: {best_result['r2']:.4f}")

                ensemble_results[firm_id] = {
                    'weights': {
                        'lightgbm': best_result['lgb_weight'],
                        'prophet': best_result['prophet_weight']
                    },
                    'rmse': best_result['rmse'],
                    'r2': best_result['r2'],
                    'all_weight_results': weight_results
                }

                firm_weights.append(best_result['lgb_weight'])
                firm_names.append(f"Firm {firm_id}")
                firm_rmses.append(best_result['rmse'])
                firm_r2s.append(best_result['r2'])

                weight_df = pd.DataFrame(weight_results)

                firm_fig = make_subplots(rows=2, cols=1,
                                         subplot_titles=[f"Firm {firm_id} - Ensemble RMSE by Weight",
                                                         f"Firm {firm_id} - Ensemble R² by Weight"],
                                         vertical_spacing=0.1)

                firm_fig.add_trace(
                    go.Scatter(
                        x=weight_df['lgb_weight'],
                        y=weight_df['rmse'],
                        mode='lines+markers',
                        name="RMSE",
                        line=dict(color='red', width=2),
                        marker=dict(color='red', size=8)
                    ),
                    row=1, col=1
                )

                firm_fig.add_trace(
                    go.Scatter(
                        x=weight_df['lgb_weight'],
                        y=weight_df['r2'],
                        mode='lines+markers',
                        name="R²",
                        line=dict(color='blue', width=2),
                        marker=dict(color='blue', size=8)
                    ),
                    row=2, col=1
                )

                firm_fig.update_layout(
                    height=600,
                    title_text=f"Firm {firm_id} - Ensemble Weight Tuning",
                    showlegend=True,
                    xaxis_title="LightGBM Weight",
                    xaxis2_title="LightGBM Weight",
                    yaxis_title="RMSE",
                    yaxis2_title="R²"
                )

                firm_fig.write_html(f"ensemble_firm_{firm_id}_tuning.html")
                print(f"  Ensemble weight tuning visualization saved for Firm {firm_id}")

                comparison_fig = go.Figure()

                firm_valid = firm_valid.sort_values('Date')

                comparison_fig.add_trace(
                    go.Scatter(
                        x=firm_valid['Date'],
                        y=firm_valid['true_sales'],
                        mode='lines+markers',
                        name='Actual Sales',
                        line=dict(color='black', width=2)
                    )
                )

                comparison_fig.add_trace(
                    go.Scatter(
                        x=firm_valid['Date'],
                        y=firm_valid['lgb_pred'],
                        mode='lines',
                        name='LightGBM',
                        line=dict(color='blue', width=1.5, dash='dash')
                    )
                )

                comparison_fig.add_trace(
                    go.Scatter(
                        x=firm_valid['Date'],
                        y=firm_valid['prophet_pred'],
                        mode='lines',
                        name='Prophet',
                        line=dict(color='green', width=1.5, dash='dash')
                    )
                )

                ensemble_preds = (
                        best_result['lgb_weight'] * firm_valid['lgb_pred'].values +
                        best_result['prophet_weight'] * firm_valid['prophet_pred'].values
                )

                comparison_fig.add_trace(
                    go.Scatter(
                        x=firm_valid['Date'],
                        y=ensemble_preds,
                        mode='lines',
                        name='Ensemble',
                        line=dict(color='red', width=2)
                    )
                )

                comparison_fig.update_layout(
                    title=f"Firm {firm_id} - Model Predictions Comparison",
                    xaxis_title="Date",
                    yaxis_title="Sales",
                    template="plotly_white",
                    legend=dict(
                        orientation="h",
                        yanchor="bottom",
                        y=1.02,
                        xanchor="right",
                        x=1
                    )
                )

                comparison_fig.write_html(f"ensemble_firm_{firm_id}_comparison.html")
                print(f"  Model comparison visualization saved for Firm {firm_id}")

        except Exception as e:
            print(f"  Error tuning ensemble for Firm {firm_id}: {e}")

    if firm_names:
        fig = make_subplots(rows=3, cols=1,
                            subplot_titles=["Best LightGBM Weight by Firm",
                                            "Ensemble RMSE by Firm",
                                            "Ensemble R² by Firm"],
                            vertical_spacing=0.1)

        fig.add_trace(
            go.Bar(
                x=firm_names,
                y=firm_weights,
                name="LightGBM Weight",
                marker_color='purple',
                text=[f"{w:.2f}" for w in firm_weights],
                textposition="outside"
            ),
            row=1, col=1
        )

        fig.add_trace(
            go.Bar(
                x=firm_names,
                y=firm_rmses,
                name="RMSE",
                marker_color='red',
                text=[f"{r:.4f}" for r in firm_rmses],
                textposition="outside"
            ),
            row=2, col=1
        )

        fig.add_trace(
            go.Bar(
                x=firm_names,
                y=firm_r2s,
                name="R²",
                marker_color='blue',
                text=[f"{r:.4f}" for r in firm_r2s],
                textposition="outside"
            ),
            row=3, col=1
        )

        fig.update_layout(
            height=900,
            title_text="Ensemble Performance by Firm",
            template="plotly_white",
            showlegend=False
        )

        fig.write_html("ensemble_all_firms_comparison.html")
        print("\nEnsemble comparison across firms visualization saved")

        summary_table = go.Figure(data=[go.Table(
            header=dict(
                values=["Firm ID", "LightGBM Weight", "Prophet Weight", "RMSE", "R²", "Improvement over LightGBM"],
                fill_color='paleturquoise',
                align='left'
            ),
            cells=dict(
                values=[
                    [name.replace("Firm ", "") for name in firm_names],
                    [f"{w:.2f}" for w in firm_weights],
                    [f"{1 - w:.2f}" for w in firm_weights],
                    [f"{r:.4f}" for r in firm_rmses],
                    [f"{r:.4f}" for r in firm_r2s],
                    [
                        # Convert to float first
                        f"{((prophet_firm_results[float(name.replace('Firm ', ''))]['best_rmse'] - r) / prophet_firm_results[float(name.replace('Firm ', ''))]['best_rmse'] * 100):.2f}%"
                        if ((prophet_firm_results[float(name.replace('Firm ', ''))]['best_rmse'] - r) > 0) else
                        f"{((prophet_firm_results[float(name.replace('Firm ', ''))]['best_rmse'] - r) / prophet_firm_results[float(name.replace('Firm ', ''))]['best_rmse'] * 100):.2f}%"
                        for name, r in zip(firm_names, firm_rmses)
                    ]
                ],
                fill_color='lavender',
                align='left'
            )
        )])

        summary_table.update_layout(title="Ensemble Weights and Performance by Firm")
        summary_table.write_html("ensemble_weights_by_firm.html")
        print("Ensemble weights by firm summary saved")

    all_lgb_weights = [ensemble_results[firm_id]['weights']['lightgbm'] for firm_id in ensemble_results]
    all_prophet_weights = [ensemble_results[firm_id]['weights']['prophet'] for firm_id in ensemble_results]

    if all_lgb_weights:
        global_lgb_weight = np.mean(all_lgb_weights)
        global_prophet_weight = np.mean(all_prophet_weights)
    else:
        global_lgb_weight = 0.5
        global_prophet_weight = 0.5

    print(f"\nGlobal ensemble weights (for firms without specific models):")
    print(f"  LightGBM: {global_lgb_weight:.2f}, Prophet: {global_prophet_weight:.2f}")

    ensemble_results['global'] = {
        'weights': {
            'lightgbm': global_lgb_weight,
            'prophet': global_prophet_weight
        }
    }

    try:
        if len(ensemble_results) > 3:
            print("\nAnalyzing factors that influence optimal ensemble weights...")

            firm_data = []
            for firm_id in ensemble_results.keys():
                if firm_id != 'global':

                    firm_count = len(df_train[df_train['FirmID'] == firm_id])
                    firm_traffic_mean = df_train[df_train['FirmID'] == firm_id]['FirmDailyTraffic'].mean()
                    firm_traffic_std = df_train[df_train['FirmID'] == firm_id]['FirmDailyTraffic'].std()
                    firm_sales_mean = df_train[df_train['FirmID'] == firm_id]['Sales'].mean()
                    firm_sales_std = df_train[df_train['FirmID'] == firm_id]['Sales'].std()

                    firm_data.append({
                        'firm_id': firm_id,
                        'lgb_weight': ensemble_results[firm_id]['weights']['lightgbm'],
                        'data_points': firm_count,
                        'traffic_mean': firm_traffic_mean,
                        'traffic_std': firm_traffic_std,
                        'sales_mean': firm_sales_mean,
                        'sales_std': firm_sales_std,
                        'traffic_variability': firm_traffic_std / firm_traffic_mean if firm_traffic_mean > 0 else 0,
                        'sales_variability': firm_sales_std / firm_sales_mean if firm_sales_mean > 0 else 0
                    })

            if firm_data:
                firm_df = pd.DataFrame(firm_data)

                correlations = firm_df.corr()['lgb_weight'].drop('lgb_weight')

                corr_fig = go.Figure()

                corr_fig.add_trace(
                    go.Bar(
                        x=correlations.index,
                        y=correlations.values,
                        marker_color=['blue' if v >= 0 else 'red' for v in correlations.values]
                    )
                )

                corr_fig.update_layout(
                    title="Factors Correlated with Optimal LightGBM Weight",
                    xaxis_title="Factor",
                    yaxis_title="Correlation Coefficient",
                    template="plotly_white"
                )

                corr_fig.write_html("weight_correlations.html")
                print("Weight correlation analysis saved")

                print("\nInsights about optimal ensemble weights:")
                for factor, corr in correlations.items():
                    direction = "higher" if corr > 0 else "lower"
                    strength = "strong" if abs(corr) > 0.7 else "moderate" if abs(corr) > 0.3 else "weak"
                    if abs(corr) > 0.2:  # Only report meaningful correlations
                        print(f"  {factor}: {strength} {direction} correlation ({corr:.2f}) with LightGBM weight")

    except Exception as e:
        print(f"Error analyzing weight correlations: {e}")

    return ensemble_results

print("\nBeginning focused multi-firm model tuning process...")
print_memory_usage()

print(f"Creating stratified firm sample for LightGBM tuning...")
sample_idx = stratified_firm_sample(X_train, y_train, max_total=10000)
X_train_sample = X_train.iloc[sample_idx]
y_train_sample = y_train.iloc[sample_idx]

print(f"Stratified sample for LightGBM tuning: {len(X_train_sample)} rows")
print(f"Number of firms in sample: {X_train_sample['FirmID'].nunique()} (out of {X_train['FirmID'].nunique()} total)")

lgb_best_params, lgb_best_rmse, lgb_best_r2, lgb_model = tune_lightgbm_optuna(
    X_train_sample, y_train_sample, X_valid, y_valid, tscv, n_trials=50
)
print_memory_usage()

top_firms = df_train['FirmID'].value_counts().iloc[:10].index.tolist()
prophet_firm_results = tune_prophet_for_firms(
    df_train, top_firms=top_firms, param_combinations=15, max_firms=10
)
print_memory_usage()

ensemble_weights = build_firm_specific_ensemble(
    X_train, y_train, X_valid, y_valid, lgb_model, prophet_firm_results
)
print_memory_usage()

print("\n" + "=" * 50)
print("SUMMARY OF MULTI-FIRM MODEL TUNING RESULTS")
print("=" * 50)

print(f"LightGBM global model:")
print(f"  Best parameters: {lgb_best_params}")
print(f"  Validation RMSE: {lgb_best_rmse:.4f}")
print(f"  Validation R²: {lgb_best_r2:.4f}")

print(f"\nProphet models:")
print(f"  Number of firm-specific models: {len(prophet_firm_results)}")
firm_prophet_rmses = []

for firm_id, results in prophet_firm_results.items():
    print(f"  Firm {firm_id}:")
    print(f"    Best parameters: {results['best_params']}")
    print(f"    Validation RMSE: {results['best_rmse']:.4f}")
    print(f"    Validation R²: {results['best_r2']:.4f}")
    firm_prophet_rmses.append(results['best_rmse'])

print(f"\nEnsemble weights:")
print(f"  Global weights: LightGBM={ensemble_weights['global']['weights']['lightgbm']:.2f}, "
      f"Prophet={ensemble_weights['global']['weights']['prophet']:.2f}")

firm_ensemble_rmses = []
for firm_id, results in ensemble_weights.items():
    if firm_id != 'global':
        print(f"  Firm {firm_id}: LightGBM={results['weights']['lightgbm']:.2f}, "
              f"Prophet={results['weights']['prophet']:.2f}, RMSE={results.get('rmse', 'N/A')}")
        if 'rmse' in results:
            firm_ensemble_rmses.append(results['rmse'])

if firm_prophet_rmses and firm_ensemble_rmses:
    avg_prophet_rmse = np.mean(firm_prophet_rmses)
    avg_ensemble_rmse = np.mean(firm_ensemble_rmses)
    improvement = (avg_prophet_rmse - avg_ensemble_rmse) / avg_prophet_rmse * 100

    print(f"\nOverall ensemble improvement:")
    print(f"  Average Prophet-only RMSE: {avg_prophet_rmse:.4f}")
    print(f"  Average Ensemble RMSE: {avg_ensemble_rmse:.4f}")
    print(f"  Average improvement: {improvement:.2f}%")

try:
    model_comparison = go.Figure()

    firm_names = [f"Firm {firm_id}" for firm_id in prophet_firm_results.keys() if firm_id in ensemble_weights]
    lgb_rmses = [lgb_best_rmse] * len(firm_names)
    prophet_rmses = [prophet_firm_results[float(name.replace('Firm ', ''))]['best_rmse'] for name in firm_names]
    ensemble_rmses = [ensemble_weights[float(name.replace('Firm ', ''))]['rmse'] for name in firm_names]

    model_comparison.add_trace(
        go.Bar(
            x=firm_names,
            y=lgb_rmses,
            name='LightGBM',
            marker_color='blue',
            text=[f"{r:.4f}" for r in lgb_rmses],
            textposition="outside"
        )
    )

    model_comparison.add_trace(
        go.Bar(
            x=firm_names,
            y=prophet_rmses,
            name='Prophet',
            marker_color='green',
            text=[f"{r:.4f}" for r in prophet_rmses],
            textposition="outside"
        )
    )

    model_comparison.add_trace(
        go.Bar(
            x=firm_names,
            y=ensemble_rmses,
            name='Ensemble',
            marker_color='red',
            text=[f"{r:.4f}" for r in ensemble_rmses],
            textposition="outside"
        )
    )

    model_comparison.update_layout(
        title="RMSE Comparison Across All Models by Firm",
        xaxis_title="Firm",
        yaxis_title="RMSE (lower is better)",
        template="plotly_white",
        barmode='group',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )

    model_comparison.write_html("model_rmse_comparison.html")
    print("\nModel RMSE comparison visualization saved")

    all_tuning_results = {
        'lightgbm': {
            'best_params': lgb_best_params,
            'validation_rmse': lgb_best_rmse,
            'validation_r2': lgb_best_r2
        },
        'prophet': {
            firm_id: {
                'best_params': results['best_params'],
                'validation_rmse': results['best_rmse'],
                'validation_r2': results['best_r2']
            } for firm_id, results in prophet_firm_results.items()
        },
        'ensemble': {
            firm_id: {
                'weights': results['weights'],
                'validation_rmse': results.get('rmse', 'N/A'),
                'validation_r2': results.get('r2', 'N/A')
            } for firm_id, results in ensemble_weights.items()
        },
        'overall_improvement': {
            'avg_prophet_rmse': avg_prophet_rmse if 'avg_prophet_rmse' in locals() else None,
            'avg_ensemble_rmse': avg_ensemble_rmse if 'avg_ensemble_rmse' in locals() else None,
            'improvement_percentage': improvement if 'improvement' in locals() else None
        }
    }

    with open('multi_firm_tuning_results.json', 'w') as f:
        json.dump(all_tuning_results, f, indent=4, default=str)

    print("All tuning results saved to multi_firm_tuning_results.json")

except Exception as e:
    print(f"Error creating final visualizations: {e}")
    print("Full traceback:")
    traceback.print_exc()
    

print("\nMulti-firm model tuning process completed successfully.")


FOCUSED MULTI-FIRM HYPERPARAMETER TUNING
Using existing training data from feature engineering step
Creating validation split while preserving time series structure...
Training set: 28100 rows, Validation set: 7050 rows
Number of firms in training: 50
Number of firms in validation: 50

Beginning focused multi-firm model tuning process...
Current memory usage: 307.74 MB
Creating stratified firm sample for LightGBM tuning...


[I 2025-03-02 11:48:00,590] A new study created in memory with name: no-name-5acf8f84-e9ab-4111-aa3d-297064476526


Stratified sample for LightGBM tuning: 10000 rows
Number of firms in sample: 50 (out of 50 total)

Tuning LightGBM hyperparameters using Optuna...


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-03-02 11:48:01,615] Trial 0 finished with value: 791.4113391899431 and parameters: {'lambda_l1': 0.13292918943162169, 'lambda_l2': 7.114476009343421, 'num_leaves': 79, 'feature_fraction': 0.779597545259111, 'bagging_fraction': 0.6468055921327309, 'bagging_freq': 1, 'min_child_samples': 12, 'learning_rate': 0.08795585311974417, 'max_depth': 6, 'n_estimators': 383}. Best is trial 0 with value: 791.4113391899431.
[I 2025-03-02 11:48:02,304] Trial 1 finished with value: 785.9678842926689 and parameters: {'lambda_l1': 0.011527987128232402, 'lambda_l2': 8.123245085588687, 'num_leaves': 87, 'feature_fraction': 0.6637017332034828, 'bagging_fraction': 0.6545474901621302, 'bagging_freq': 1, 'min_child_samples': 22, 'learning_rate': 0.05722807884690141, 'max_depth': 5, 'n_estimators': 216}. Best is trial 1 with value: 785.9678842926689.
[I 2025-03-02 11:48:02,987] Trial 2 finished with value: 791.5256689270853 and parameters: {'lambda_l1': 0.6847920095574779, 'lambda_l2': 0.02621087878265

...

  Parameters {'changepoint_prior_scale': 0.05, 'seasonality_prior_scale': 0.1, 'holidays_prior_scale': 1.0, 'seasonality_mode': 'additive', 'changepoint_range': 0.8} - RMSE: 835.0006, R²: -4.8659


11:49:18 - cmdstanpy - INFO - Chain [1] done processing
11:49:18 - cmdstanpy - INFO - Chain [1] start processing


  Parameters {'changepoint_prior_scale': 0.1, 'seasonality_prior_scale': 1.0, 'holidays_prior_scale': 0.01, 'seasonality_mode': 'additive', 'changepoint_range': 0.8} - RMSE: 829.3860, R²: -4.7873


11:49:18 - cmdstanpy - INFO - Chain [1] done processing


  Parameters {'changepoint_prior_scale': 0.05, 'seasonality_prior_scale': 0.1, 'holidays_prior_scale': 0.1, 'seasonality_mode': 'additive', 'changepoint_range': 0.95} - RMSE: 831.4445, R²: -4.8161
  Best Prophet parameters for Firm 36.0: {'changepoint_prior_scale': 0.001, 'seasonality_prior_scale': 0.01, 'holidays_prior_scale': 10.0, 'seasonality_mode': 'additive', 'changepoint_range': 0.8}
  Best RMSE: 611.9192
  Best validation R²: -2.1503
  Parameter importance visualization saved for Firm 36.0

Prophet predictions by firm visualization saved
Prophet parameters by firm summary saved
Cross-firm parameter impact visualization saved
Current memory usage: 343.06 MB

Tuning ensemble weights for each firm...
Validation set contains 50 unique firms

Tuning ensemble for Firm 0.0
  Error tuning ensemble for Firm 0.0: Regressor 'TrafficPctChange' missing from dataframe

Tuning ensemble for Firm 30.0
  Error tuning ensemble for Firm 30.0: Regressor 'TrafficPctChange' missing from dataframe

Tu

# Advanced Models with Tuned Hyperparameters and Forecast Save

In [27]:
print("\nPreparing forecast dataframe...")
df_forecast = df_test.copy()

# Use the simplified feature set to match the training approach
volatility_features = [
    'FirmID', 'FirmDailyTraffic',  # Core features
    'DayOfWeek', 'Month', 'IsWeekend',  # Time components
    'Sales_lag1', 'Sales_lag7'  # Minimal lag features
]
X_forecast = df_forecast[volatility_features]

cutoff_date = df_train['Date'].max()
print(f"Cutoff date for forecasting: {cutoff_date}")

model_results = {}

#-----------------------------------------------------------
# 9.1. LightGBM Model with Simplified Features
#-----------------------------------------------------------
print("\nTraining LGBM model with simplified features...")
print_memory_usage()

try:
    # Prepare categorical features
    categorical_columns = ['FirmID', 'DayOfWeek', 'Month', 'IsWeekend']
    for col in categorical_columns:
        if col in X_train.columns:
            X_train[col] = X_train[col].astype(int)
            X_forecast[col] = X_forecast[col].astype(int)
    
    categorical_features = [X_train.columns.get_loc(col) for col in categorical_columns 
                          if col in X_train.columns]
    
    lgbm_model = lgb.LGBMRegressor(**lgb_best_params, 
                                   random_state=42, 
                                   categorical_feature=categorical_features)
    lgbm_model.fit(X_train, y_train)

    #-----------------------------------------------------------
    # Implement rolling forecast approach for LightGBM
    #-----------------------------------------------------------
    print("\nImplementing rolling forecast for LightGBM...")
    for firm_id in df_forecast['FirmID'].unique():
        print(f"Processing firm {firm_id} with rolling forecast...")
        
        # Get firm-specific data sorted by date
        firm_forecast = df_forecast[df_forecast['FirmID'] == firm_id].sort_values('Date')
        
        if len(firm_forecast) == 0:
            continue
            
        # Get latest values from training data
        firm_train = df_train[df_train['FirmID'] == firm_id].sort_values('Date')
        
        if len(firm_train) > 0:
            # Initialize lag values from training data
            last_sales = firm_train['Sales'].iloc[-1] if len(firm_train) > 0 else 0
            last_sales_7 = firm_train['Sales'].iloc[-7] if len(firm_train) >= 7 else last_sales
        else:
            last_sales = 0
            last_sales_7 = 0
            
        # Process each date in sequence to update lag features
        for i, (idx, row) in enumerate(firm_forecast.iterrows()):
            # Update lag1 feature with the previous prediction or training value
            if i == 0:  # First forecast day
                df_forecast.loc[idx, 'Sales_lag1'] = last_sales
                df_forecast.loc[idx, 'Sales_lag7'] = last_sales_7
            else:
                # Use previous day's prediction as lag1
                prev_idx = firm_forecast.index[i-1]
                df_forecast.loc[idx, 'Sales_lag1'] = df_forecast.loc[prev_idx, 'Sales_pred_lgbm']
                
                # Update lag7 feature if available
                if i >= 7:
                    week_ago_idx = firm_forecast.index[i-7]
                    df_forecast.loc[idx, 'Sales_lag7'] = df_forecast.loc[week_ago_idx, 'Sales_pred_lgbm']
                elif len(firm_train) > 0:
                    # Use training data for the lag7 if available
                    day_diff = (row['Date'] - firm_train['Date'].iloc[-1]).days
                    if day_diff <= 7:
                        lag7_idx = day_diff - 7
                        if abs(lag7_idx) < len(firm_train):
                            df_forecast.loc[idx, 'Sales_lag7'] = firm_train['Sales'].iloc[lag7_idx]
            
            # Make prediction for current row with updated features
            X_row = df_forecast.loc[idx:idx, volatility_features]
            df_forecast.loc[idx, 'Sales_pred_lgbm'] = lgbm_model.predict(X_row)[0]
            
            # Store prediction for use in next iteration
            last_sales = df_forecast.loc[idx, 'Sales_pred_lgbm']

    lgbm_cv_scores = []
    for train_idx, val_idx in tscv.split(X_train):
        X_train_cv, X_val_cv = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_train_cv, y_val_cv = y_train.iloc[train_idx], y_train.iloc[val_idx]
        
        lgbm_cv = lgb.LGBMRegressor(**lgb_best_params, 
                                    random_state=42, 
                                    categorical_feature=categorical_features)
        lgbm_cv.fit(X_train_cv, y_train_cv)

        y_pred_cv = lgbm_cv.predict(X_val_cv)
        mae = mean_absolute_error(y_val_cv, y_pred_cv)
        rmse = np.sqrt(mean_squared_error(y_val_cv, y_pred_cv))
        r2 = r2_score(y_val_cv, y_pred_cv)
        lgbm_cv_scores.append((mae, rmse, r2))

    lgbm_mean_mae = np.mean([score[0] for score in lgbm_cv_scores])
    lgbm_mean_rmse = np.mean([score[1] for score in lgbm_cv_scores])
    lgbm_mean_r2 = np.mean([score[2] for score in lgbm_cv_scores])
    print(f"LightGBM cross-validation - Mean MAE: {lgbm_mean_mae:.4f}, Mean RMSE: {lgbm_mean_rmse:.4f}, Mean R²: {lgbm_mean_r2:.4f}")

    model_results['LightGBM'] = {
        'cv_mae': lgbm_mean_mae,
        'cv_rmse': lgbm_mean_rmse,
        'cv_r2': lgbm_mean_r2,
        'feature_importance': dict(zip(volatility_features, lgbm_model.feature_importances_))
    }

    importances = model_results['LightGBM']['feature_importance']
    top_features = sorted(importances.items(), key=lambda x: x[1], reverse=True)[:5]
    print("Top 5 important features for LightGBM:")
    for feature, importance in top_features:
        print(f"  {feature}: {importance:.4f}")

    try:
        sorted_features = sorted(importances.items(), key=lambda x: x[1], reverse=True)
        features_names = [item[0] for item in sorted_features]
        feature_values = [item[1] for item in sorted_features]

        fig = px.bar(
            x=feature_values,
            y=features_names,
            orientation='h',
            title='LightGBM Feature Importance',
            labels={'x': 'Importance', 'y': 'Feature'},
            color=feature_values,
            color_continuous_scale='Viridis'
        )

        fig.update_layout(
            height=800,
            width=1000,
            template='plotly_white',
            yaxis={'categoryorder': 'total ascending'}
        )

        fig.write_html('lightgbm_feature_importance.html')
        print("LightGBM feature importance visualization saved to 'lightgbm_feature_importance.html'")
    except Exception as e:
        print(f"Error creating feature importance visualization: {e}")

    try:
        sample_size = min(1000, len(X_train))
        sample_idx = np.random.choice(len(X_train), sample_size, replace=False)
        X_sample = X_train.iloc[sample_idx]
        y_sample = y_train.iloc[sample_idx]
        y_pred_sample = lgbm_model.predict(X_sample)
        
        fig = px.scatter(
            x=y_sample,
            y=y_pred_sample,
            opacity=0.6,
            title=f'LightGBM: Actual vs Predicted Sales (R² = {lgbm_mean_r2:.4f})',
            labels={'x': 'Actual Sales', 'y': 'Predicted Sales'}
        )

        max_val = max(max(y_sample), max(y_pred_sample))
        min_val = min(min(y_sample), min(y_pred_sample))
        
        fig.add_trace(
            go.Scatter(
                x=[min_val, max_val],
                y=[min_val, max_val],
                mode='lines',
                line=dict(color='red', dash='dash'),
                name='Perfect Prediction'
            )
        )
        
        fig.update_layout(template='plotly_white')
        fig.write_html('lightgbm_actual_vs_predicted.html')
        print("LightGBM actual vs predicted plot saved")
    except Exception as e:
        print(f"Error creating LightGBM actual vs predicted plot: {e}")

except Exception as e:
    print(f"Error in LightGBM model training: {e}")

    df_forecast['Sales_pred_lgbm'] = np.nan
    lgbm_mean_r2 = 0
    
print_memory_usage()

#-----------------------------------------------------------
# 9.2. Prophet Model with Simplified Regressors
#-----------------------------------------------------------
print("\nTraining Prophet models for each firm with simplified regressors...")

prophet_forecasts = []
prophet_components = []

# Simplified regressors list
prophet_regressors = [
    'FirmDailyTraffic',
    'TrafficPctChange'
]

for firm_id in df_model['FirmID'].unique():
    try:
        print(f"\nTraining Prophet model for Firm {firm_id}...")
        firm_train = df_train[df_train['FirmID'] == firm_id].copy()
        firm_forecast = df_forecast[df_forecast['FirmID'] == firm_id].copy()

        if len(firm_train) < 10:
            print(f"  Skipping Firm {firm_id}: insufficient data ({len(firm_train)} rows)")
            continue

        prophet_train = firm_train[['Date', 'Sales']].rename(columns={'Date': 'ds', 'Sales': 'y'})

        for regressor in prophet_regressors:
            if regressor in firm_train.columns:
                prophet_train[regressor] = firm_train[regressor].fillna(firm_train[regressor].median())

        if firm_id in prophet_firm_results:
            firm_params = prophet_firm_results[firm_id]['best_params']
            print(f"  Using firm-specific Prophet parameters: {firm_params}")
        else:
            firm_params = {
                'changepoint_prior_scale': 0.05,
                'seasonality_prior_scale': 10.0, 
                'holidays_prior_scale': 10.0,
                'seasonality_mode': 'additive',
                'changepoint_range': 0.8
            }
            print(f"  Using default Prophet parameters: {firm_params}")

        prophet_model = Prophet(
            changepoint_prior_scale=firm_params['changepoint_prior_scale'],
            seasonality_prior_scale=firm_params['seasonality_prior_scale'],
            holidays_prior_scale=firm_params['holidays_prior_scale'],
            seasonality_mode=firm_params['seasonality_mode'],
            changepoint_range=firm_params['changepoint_range'],
            yearly_seasonality=True,
            weekly_seasonality=True,
            daily_seasonality=False
        )

        for regressor in prophet_regressors:
            if regressor in prophet_train.columns:
                prophet_model.add_regressor(regressor)

        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            prophet_model.fit(prophet_train)

        # Create future dataframe for predictions
        future = firm_forecast[['Date']].rename(columns={'Date': 'ds'})

        for regressor in prophet_regressors:
            if regressor in firm_forecast.columns:
                future[regressor] = firm_forecast[regressor].values

        forecast = prophet_model.predict(future)

        firm_forecast['Sales_pred_prophet'] = forecast['yhat'].values
        prophet_forecasts.append(firm_forecast)

        if len(prophet_components) < 3: 
            components_df = pd.DataFrame({
                'Date': forecast['ds'],
                'Trend': forecast['trend'],
                'Weekly': forecast['weekly'],
                'Yearly': forecast['yearly'] if 'yearly' in forecast.columns else 0,
                'FirmID': firm_id
            })

            for regressor in prophet_regressors:
                regressor_effect = f'{regressor}_effect'
                if regressor_effect in forecast.columns:
                    components_df[f'{regressor}_Effect'] = forecast[regressor_effect]
            
            prophet_components.append(components_df)
            
        print(f"  Prophet model for Firm {firm_id} successfully trained")
        
    except Exception as e:
        print(f"  Error training Prophet model for Firm {firm_id}: {e}")
        if 'firm_forecast' in locals():
            firm_forecast['Sales_pred_prophet'] = np.nan
            prophet_forecasts.append(firm_forecast)

if prophet_components:
    try:
        from plotly.subplots import make_subplots
        import plotly.graph_objects as go
        
        components_df = pd.concat(prophet_components)

        component_cols = [col for col in components_df.columns if col not in ['Date', 'FirmID']]

        fig = make_subplots(
            rows=len(component_cols), cols=1,
            subplot_titles=component_cols,
            vertical_spacing=0.1,
            shared_xaxes=True
        )
        
        for i, component in enumerate(component_cols):
            for firm_id in components_df['FirmID'].unique():
                firm_data = components_df[components_df['FirmID'] == firm_id]
                
                fig.add_trace(
                    go.Scatter(
                        x=firm_data['Date'],
                        y=firm_data[component],
                        mode='lines',
                        name=f'Firm {firm_id} - {component}',
                        line=dict(width=2)
                    ),
                    row=i+1, col=1
                )
        
        fig.update_layout(
            height=300 * len(component_cols),
            width=1000,
            title_text="Prophet Model Components",
            template='plotly_white',
            showlegend=True,
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
        )
        
        fig.write_html('prophet_components.html')
        print(f"Prophet components visualization saved to 'prophet_components.html'")
    except Exception as e:
        print(f"Error creating Prophet components visualization: {e}")

print("\nCombining Prophet forecasts...")
try:
    df_prophet_forecast = pd.concat(prophet_forecasts)
    df_forecast = df_forecast.merge(
        df_prophet_forecast[['FirmID', 'Date', 'Sales_pred_prophet']],
        on=['FirmID', 'Date'],
        how='left'
    )

    missing_prophet_firms = df_forecast[df_forecast['Sales_pred_prophet'].isna()]['FirmID'].unique()
    if len(missing_prophet_firms) > 0:
        print(f"Warning: {len(missing_prophet_firms)} firms have no Prophet predictions.")
        print(f"First few firms without Prophet predictions: {missing_prophet_firms[:5]}")
except Exception as e:
    print(f"Error combining Prophet forecasts: {e}")
    df_forecast['Sales_pred_prophet'] = np.nan

print("Prophet models trained for all firms with simplified regressors")
print_memory_usage()

#-----------------------------------------------------------
# 9.3. OLS Model Predictions
#-----------------------------------------------------------
print("\nIntegrating OLS model predictions...")

if 'Sales_pred_ols' not in df_forecast.columns:
    try:
        if 'firm_models' in globals() or 'firm_models' in locals():
            print("Using firm-specific OLS models")

            df_forecast['Sales_pred_ols'] = np.nan

            for firm_id, model in firm_models.items():
                firm_mask = df_forecast['FirmID'] == firm_id
                if firm_mask.sum() > 0:
                    try:
                        firm_forecast = df_forecast[firm_mask]
                        df_forecast.loc[firm_mask, 'Sales_pred_ols'] = model.predict(firm_forecast)
                    except Exception as e:
                        print(f"Error predicting with OLS model for Firm {firm_id}: {e}")

            if 'ols_model' in globals() or 'ols_model' in locals():
                missing_mask = df_forecast['Sales_pred_ols'].isna()
                if missing_mask.sum() > 0:
                    print(f"Using global OLS model for {missing_mask.sum()} rows without firm-specific predictions")
                    df_forecast.loc[missing_mask, 'Sales_pred_ols'] = ols_model.predict(df_forecast[missing_mask])
        
        elif 'ols_model' in globals() or 'ols_model' in locals():
            print("Using global OLS model for all firms")
            df_forecast['Sales_pred_ols'] = ols_model.predict(df_forecast)
        
        else:
            print("No OLS models available, skipping OLS predictions")
            df_forecast['Sales_pred_ols'] = np.nan
    
    except Exception as e:
        print(f"Error generating OLS predictions: {e}")
        df_forecast['Sales_pred_ols'] = np.nan
else:
    print("OLS predictions already present in forecast dataframe")

ols_coverage = (df_forecast['Sales_pred_ols'].notna().sum() / len(df_forecast)) * 100
print(f"OLS predictions available for {ols_coverage:.1f}% of forecast rows")
print_memory_usage()

#-----------------------------------------------------------
# 9.4. Ensemble Model: Combining Predictions
#-----------------------------------------------------------
print("\nCreating ensemble forecast...")

models_to_ensemble = []
for model_col in ['Sales_pred_ols', 'Sales_pred_lgbm', 'Sales_pred_prophet']:
    if model_col in df_forecast.columns and df_forecast[model_col].notna().any():
        models_to_ensemble.append(model_col)

print(f"Models available for ensemble: {models_to_ensemble}")

df_forecast['Sales_pred_ensemble'] = np.nan

if 'ensemble_weights' in globals() or 'ensemble_weights' in locals():
    print("Using weights from hyperparameter tuning for ensemble")

    for firm_id in df_forecast['FirmID'].unique():
        firm_mask = df_forecast['FirmID'] == firm_id
        
        if firm_id in ensemble_weights and firm_id != 'global':
            weights = ensemble_weights[firm_id]['weights']
            print(f"Using firm-specific weights for Firm {firm_id}: {weights}")
        else:
            weights = ensemble_weights['global']['weights']

        firm_preds = df_forecast.loc[firm_mask]

        lgbm_available = 'Sales_pred_lgbm' in firm_preds.columns and not firm_preds['Sales_pred_lgbm'].isna().all()
        prophet_available = 'Sales_pred_prophet' in firm_preds.columns and not firm_preds['Sales_pred_prophet'].isna().all()
        
        if lgbm_available and prophet_available:
            df_forecast.loc[firm_mask, 'Sales_pred_ensemble'] = (
                weights['lightgbm'] * firm_preds['Sales_pred_lgbm'] +
                weights['prophet'] * firm_preds['Sales_pred_prophet']
            )
        elif lgbm_available:
            df_forecast.loc[firm_mask, 'Sales_pred_ensemble'] = firm_preds['Sales_pred_lgbm']
        elif prophet_available:
            df_forecast.loc[firm_mask, 'Sales_pred_ensemble'] = firm_preds['Sales_pred_prophet']
        else:
            if 'Sales_pred_ols' in firm_preds.columns and not firm_preds['Sales_pred_ols'].isna().all():
                df_forecast.loc[firm_mask, 'Sales_pred_ensemble'] = firm_preds['Sales_pred_ols']
else:
    print("No ensemble weights from hyperparameter tuning, using default weights")

    model_weights = {
        'Sales_pred_ols': 0.1,      
        'Sales_pred_lgbm': 0.4,     
        'Sales_pred_prophet': 0.5,  
    }

    available_models = [m for m in model_weights.keys() if m in models_to_ensemble]
    weights_sum = sum([model_weights[m] for m in available_models])
    normalized_weights = {m: model_weights[m]/weights_sum for m in available_models}
    
    print("Default ensemble weights:")
    for model, weight in normalized_weights.items():
        print(f"  {model}: {weight:.2f}")

    df_forecast['Sales_pred_ensemble'] = 0.0
    for model, weight in normalized_weights.items():
        temp_values = df_forecast[model].fillna(0)
        df_forecast['Sales_pred_ensemble'] += temp_values * weight

    all_nan_mask = True
    for model in available_models:
        all_nan_mask = all_nan_mask & df_forecast[model].isna()
    df_forecast.loc[all_nan_mask, 'Sales_pred_ensemble'] = np.nan

ensemble_coverage = (df_forecast['Sales_pred_ensemble'].notna().sum() / len(df_forecast)) * 100
print(f"Ensemble predictions available for {ensemble_coverage:.1f}% of forecast rows")
print_memory_usage()

#-----------------------------------------------------------
# 9.5. Model Evaluation and Comparison
#-----------------------------------------------------------
print("\nComparing model performance...")

if 'Sales' in df_forecast.columns and df_forecast['Sales'].notna().any():
    metrics = {}
    for model in ['Sales_pred_ols', 'Sales_pred_lgbm', 'Sales_pred_prophet', 'Sales_pred_ensemble']:
        if model in df_forecast.columns and df_forecast[model].notna().any():
            mask = df_forecast['Sales'].notna() & df_forecast[model].notna()
            if mask.sum() > 0:
                y_true = df_forecast.loc[mask, 'Sales']
                y_pred = df_forecast.loc[mask, model]
                
                mae = mean_absolute_error(y_true, y_pred)
                rmse = np.sqrt(mean_squared_error(y_true, y_pred))
                r2 = r2_score(y_true, y_pred)

                mape = np.mean(np.abs((y_true - y_pred) / y_true.clip(lower=1))) * 100  # Avoid div by 0
                median_ae = median_absolute_error(y_true, y_pred)
                
                metrics[model] = {
                    'MAE': mae, 
                    'RMSE': rmse, 
                    'R²': r2,
                    'MAPE': mape,
                    'MedianAE': median_ae
                }

    print("\nForecast period evaluation metrics:")
    print(f"{'Model':<20} {'MAE':<12} {'RMSE':<12} {'R²':<8} {'MAPE':<8} {'MedianAE':<8}")
    print("-" * 65)
    for model, values in metrics.items():
        print(f"{model:<20} {values['MAE']:<12.4f} {values['RMSE']:<12.4f} {values['R²']:<8.4f} {values['MAPE']:<8.2f}% {values['MedianAE']:<8.4f}")

    if metrics:
        best_model = min(metrics.items(), key=lambda x: x[1]['RMSE'])[0]
        print(f"\nBest performing model: {best_model}")
        final_model = best_model
    else:
        print("No metrics available for evaluation, defaulting to ensemble")
        final_model = 'Sales_pred_ensemble'

    try:
        metrics_long = []
        for model, metrics_dict in metrics.items():
            for metric, value in metrics_dict.items():
                if metric not in ['MAPE', 'MedianAE']:  
                    metrics_long.append({
                        'Model': model.replace('Sales_pred_', ''),
                        'Metric': metric,
                        'Value': value
                    })
        
        metrics_df = pd.DataFrame(metrics_long)

        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=("RMSE (lower is better)", "MAE (lower is better)", 
                          "R² (higher is better)", "Model Comparison"),
            specs=[[{"type": "bar"}, {"type": "bar"}],
                  [{"type": "bar"}, {"type": "scatter"}]]
        )

        rmse_data = metrics_df[metrics_df['Metric'] == 'RMSE']
        fig.add_trace(
            go.Bar(
                x=rmse_data['Model'],
                y=rmse_data['Value'],
                marker_color='red',
                name='RMSE',
                text=[f"{v:.2f}" for v in rmse_data['Value']],
                textposition="outside"
            ),
            row=1, col=1
        )

        mae_data = metrics_df[metrics_df['Metric'] == 'MAE']
        fig.add_trace(
            go.Bar(
                x=mae_data['Model'],
                y=mae_data['Value'],
                marker_color='blue',
                name='MAE',
                text=[f"{v:.2f}" for v in mae_data['Value']],
                textposition="outside"
            ),
            row=1, col=2
        )

        r2_data = metrics_df[metrics_df['Metric'] == 'R²']
        fig.add_trace(
            go.Bar(
                x=r2_data['Model'],
                y=r2_data['Value'],
                marker_color='green',
                name='R²',
                text=[f"{v:.3f}" for v in r2_data['Value']],
                textposition="outside"
            ),
            row=2, col=1
        )

        model_names = list(metrics.keys())
        model_display_names = [name.replace('Sales_pred_', '') for name in model_names]

        max_rmse = max([m['RMSE'] for m in metrics.values()]) * 1.1 
        max_mae = max([m['MAE'] for m in metrics.values()]) * 1.1
        
        normalized_metrics = []
        for model in model_names:
            normalized_metrics.append({
                'Model': model.replace('Sales_pred_', ''),
                'RMSE': 1 - (metrics[model]['RMSE'] / max_rmse),  
                'MAE': 1 - (metrics[model]['MAE'] / max_mae),     
                'R²': max(0, metrics[model]['R²'])               
            })

        for model_data in normalized_metrics:
            model_name = model_data['Model']
            fig.add_trace(
                go.Scatterpolar(
                    r=[model_data['RMSE'], model_data['MAE'], model_data['R²']],
                    theta=['RMSE', 'MAE', 'R²'],
                    fill='toself',
                    name=model_name
                ),
                row=2, col=2
            )

        fig.update_layout(
            polar=dict(
                radialaxis=dict(
                    visible=True,
                    range=[0, 1]
                )
            ),
            showlegend=True
        )

        best_model_name = best_model.replace('Sales_pred_', '')

        for i, metric in enumerate(['RMSE', 'MAE', 'R²']):
            best_for_metric = min(metrics.items(), key=lambda x: x[1][metric])[0] if metric in ['RMSE', 'MAE'] else max(metrics.items(), key=lambda x: x[1][metric])[0]
            best_name = best_for_metric.replace('Sales_pred_', '')
            
            row, col = (1, 1) if metric == 'RMSE' else ((1, 2) if metric == 'MAE' else (2, 1))
            
            fig.add_annotation(
                x=best_name,
                y=metrics[best_for_metric][metric] * 1.1,  # Position above the chart
                text="Best for " + metric,
                showarrow=True,
                arrowhead=1,
                xref=f"x{i+1}" if i > 0 else "x",
                yref=f"y{i+1}" if i > 0 else "y",
                font=dict(size=12, color="green")
            )
        
        fig.update_layout(
            height=800,
            width=1000,
            template='plotly_white',
            title_text="Model Performance Comparison",
            showlegend=True
        )
        
        fig.write_html('model_performance_comparison.html')
        print("Model performance comparison visualization saved to 'model_performance_comparison.html'")
    except Exception as e:
        print(f"Error creating model comparison visualization: {e}")    
else:
    print("No actual Sales data in forecast period for evaluation.")
    final_model = 'Sales_pred_ensemble'

print_memory_usage()

#-----------------------------------------------------------
# 9.6. Time Series Forecast Visualization
#-----------------------------------------------------------
print("\nCreating forecast visualizations...")

try:
    vis_firms = np.random.choice(df_forecast['FirmID'].unique(), 
                                min(5, len(df_forecast['FirmID'].unique())), 
                                replace=False)

    for firm_id in vis_firms:
        try:
            firm_train = df_train[df_train['FirmID'] == firm_id].copy().sort_values('Date')
            firm_forecast = df_forecast[df_forecast['FirmID'] == firm_id].copy().sort_values('Date')

            fig = make_subplots(
                rows=2, cols=1, 
                shared_xaxes=True,
                subplot_titles=("Sales Forecast", "Shopper Traffic"),
                vertical_spacing=0.1,
                specs=[[{"secondary_y": False}], [{"secondary_y": True}]]
            )

            fig.add_trace(
                go.Scatter(
                    x=firm_train['Date'], 
                    y=firm_train['Sales'],
                    mode='lines+markers',
                    name='Actual Sales (Historical)',
                    line=dict(color='black')
                ),
                row=1, col=1
            )

            model_colors = {
                'Sales_pred_ols': 'blue',
                'Sales_pred_lgbm': 'red',
                'Sales_pred_prophet': 'green',
                'Sales_pred_ensemble': 'orange'
            }
            
            model_names = {
                'Sales_pred_ols': 'OLS Baseline',
                'Sales_pred_lgbm': 'LightGBM',
                'Sales_pred_prophet': 'Prophet',
                'Sales_pred_ensemble': 'Ensemble'
            }
            
            for model, color in model_colors.items():
                if model in firm_forecast.columns and firm_forecast[model].notna().any():
                    fig.add_trace(
                        go.Scatter(
                            x=firm_forecast['Date'], 
                            y=firm_forecast[model],
                            mode='lines+markers',
                            name=model_names[model],
                            line=dict(color=color)
                        ),
                        row=1, col=1
                    )

            if 'Sales' in firm_forecast.columns and firm_forecast['Sales'].notna().any():
                fig.add_trace(
                    go.Scatter(
                        x=firm_forecast['Date'], 
                        y=firm_forecast['Sales'],
                        mode='lines+markers',
                        name='Actual Sales (Forecast)',
                        line=dict(color='black', dash='dash')
                    ),
                    row=1, col=1
                )

            fig.add_trace(
                go.Scatter(
                    x=firm_train['Date'], 
                    y=firm_train['FirmDailyTraffic'],
                    mode='lines',
                    name='Training Traffic',
                    line=dict(color='darkblue')
                ),
                row=2, col=1
            )
            
            fig.add_trace(
                go.Scatter(
                    x=firm_forecast['Date'], 
                    y=firm_forecast['FirmDailyTraffic'],
                    mode='lines',
                    name='Forecast Traffic',
                    line=dict(color='darkblue', dash='dash')
                ),
                row=2, col=1
            )

            fig.add_vline(x=cutoff_date, line_dash="dash", line_color="gray", row=1, col=1)
            fig.add_vline(x=cutoff_date, line_dash="dash", line_color="gray", row=2, col=1)

            fig.update_layout(
                title_text=f"Sales Forecast and Traffic for Firm {firm_id}",
                height=800,
                width=1200,
                legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
                template="plotly_white"
            )

            fig.update_yaxes(title_text="Sales", row=1, col=1)
            fig.update_yaxes(title_text="Traffic", row=2, col=1)
            fig.update_xaxes(title_text="Date", row=2, col=1)

            fig.write_html(f"forecast_firm{firm_id}.html")
            print(f"Sales forecast visualization saved for firm {firm_id}")
        except Exception as e:
            print(f"Error creating visualization for firm {firm_id}: {e}")

except Exception as e:
    print(f"Error in forecast visualization: {e}")

print_memory_usage()

#-----------------------------------------------------------
# 9.7. Create Combined Forecast Dashboard
#-----------------------------------------------------------
print("\nCreating combined forecast dashboard...")

try:
    if 'vis_firms' not in locals():

        vis_firms = np.random.choice(df_forecast['FirmID'].unique(), 
                                   min(6, len(df_forecast['FirmID'].unique())), 
                                   replace=False)
    
    dashboard_firms = vis_firms[:min(6, len(vis_firms))]

    agg_data = []

    for firm_id in dashboard_firms:
        firm_train = df_train[df_train['FirmID'] == firm_id].copy()
        firm_forecast = df_forecast[df_forecast['FirmID'] == firm_id].copy()

        for _, row in firm_train.iterrows():
            agg_data.append({
                'FirmID': firm_id,
                'Date': row['Date'],
                'Period': 'Training',
                'Traffic': row['FirmDailyTraffic'],
                'Actual_Sales': row['Sales'],
                'OLS': None,
                'LightGBM': None,
                'Prophet': None,
                'Ensemble': None
            })

        for _, row in firm_forecast.iterrows():
            forecast_row = {
                'FirmID': firm_id,
                'Date': row['Date'],
                'Period': 'Forecast',
                'Traffic': row['FirmDailyTraffic'],
                'Actual_Sales': row['Sales'] if 'Sales' in row and not pd.isna(row['Sales']) else None,
                'OLS': row['Sales_pred_ols'] if 'Sales_pred_ols' in row else None,
                'LightGBM': row['Sales_pred_lgbm'] if 'Sales_pred_lgbm' in row else None,
                'Prophet': row['Sales_pred_prophet'] if 'Sales_pred_prophet' in row else None,
                'Ensemble': row['Sales_pred_ensemble'] if 'Sales_pred_ensemble' in row else None
            }
            agg_data.append(forecast_row)

    agg_df = pd.DataFrame(agg_data)

    fig = make_subplots(
        rows=len(dashboard_firms), 
        cols=1,
        subplot_titles=[f"Firm {firm_id} Forecast" for firm_id in dashboard_firms],
        vertical_spacing=0.1,
        shared_xaxes=True
    )

    row_idx = 1
    for firm_id in dashboard_firms:
        firm_data = agg_df[agg_df['FirmID'] == firm_id]

        train_data = firm_data[firm_data['Period'] == 'Training']
        fig.add_trace(
            go.Scatter(
                x=train_data['Date'],
                y=train_data['Actual_Sales'],
                mode='lines',
                name=f'Actual (Firm {firm_id})',
                line=dict(color='black', width=2),
                showlegend=(row_idx == 1) 
            ),
            row=row_idx, col=1
        )

        forecast_data = firm_data[firm_data['Period'] == 'Forecast']

        if not forecast_data['Actual_Sales'].isna().all():
            fig.add_trace(
                go.Scatter(
                    x=forecast_data['Date'],
                    y=forecast_data['Actual_Sales'],
                    mode='lines+markers',
                    name=f'Actual (Forecast)',
                    line=dict(color='black', width=2, dash='dash'),
                    showlegend=(row_idx == 1)
                ),
                row=row_idx, col=1
            )

        model_colors = {
            'OLS': 'blue',
            'LightGBM': 'red',
            'Prophet': 'green',
            'Ensemble': 'orange'
        }
        
        for model, color in model_colors.items():
            if model in forecast_data.columns and not forecast_data[model].isna().all():
                fig.add_trace(
                    go.Scatter(
                        x=forecast_data['Date'],
                        y=forecast_data[model],
                        mode='lines+markers',
                        name=model,
                        line=dict(color=color),
                        showlegend=(row_idx == 1)
                    ),
                    row=row_idx, col=1
                )

        fig.add_vline(x=cutoff_date, line_dash="dash", line_color="gray", row=row_idx, col=1)

        fig.update_yaxes(title_text="Sales", row=row_idx, col=1)
        
        row_idx += 1

    fig.update_layout(
        title_text="Multi-Firm Sales Forecasting Dashboard",
        height=300 * len(dashboard_firms),
        width=1200,
        template="plotly_white",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )

    fig.update_xaxes(title_text="Date", row=len(dashboard_firms), col=1)

    fig.write_html("sales_forecast_dashboard.html")
    print("Combined sales forecast dashboard saved to 'sales_forecast_dashboard.html'")

except Exception as e:
    print(f"Error creating combined forecast dashboard: {e}")

print_memory_usage()

#-----------------------------------------------------------
# 9.8. Create Final Forecast Output for Submission
#-----------------------------------------------------------
print("\nPreparing final forecast output for submission...")

try:
    if final_model not in df_forecast.columns:
        print(f"Warning: {final_model} not found in forecast dataframe. Falling back to ensemble.")
        final_model = 'Sales_pred_ensemble'

    final_coverage = (df_forecast[final_model].notna().sum() / len(df_forecast)) * 100
    print(f"Final model ({final_model}) has predictions for {final_coverage:.1f}% of forecast rows")

    if final_coverage < 100:
        print("Filling missing predictions using fallback models")

        df_forecast['Sales_pred_final'] = df_forecast[final_model]

        fallback_models = [m for m in ['Sales_pred_ensemble', 'Sales_pred_lgbm', 'Sales_pred_prophet', 'Sales_pred_ols'] 
                          if m in df_forecast.columns and m != final_model]

        for model in fallback_models:
            missing_mask = df_forecast['Sales_pred_final'].isna()
            if missing_mask.sum() > 0:
                df_forecast.loc[missing_mask, 'Sales_pred_final'] = df_forecast.loc[missing_mask, model]
                filled = missing_mask.sum() - df_forecast['Sales_pred_final'].isna().sum()
                if filled > 0:
                    print(f"  Filled {filled} missing values using {model}")
    else:
        df_forecast['Sales_pred_final'] = df_forecast[final_model]

    still_missing = df_forecast['Sales_pred_final'].isna().sum()
    if still_missing > 0:
        print(f"Warning: {still_missing} rows still have missing predictions.")
        print("Filling remaining missing values with the global median prediction")

        global_median = df_forecast['Sales_pred_final'].median()

        df_forecast['Sales_pred_final'] = df_forecast['Sales_pred_final'].fillna(global_median)

    sales_with_forecasts = df_sales.copy()

    forecast_data = df_forecast[['FirmID', 'Date', 'Sales_pred_final']]
    sales_with_forecasts = sales_with_forecasts.merge(
        forecast_data,
        on=['FirmID', 'Date'],
        how='left'
    )

    sales_with_forecasts['Sales'] = sales_with_forecasts['Sales'].fillna(
        sales_with_forecasts['Sales_pred_final']
    )

    sales_with_forecasts = sales_with_forecasts.drop(columns=['Sales_pred_final'])

    sales_with_forecasts.to_csv('Sales_with_forecasts.csv', index=False)
    print("Final forecasts saved to 'Sales_with_forecasts.csv'")

    df_forecast[['FirmID', 'Date', 'Sales_pred_final']].to_csv('Forecasts_only.csv', index=False)
    print("Forecast-only data saved to 'Forecasts_only.csv'")

except Exception as e:
    print(f"Error preparing final forecast output: {e}")

print("\nAll model training and visualization complete!")
print_memory_usage()


Preparing forecast dataframe...
Cutoff date for forecasting: 2021-12-03 00:00:00

Training LGBM model with simplified features...
Current memory usage: 308.57 MB

Implementing rolling forecast for LightGBM...
Processing firm 0.0 with rolling forecast...
Processing firm 1.0 with rolling forecast...
Processing firm 2.0 with rolling forecast...
Processing firm 3.0 with rolling forecast...
Processing firm 4.0 with rolling forecast...
Processing firm 5.0 with rolling forecast...
Processing firm 6.0 with rolling forecast...
Processing firm 7.0 with rolling forecast...
Processing firm 8.0 with rolling forecast...
Processing firm 9.0 with rolling forecast...
Processing firm 10.0 with rolling forecast...
Processing firm 11.0 with rolling forecast...
Processing firm 12.0 with rolling forecast...
Processing firm 13.0 with rolling forecast...
Processing firm 14.0 with rolling forecast...
Processing firm 15.0 with rolling forecast...
Processing firm 16.0 with rolling forecast...
Processing firm 17

11:53:04 - cmdstanpy - INFO - Chain [1] start processing


LightGBM actual vs predicted plot saved
Current memory usage: 315.40 MB

Training Prophet models for each firm with simplified regressors...

Training Prophet model for Firm 0.0...
  Using firm-specific Prophet parameters: {'changepoint_prior_scale': 0.001, 'seasonality_prior_scale': 0.01, 'holidays_prior_scale': 1.0, 'seasonality_mode': 'additive', 'changepoint_range': 0.8}


11:53:04 - cmdstanpy - INFO - Chain [1] done processing
11:53:04 - cmdstanpy - INFO - Chain [1] start processing


  Prophet model for Firm 0.0 successfully trained

Training Prophet model for Firm 1.0...
  Using default Prophet parameters: {'changepoint_prior_scale': 0.05, 'seasonality_prior_scale': 10.0, 'holidays_prior_scale': 10.0, 'seasonality_mode': 'additive', 'changepoint_range': 0.8}


...

  Prophet model for Firm 47.0 successfully trained

Training Prophet model for Firm 48.0...
  Using default Prophet parameters: {'changepoint_prior_scale': 0.05, 'seasonality_prior_scale': 10.0, 'holidays_prior_scale': 10.0, 'seasonality_mode': 'additive', 'changepoint_range': 0.8}


11:53:18 - cmdstanpy - INFO - Chain [1] done processing
11:53:18 - cmdstanpy - INFO - Chain [1] start processing


  Prophet model for Firm 48.0 successfully trained

Training Prophet model for Firm 49.0...
  Using default Prophet parameters: {'changepoint_prior_scale': 0.05, 'seasonality_prior_scale': 10.0, 'holidays_prior_scale': 10.0, 'seasonality_mode': 'additive', 'changepoint_range': 0.8}


11:53:18 - cmdstanpy - INFO - Chain [1] done processing


  Prophet model for Firm 49.0 successfully trained
Prophet components visualization saved to 'prophet_components.html'

Combining Prophet forecasts...
Prophet models trained for all firms with simplified regressors
Current memory usage: 316.16 MB

Integrating OLS model predictions...
Using firm-specific OLS models
OLS predictions available for 100.0% of forecast rows
Current memory usage: 316.16 MB

Creating ensemble forecast...
Models available for ensemble: ['Sales_pred_ols', 'Sales_pred_lgbm', 'Sales_pred_prophet']
Using weights from hyperparameter tuning for ensemble
Ensemble predictions available for 100.0% of forecast rows
Current memory usage: 316.16 MB

Comparing model performance...
No actual Sales data in forecast period for evaluation.
Current memory usage: 316.16 MB

Creating forecast visualizations...
Sales forecast visualization saved for firm 43.0
Sales forecast visualization saved for firm 46.0
Sales forecast visualization saved for firm 37.0
Sales forecast visualizatio